In [0]:
#LOGGING
import logging
class SWIFTParserLogger:
    def getSWIFTParserLogger(logger_level):
        logging.basicConfig(level=logger_level, format='%(asctime)s - %(levelname)s - %(message)s')
        logger = logging.getLogger('SWIFT_LOGGER')

        return logger

In [0]:
#SPARK 
from pyspark.sql import SparkSession

def get_dbutils(spark: SparkSession):
    try:
        import IPython
        return IPython.get_ipython().user_ns['dbutils']
    except (KeyError, ImportError):
        from pyspark.dbutils import DBUtils
    else:
        raise RuntimeError("dbutils is not available outside Databricks")

In [0]:
#New_MT_Names
import pandas as pd
import numpy as np
import re

##############################################################################################################
## Funcion 1: FUNCIÓN PARA RELLENAR LOS NAN POR OTRA COLUMNA DEL DATAFRAME SI ESTA COLUMNA EXISTE
## ------------------------------------------------------------
## Se aplica en la función funciones_complejas y cambiar_nombres_direccion
##############################################################################################################

def fillna_if_exist(df, target_column, source_columns, default=None):
    for source_column in source_columns:
        try:
            df[target_column] = df[target_column].fillna(df[source_column])
        except KeyError:
            pass  # La columna de origen no existe, continuar con la siguiente

    if default is not None:
        df[target_column].fillna(default, inplace=True)

##############################################################################################################
## Funcion 2: FUNCIÓN PARA RELLENAR LA COLUMNA NARRATIVE
## ------------------------------------------------------------
## La función extrae y devuelve una sección del texto a partir del primer marcador encontrado.
##############################################################################################################

def extract_copy_value(row):
    content = row['Message_Content']
    start_idx_11S = content.find(':11S:')
    start_idx_75 = content.find(':75:')
    start_idx_76 = content.find(':76:')
    start_idx_79 = content.find(':79:')
    indices = [idx for idx in [start_idx_11S, start_idx_75, start_idx_76, start_idx_79] if idx != -1]
    if not indices:
        return None
    if indices:
        min_index = min(indices)
        return content[min_index:]

##############################################################################################################
## Funcion 3: FUNCIÓN PARA DIFERENCIAR LAS COLUMNAS LOCALES CON LAS NO LOCALES EN LOS MT 760
## ------------------------------------------------------------
##############################################################################################################

def combinar_columnas(row,df):
    mensaje = row['Message_Content']
    Message_type = row['SWIFT_Message_Type']
    if Message_type== '760':
        sequence_15C = mensaje.count(":15C:")
        ocurrencias_32 = mensaje.count(":50:")
        if sequence_15C==1:
            if ocurrencias_32 == 2:
                applicant = row['Applicant_50']
                applicant_1 = row['Applicant_50_1']
                resultado_applicant = f"local:{applicant_1}"
                columna_combinada = f"{resultado_applicant} // {applicant}"
            else:
                applicant = row['Applicant_50']
                resultado_applicant = f"local:{applicant}"
                columna_combinada = f"{resultado_applicant}"
        else:
            if ocurrencias_32 == 1:
                applicant = row['Applicant_50']
                columna_combinada = f"//{applicant}"
            else:
                columna_combinada= np.nan
        ocurrencias_59 = mensaje.count(":59:")
        if sequence_15C==1:
            if ocurrencias_59 == 2:
                Beneficiary_name = row['Beneficiary_59_Name_and_Address']
                Beneficiary_name_1 = row['Beneficiary_59_1_Name_and_Address']
                resultado_applicant = f"local:{Beneficiary_name_1}"
                Beneficiary_name_combinada = f"{resultado_applicant} // {Beneficiary_name}"
                if 'Beneficiary_59_Account' in df.columns and 'Beneficiary_59_1_Account' in df.columns:
                    Beneficiary_account = row['Beneficiary_59_Account']
                    Beneficiary_account_1 = row['Beneficiary_59_1_Account']
                    resultado_applicant = f"local:{Beneficiary_account_1}"
                    Beneficiary_account_combinada = f"{resultado_applicant} // {Beneficiary_account}"
                if 'Beneficiary_59_1_Account' in df.columns and 'Beneficiary_59_Account' not in df.columns:
                    Beneficiary_account_1 = row['Beneficiary_59_1_Account']
                    resultado_applicant = f"local:{Beneficiary_account_1}"
                    Beneficiary_account_combinada = f"{resultado_applicant}"
                if 'Beneficiary_59_Account' in df.columns and 'Beneficiary_59_1_Account' not in df.columns:
                    Beneficiary_account = row['Beneficiary_59_Account']
                    resultado_applicant = f"//{Beneficiary_account}"
                    Beneficiary_account_combinada = f"{resultado_applicant}"
                else:
                    Beneficiary_account_combinada = np.nan
            else:
                Beneficiary_name = row['Beneficiary_59_Name_and_Address']
                resultado_applicant = f"local:{Beneficiary_name}"
                Beneficiary_name_combinada = f"{resultado_applicant}"
                if 'Beneficiary_59_Account' in df.columns:
                    Beneficiary_account = row['Beneficiary_59_Account']
                    resultado_applicant = f"local:{Beneficiary_account}"
                    Beneficiary_account_combinada = f"{resultado_applicant}"
                else:
                    Beneficiary_account_combinada= np.nan
        else:
            if ocurrencias_59 == 1:
                Beneficiary_name = row['Beneficiary_59_Name_and_Address']
                Beneficiary_name_combinada = f"//:{Beneficiary_name}"
                if 'Beneficiary_59_Account' in df.columns:
                    Beneficiary_account = row['Beneficiary_59_Account']
                    Beneficiary_account_combinada = f"//:{Beneficiary_account}"
                else:
                    Beneficiary_account_combinada= np.nan
            else:
                Beneficiary_account_combinada= np.nan
                Beneficiary_name_combinada= np.nan
        ocurrencias_45L = mensaje.count(":45L:")
        if sequence_15C==1:
            if ocurrencias_45L == 2:
                Transaction_Details = row['Underlying_Transaction_Details_45L']
                Transaction_Details_1 = row['Underlying_Transaction_Details_45L_1']
                resultado_applicant = f"local:{Transaction_Details_1}"
                Transaction_Details_combinada = f"{resultado_applicant} // {Transaction_Details}"
            else:
                Transaction_Details = row['Underlying_Transaction_Details_45L']
                resultado_applicant = f"local:{Transaction_Details}"
                Transaction_Details_combinada = f"{resultado_applicant}"
        else:
            if ocurrencias_45L == 1:
                Transaction_Details = row['Underlying_Transaction_Details_45L']
                Transaction_Details_combinada = f"//{Transaction_Details}"
            else:
                Transaction_Details_combinada= np.nan
    else:
        columna_combinada=np.nan
        Beneficiary_name_combinada=np.nan
        Beneficiary_account_combinada=np.nan
        Transaction_Details_combinada=np.nan
    return columna_combinada, Beneficiary_name_combinada, Beneficiary_account_combinada,Transaction_Details_combinada

def funciones_complejas(df):
    if 'Ordering_Customer_50F_Party_Identifier' in df.columns:
        df['Ordering_Customer_50F_Party_Identifier'] = df['Ordering_Customer_50F_Party_Identifier'].str.replace('/','')
        df['Ordering_Customer_50F_Party_Identifier_Account'] = df['Ordering_Customer_50F_Party_Identifier'].apply(lambda x: x if (isinstance(x, str) and x[:2].isalpha() and x[2:].isdigit() and len(x) >= 16) else None)
        df['Ordering_Customer_50F_Party_Identifier_Id'] = df['Ordering_Customer_50F_Party_Identifier'].mask(df['Ordering_Customer_50F_Party_Identifier_Account'].notna())

    lista_Ordering_Customer_50H_10_Account=[]
    lista_Ordering_Customer_50G_10_Account=[]
    for i in range(10,0,-1):
        lista_Ordering_Customer_50H_10_Account.append(f'Ordering_Customer_50H_{i}_Account')
        lista_Ordering_Customer_50G_10_Account.append(f'Ordering_Customer_50G_{i}_Account')
    lista_Ordering_Customer_50H_10_Account=lista_Ordering_Customer_50H_10_Account+lista_Ordering_Customer_50G_10_Account
    lista_Ordering_Customer_50H_10_Account.extend(['Ordering_Customer_50A_Account', 'Ordering_Customer_50K_Account','Ordering_Customer_50F_Party_Identifier_Account','Ordering_Customer_50H_Account','Ordering_Customer_50G_Account'])
    df['Ordering_Customer_Account'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50H_10_Account' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Account',lista_Ordering_Customer_50H_10_Account)
    else:
        df['Ordering_Customer_50H_10_Account'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Account',lista_Ordering_Customer_50H_10_Account)

    df['Ordering_Customer_Id'] = np.nan  # Inicializar con NaN
    lista_Ordering_Customer_50G_Identifier_Code=[]
    for i in range(10,0,-1):
        lista_Ordering_Customer_50G_Identifier_Code.append(f'Ordering_Customer_50G_{i}_Identifier_Code')
    lista_Ordering_Customer_50G_Identifier_Code.extend(['Ordering_Customer_50A_Identifier_Code', 'Ordering_Customer_50F_Party_Identifier_Id','Ordering_Customer_50G_Identifier_Code'])
    if 'Ordering_Customer_50G_10_Identifier_Code' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Id',lista_Ordering_Customer_50G_Identifier_Code)
    else:
        df['Ordering_Customer_50G_10_Identifier_Code'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Id',lista_Ordering_Customer_50G_Identifier_Code)

    if 'Ordering_Customer_50F_Number_Name_and_Address' in df.columns:
        patron = r'^1/([^\n]+)'
        df['Ordering_Customer_50F_Number_Name_and_Address_Name'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(patron,flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Address_Name'] = df['Ordering_Customer_50F_Number_Name_and_Address_Name'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Address_Name'] = df['Ordering_Customer_50F_Number_Name_and_Address_Name'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Address_Name'] = df['Ordering_Customer_50F_Number_Name_and_Address_Name'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_Address_Name']=df['Ordering_Customer_50F_Number_Name_and_Address_Name'].replace('nan', np.nan)
    for i in range(1,11):
        if f'Ordering_Customer_50F_{i}_Number_Name_and_Address' in df.columns:
            patron = r'^1/([^\n]+)'
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(patron,flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name']=df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name'].replace('nan', np.nan)

    df['Ordering_Customer_Name'] = np.nan
    lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name=[]
    Lista_Intructing_Party_50L=[]
    for i in range(10, 0, -1):
            lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name.append(f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Name')
            lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name.append(f'Ordering_Customer_50H_{i}_Name_and_Address')
            Lista_Intructing_Party_50L.append(f'Intructing_Party_50L_{i}')
    lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name=lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name+Lista_Intructing_Party_50L
    lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name.extend(['Ordering_Customer_50F_Number_Name_and_Address_Name','Ordering_Customer_50H_Name_and_Address','Ordering_Customer_50K_Name_and_Address','Intructing_Party_50L'])
    if 'Ordering_Customer_50F_10_Number_Name_and_Address_Name' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Name', lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_Address_Name'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Name',lista_Ordering_Customer_50F_6_Number_Name_and_Address_Name)

    if 'Ordering_Customer_50F_Number_Name_and_Address' in df.columns:
        df['Ordering_Customer_50F_Number_Name_and_Address_Address'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^2/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address'].replace('nan', np.nan, inplace=True)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'].replace('nan', np.nan, inplace=True)
        df['Ordering_Customer_50F_Number_Name_and_Address_Address'] = df['Ordering_Customer_50F_Number_Name_and_Address_Address'].fillna(df['Ordering_Customer_50F_Number_Name_and_Address_Address_1'])
    for i in range(1,11):
        if f'Ordering_Customer_50F_{i}_Number_Name_and_Address' in df.columns:
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^2/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'].replace('nan', np.nan, inplace=True)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address'].fillna(df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address_1'])
    df['Ordering_Customer_Address'] = np.nan  # Inicializar con NaN
    lista_Ordering_Customer_50F_10_Number_Name_and_Address_Address=[]
    for i in range(10,0,-1):
        lista_Ordering_Customer_50F_10_Number_Name_and_Address_Address.append(f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Address')
    lista_Ordering_Customer_50F_10_Number_Name_and_Address_Address.extend(['Ordering_Customer_50F_Number_Name_and_Address_Address', 'Ordering_Customer_50K_Name_and_Address'])
    if 'Ordering_Customer_50F_10_Number_Name_and_Address_Address' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Address',lista_Ordering_Customer_50F_10_Number_Name_and_Address_Address)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_Address_Address'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Address',lista_Ordering_Customer_50F_10_Number_Name_and_Address_Address)

    if 'Ordering_Customer_50F_Number_Name_and_Address' in df.columns:
        df['Ordering_Customer_50F_Number_Name_and_Address_Country'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Address_Country'] = df['Ordering_Customer_50F_Number_Name_and_Address_Country'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Address_Country'] = df['Ordering_Customer_50F_Number_Name_and_Address_Country'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Address_Country'] = df['Ordering_Customer_50F_Number_Name_and_Address_Country'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^4/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^5/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'] = df['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^6/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_ID_Number'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_ID_Number'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_ID_Number'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^7/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'] = df['Ordering_Customer_50F_Number_Name_and_National_ID_Number'].replace('', np.nan)
        df['Ordering_Customer_50F_Number_Name_and_Additional_Information'] = df['Ordering_Customer_50F_Number_Name_and_Address'].astype(str).str.findall(r'^8/([^\n]+)',flags=re.MULTILINE)
        df['Ordering_Customer_50F_Number_Name_and_Additional_Information'] = df['Ordering_Customer_50F_Number_Name_and_Additional_Information'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Ordering_Customer_50F_Number_Name_and_Additional_Information'] = df['Ordering_Customer_50F_Number_Name_and_Additional_Information'].astype(str)
        df['Ordering_Customer_50F_Number_Name_and_Additional_Information'] = df['Ordering_Customer_50F_Number_Name_and_Additional_Information'].replace('', np.nan)
    for i in range(1,11):
        if f'Ordering_Customer_50F_{i}_Number_Name_and_Address' in df.columns:
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^4/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^5/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^6/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^7/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number'].replace('', np.nan)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^8/([^\n]+)',flags=re.MULTILINE)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'].astype(str)
            df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'] = df[f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information'].replace('', np.nan)
    lista_Ordering_Customer_50F_10_Number_Name_and_Address_Country=[]
    lista_Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth=[]
    lista_Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth=[]
    lista_Ordering_Customer_50F_10_Number_Name_and_ID_Number=[]
    lista_Ordering_Customer_50F_10_Number_Name_and_National_ID_Number=[]
    lista_Ordering_Customer_50F_10_Number_Name_and_Additional_Information=[]
    for i in range(10,0,-1):
        lista_Ordering_Customer_50F_10_Number_Name_and_Address_Country.append(f'Ordering_Customer_50F_{i}_Number_Name_and_Address_Country')
        lista_Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth.append(f'Ordering_Customer_50F_{i}_Number_Name_and_Date_Of_Birth')
        lista_Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth.append(f'Ordering_Customer_50F_{i}_Number_Name_and_Place_Of_Birth')
        lista_Ordering_Customer_50F_10_Number_Name_and_ID_Number.append(f'Ordering_Customer_50F_{i}_Number_Name_and_ID_Number')
        lista_Ordering_Customer_50F_10_Number_Name_and_National_ID_Number.append(f'Ordering_Customer_50F_{i}_Number_Name_and_National_ID_Number')
        lista_Ordering_Customer_50F_10_Number_Name_and_Additional_Information.append(f'Ordering_Customer_50F_{i}_Number_Name_and_Additional_Information')
    lista_Ordering_Customer_50F_10_Number_Name_and_Address_Country.extend(['Ordering_Customer_50F_Number_Name_and_Country'])
    lista_Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth.extend(['Ordering_Customer_50F_Number_Name_and_Date_Of_Birth'])
    lista_Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth.extend(['Ordering_Customer_50F_Number_Name_and_Place_Of_Birth'])
    lista_Ordering_Customer_50F_10_Number_Name_and_ID_Number.extend(['Ordering_Customer_50F_Number_Name_and_ID_Number'])
    lista_Ordering_Customer_50F_10_Number_Name_and_National_ID_Number.extend(['Ordering_Customer_50F_Number_Name_and_National_ID_Number'])
    lista_Ordering_Customer_50F_10_Number_Name_and_Additional_Information.extend(['Ordering_Customer_50F_Number_Name_and_Additional_Information'])
    df['Ordering_Customer_Address_Country'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50F_10_Number_Name_and_Address_Country' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Address_Country',lista_Ordering_Customer_50F_10_Number_Name_and_Address_Country)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_Address_Country'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Address_Country',lista_Ordering_Customer_50F_10_Number_Name_and_Address_Country)
    df['Ordering_Customer_Date_Of_Birth'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Date_Of_Birth',lista_Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Date_Of_Birth',lista_Ordering_Customer_50F_10_Number_Name_and_Date_Of_Birth)
    df['Ordering_Customer_Place_Of_Birth'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Place_Of_Birth',lista_Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Place_Of_Birth',lista_Ordering_Customer_50F_10_Number_Name_and_Place_Of_Birth)
    df['Ordering_Customer_ID_Number'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50F_10_Number_Name_and_ID_Number' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_ID_Number',lista_Ordering_Customer_50F_10_Number_Name_and_ID_Number)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_ID_Number'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_ID_Number',lista_Ordering_Customer_50F_10_Number_Name_and_ID_Number)
    df['Ordering_Customer_National_ID_Number'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50F_10_Number_Name_and_National_ID_Number' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_National_ID_Number',lista_Ordering_Customer_50F_10_Number_Name_and_National_ID_Number)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_National_ID_Number'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_National_ID_Number',lista_Ordering_Customer_50F_10_Number_Name_and_National_ID_Number)
    df['Ordering_Customer_Additional_Information'] = np.nan  # Inicializar con NaN
    if 'Ordering_Customer_50F_10_Number_Name_and_Additional_Information' in df.columns:
        fillna_if_exist(df, 'Ordering_Customer_Additional_Information',lista_Ordering_Customer_50F_10_Number_Name_and_Additional_Information)
    else:
        df['Ordering_Customer_50F_10_Number_Name_and_Additional_Information'] = np.nan
        fillna_if_exist(df, 'Ordering_Customer_Additional_Information',lista_Ordering_Customer_50F_10_Number_Name_and_Additional_Information)

    if 'Beneficiary_Customer_59F_Number_Name_and_Address' in df.columns:
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'] = df['Beneficiary_Customer_59F_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE).apply(lambda x: x[0] if len(x) > 0 else np.nan)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'].astype(str)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'].replace('', np.nan)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Country'].replace('nan', np.nan, inplace=True)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address'].astype(str).str.findall(r'^1/([^\n]+)',flags=re.MULTILINE).apply(lambda x: x[0] if len(x) > 0 else np.nan)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'].astype(str)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'].replace('', np.nan)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1'].replace('nan', np.nan, inplace=True)
    if 'Beneficiary_59F_Number_Name_and_Address' in df.columns:
        df['Beneficiary_59F_Number_Name_and_Address_Country'] = df['Beneficiary_59F_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE).apply(lambda x: x[0] if len(x) > 0 else np.nan)
        df['Beneficiary_59F_Number_Name_and_Address_Country'] = df['Beneficiary_59F_Number_Name_and_Address_Country'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_59F_Number_Name_and_Address_Country'] = df['Beneficiary_59F_Number_Name_and_Address_Country'].astype(str)
        df['Beneficiary_59F_Number_Name_and_Address_Country'] = df['Beneficiary_59F_Number_Name_and_Address_Country'].replace('', np.nan)
        df['Beneficiary_59F_Number_Name_and_Address_Country'].replace('nan', np.nan, inplace=True)
        df['Beneficiary_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_59F_Number_Name_and_Address'].astype(str).str.findall(r'^1/([^\n]+)',flags=re.MULTILINE).apply(lambda x: x[0] if len(x) > 0 else np.nan)
        df['Beneficiary_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_59F_Number_Name_and_Address_Name_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_59F_Number_Name_and_Address_Name_1'].astype(str)
        df['Beneficiary_59F_Number_Name_and_Address_Name_1'] = df['Beneficiary_59F_Number_Name_and_Address_Name_1'].replace('', np.nan)
        df['Beneficiary_59F_Number_Name_and_Address_Name_1'].replace('nan', np.nan, inplace=True)
    if f'Beneficiary_59F_{i}_Number_Name_and_Address' in df.columns:
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^3/([^\n]+)',flags=re.MULTILINE).apply(lambda x: x[0] if len(x) > 0 else np.nan)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'].astype(str)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'].replace('', np.nan)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Country'].replace('nan', np.nan, inplace=True)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address'].astype(str).str.findall(r'^1/([^\n]+)',flags=re.MULTILINE).apply(lambda x: x[0] if len(x) > 0 else np.nan)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'].astype(str)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'].replace('', np.nan)
        df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1'].replace('nan', np.nan, inplace=True)
    lista_Beneficiary_59F_10_Number_Name_and_Address_Country=[]
    for i in range(10,0,-1):
        lista_Beneficiary_59F_10_Number_Name_and_Address_Country.append(f'lista_Beneficiary_59F_{i}_Number_Name_and_Address_Country')
    lista_Beneficiary_59F_10_Number_Name_and_Address_Country.extend(['Beneficiary_Customer_59F_Number_Name_and_Address_Country', 'Beneficiary_59F_Number_Name_and_Address_Country'])
    df["Beneficiary_Address_Country"] = np.nan  # Inicializar con NaN
    if 'Beneficiary_59F_10_Number_Name_and_Address_Country' in df.columns:
        fillna_if_exist(df, 'Beneficiary_Address_Country',lista_Beneficiary_59F_10_Number_Name_and_Address_Country)
    else:
        df["Beneficiary_59F_10_Number_Name_and_Address_Country"] = np.nan
        fillna_if_exist(df, 'Beneficiary_Address_Country',lista_Beneficiary_59F_10_Number_Name_and_Address_Country)

    if 'Beneficiary_Customer_59F_Number_Name_and_Address' in df.columns:
        patron_beneficiary_address = r'^2/(.*)'
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'] = df['Beneficiary_Customer_59F_Number_Name_and_Address'].astype(str).apply(lambda x: re.findall(patron_beneficiary_address, x, re.DOTALL))
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        patron_beneficiary_address_1 = r'^3/(.*)'
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address'].astype(str).apply(lambda x: re.findall(patron_beneficiary_address_1, x,re.DOTALL))
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'].astype(str)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'].replace('', np.nan)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'].astype(str)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'].replace('', np.nan)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'].replace('nan', np.nan, inplace=True)
        df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'] = df['Beneficiary_Customer_59F_Number_Name_and_Address_Address'].fillna(df['Beneficiary_Customer_59F_Number_Name_and_Address_Address_1'])
    if 'Beneficiary_59F_Number_Name_and_Address' in df.columns:
        patron_beneficiary_address = r'^2/(.*)'
        df['Beneficiary_59F_Number_Name_and_Address_Address'] = df['Beneficiary_59F_Number_Name_and_Address'].astype(str).apply(lambda x: re.findall(patron_beneficiary_address, x, re.DOTALL))
        df['Beneficiary_59F_Number_Name_and_Address_Address'] = df['Beneficiary_59F_Number_Name_and_Address_Address'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        patron_beneficiary_address_1 = r'^3/(.*)'
        df['Beneficiary_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_59F_Number_Name_and_Address'].astype(str).apply(lambda x: re.findall(patron_beneficiary_address_1, x,re.DOTALL))
        df['Beneficiary_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_59F_Number_Name_and_Address_Address_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_59F_Number_Name_and_Address_Address_1'].astype(str)
        df['Beneficiary_59F_Number_Name_and_Address_Address_1'] = df['Beneficiary_59F_Number_Name_and_Address_Address_1'].replace('', np.nan)
        df['Beneficiary_59F_Number_Name_and_Address_Address'] = df['Beneficiary_59F_Number_Name_and_Address_Address'].astype(str)
        df['Beneficiary_59F_Number_Name_and_Address_Address'] = df['Beneficiary_59F_Number_Name_and_Address_Address'].replace('', np.nan)
        df['Beneficiary_59F_Number_Name_and_Address_Address'].replace('nan', np.nan, inplace=True)
        df['Beneficiary_59F_Number_Name_and_Address_Address'] = df['Beneficiary_59F_Number_Name_and_Address_Address'].fillna(df['Beneficiary_59F_Number_Name_and_Address_Address_1'])
    for i in range(1,11):
        if f'Beneficiary_59F_{i}_Number_Name_and_Address' in df.columns:
            patron_beneficiary_address = r'^2/(.*)'
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address'].astype(str).apply(lambda x: re.findall(patron_beneficiary_address, x, re.DOTALL))
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            patron_beneficiary_address_1 = r'^3/(.*)'
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address'].astype(str).apply(lambda x: re.findall(patron_beneficiary_address_1, x,re.DOTALL))
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'].astype(str)
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'].replace('', np.nan)
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'].astype(str)
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'].replace('', np.nan)
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'].replace('nan', np.nan, inplace=True)
            df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'] = df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address'].fillna(df[f'Beneficiary_59F_{i}_Number_Name_and_Address_Address_1'])

    lista_Beneficiary_59F_10_Number_Name_and_Address_Address=[]
    for i in range(10,0,-1):
        lista_Beneficiary_59F_10_Number_Name_and_Address_Address.append(f'Beneficiary_59F_{i}_Number_Name_and_Address_Address')
    lista_Beneficiary_59F_10_Number_Name_and_Address_Address.extend(['Beneficiary_Customer_59_Name_and_Address', 'Beneficiary_Customer_59F_Number_Name_and_Address_Address','Beneficiary_59_Name_and_Address','Second_Beneficiary_59_Name_and_Address'])
    df["Beneficiary_Address"] = np.nan  # Inicializar con NaN
    if 'Beneficiary_59F_10_Number_Name_and_Address_Address' in df.columns:
        fillna_if_exist(df, 'Beneficiary_Address',lista_Beneficiary_59F_10_Number_Name_and_Address_Address)
    else:
        df["Beneficiary_59F_10_Number_Name_and_Address_Address"] = np.nan
        fillna_if_exist(df, 'Beneficiary_Address',lista_Beneficiary_59F_10_Number_Name_and_Address_Address)

    df['Applicant_50_combinada'] = df.apply(lambda row: combinar_columnas(row,df)[0], axis=1)
    if 'Applicant_50_combinada' in df.columns:
        patron_Applicant_50_combinada = r'local:(.*?)(?://|$)'
        df['Local_Applicant_Name'] = df['Applicant_50_combinada'].astype(str).apply(lambda x: re.findall(patron_Applicant_50_combinada, x, re.DOTALL))
        df['Local_Applicant_Name'] = df['Local_Applicant_Name'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        patron_Applicant_50_combinada_1 = r'//(.*)$'
        df['Applicant_Name_Applicant_50_combinada'] = df['Applicant_50_combinada'].astype(str).apply(lambda x: re.findall(patron_Applicant_50_combinada_1, x, re.DOTALL))
        df['Applicant_Name_Applicant_50_combinada'] = df['Applicant_Name_Applicant_50_combinada'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Applicant_Name_Applicant_50_combinada'] = df['Applicant_Name_Applicant_50_combinada'].astype(str)
        df['Applicant_Name_Applicant_50_combinada'] = df['Applicant_Name_Applicant_50_combinada'].replace('', np.nan)
        df['Applicant_Name_Applicant_50_combinada'].replace('nan', np.nan, inplace=True)

    df["Applicant_Name"] = np.nan  # Inicializar con NaN
    if 'Applicant_Name_Applicant_50_combinada' in df.columns:
        fillna_if_exist(df, 'Applicant_Name',['Applicant_Name_Applicant_50_combinada','First_Beneficiary_50','Changed_Applicant_Details_50'])
    else:
        df["Applicant_Name_Applicant_50_combinada"] = np.nan
        fillna_if_exist(df, 'Applicant_Name',['Applicant_Name_Applicant_50_combinada','First_Beneficiary_50','Changed_Applicant_Details_50'])

    if 'Applicant_Name' in df.columns and 'Applicant_50' in df.columns:
        mask = df['SWIFT_Message_Type'] != 760
        df.loc[mask, 'Applicant_Name'] = df.loc[mask, 'Applicant_Name'].fillna(df.loc[mask, 'Applicant_50'])

    df['Beneficiary_59_Name_and_Address_combinada'] = df.apply(lambda row: combinar_columnas(row,df)[1], axis=1)
    if 'Beneficiary_59_Name_and_Address_combinada' in df.columns:
        patron_Applicant_50_combinada = r'local:(.*?)(?://|$)'
        df['Local_Beneficiary_Name'] = df['Beneficiary_59_Name_and_Address_combinada'].astype(str).apply(lambda x: re.findall(patron_Applicant_50_combinada, x, re.DOTALL))
        df['Local_Beneficiary_Name'] = df['Local_Beneficiary_Name'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        patron_Applicant_50_combinada_1 = r'//(.*)$'
        df['Beneficiary_Name_nolocal'] = df['Beneficiary_59_Name_and_Address_combinada'].astype(str).apply(lambda x: re.findall(patron_Applicant_50_combinada_1, x, re.DOTALL))
        df['Beneficiary_Name_nolocal'] = df['Beneficiary_Name_nolocal'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_Name_nolocal'] = df['Beneficiary_Name_nolocal'].astype(str)
        df['Beneficiary_Name_nolocal'] = df['Beneficiary_Name_nolocal'].replace('', np.nan)
        df['Beneficiary_Name_nolocal'].replace('nan', np.nan, inplace=True)

    lista_Beneficiary_59_10_Name_and_Address=[]
    lista_Beneficiary_59F_10_Number_Name_and_Address=[]
    for i in range(10,0,-1):
        lista_Beneficiary_59_10_Name_and_Address.append(f'Beneficiary_59_{i}_Name_and_Address')
        lista_Beneficiary_59F_10_Number_Name_and_Address.append(f'Beneficiary_59F_{i}_Number_Name_and_Address_Name_1')
    lista_Beneficiary_59_10_Name_and_Address=lista_Beneficiary_59_10_Name_and_Address+lista_Beneficiary_59F_10_Number_Name_and_Address
    lista_Beneficiary_59_10_Name_and_Address.extend(['Beneficiary_Customer_59F_Number_Name_and_Address_Name_1','Beneficiary_Customer_59_Name_and_Address','Beneficiary_Name_nolocal','Beneficiary_59_Name_and_Address','Beneficiary_Customer_59F_Number_Name_and_Address_Name_1','Second_Beneficiary_59_Name_and_Address'])
    df["Beneficiary_Name"] = np.nan
    if 'Beneficiary_59_10_Name_and_Address' in df.columns:
        fillna_if_exist(df, 'Beneficiary_Name',lista_Beneficiary_59_10_Name_and_Address)
    else:
        df["Beneficiary_59_10_Name_and_Address"] = np.nan
        fillna_if_exist(df, 'Beneficiary_Name',lista_Beneficiary_59_10_Name_and_Address)

    df['Beneficiary_59_Account_combinada'] = df.apply(lambda row: combinar_columnas(row,df)[2], axis=1)
    if 'Beneficiary_59_Account_combinada' in df.columns:
        patron_Applicant_50_combinada = r'local:(.*?)(?://|$)'
        df['Local_Beneficiary_Account'] = df['Beneficiary_59_Account_combinada'].astype(str).apply(lambda x: re.findall(patron_Applicant_50_combinada, x, re.DOTALL))
        df['Local_Beneficiary_Account'] = df['Local_Beneficiary_Account'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        patron_Applicant_50_combinada_1 = r'//(.*)$'
        df['Beneficiary_59_Account_nueva'] = df['Beneficiary_59_Account_combinada'].astype(str).apply(lambda x: re.findall(patron_Applicant_50_combinada_1, x, re.DOTALL))
        df['Beneficiary_59_Account_nueva'] = df['Beneficiary_59_Account_nueva'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Beneficiary_59_Account_nueva'] = df['Beneficiary_59_Account_nueva'].astype(str)
        df['Beneficiary_59_Account_nueva'] = df['Beneficiary_59_Account_nueva'].replace('', np.nan)
        df['Beneficiary_59_Account_nueva'].replace('nan', np.nan, inplace=True)


    df['Underlying_Transaction_Details_45L_combinada'] = df.apply(lambda row: combinar_columnas(row,df)[3], axis=1)
    if 'Underlying_Transaction_Details_45L_combinada' in df.columns:
        patron = r'local:(.*?)(?://|$)'
        df['Local_Transaction_Details'] = df['Underlying_Transaction_Details_45L_combinada'].astype(str).apply(lambda x: re.findall(patron, x, re.DOTALL))
        df['Local_Transaction_Details'] = df['Local_Transaction_Details'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        patron_1 = r'//(.*)$'
        df['Transaction_Details'] = df['Underlying_Transaction_Details_45L_combinada'].astype(str).apply(lambda x: re.findall(patron_1, x, re.DOTALL))
        df['Transaction_Details'] = df['Transaction_Details'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
        df['Transaction_Details'] = df['Transaction_Details'].astype(str)
        df['Transaction_Details'] = df['Transaction_Details'].replace('', np.nan)
        df['Transaction_Details'].replace('nan', np.nan, inplace=True)

    return df

def cambiar_nombres_direccion(df):
    # Copiar el DataFrame para evitar modificar el original
    nombres_nuevos = {
    'SWIFT_Message_Type': 'Message_type',
    'Input_Output': 'Direction',
    'Currency/Instructed_Amount_33B_Currency':'Instructed_Currency',
    'Currency/Instructed_Amount_33B_Amount':'Instructed_Amount',
    'Beneficiary_Customer_59A_Identifier_Code': 'Beneficiary_Id',
    'Date_and_Place_of_Expiry_31D_Place':'Expiry_Place',
    'Applicant_Bank_51D_Name_and_Address':'Applicant_Bank_Name_and_Address',
    'Obligor/Instructing_Party_51':'Instructing_Party',
    'Place_of_Taking_in_Charge/Dispatch_from/Place_of_Receipt_44A':'Place_Dispatch_Receipt',
    'Port_of_Loading/Airport_of_Departure_44E':'PortLoading_AirportDeparture',
    'Place_of_Final_Destination/For_Transportation_to/Place_of_Delivery_44B':'Place_Destination_Delivery',
    'Latest_Date_of_Shipment_44C':'Latest_Date_Shipment',
    'Requested_Confirmation_Party_58D_Name_and_Address':'Requested_Confirmation_Party_Name',
    'Undertaking_Terms_and_Conditions_77U':'Undertaking_Terms',
    'Requested_Local_Undertaking_Terms_and_Conditions_77L':'Local_Undertaking_Terms',
    'Drawee_42D_Name_and_Address':'Drawee_Name',
    'Reimbursing_Bank_53D_Name_and_Address':'Reimbursing_Bank_Name',
    'Advising_Bank_56D_Name_and_Address': 'Advising_Bank_Name',
    'Type_of_Undertaking_22K_Code':'Local_Type_Purpose',
    'Beneficiary_Institution_58D_Name_and_Address':'Beneficiary_Institution_Name'
    }

    df_copiado = df.copy()
    df_copiado.rename(columns=nombres_nuevos, inplace=True)
    df_copiado = df_copiado.replace({None: np.nan})
    df_copiado['LTAddress'] = df_copiado['LTAddress'].astype(str)
    if 'Destination_Address' in df_copiado.columns:
        df_copiado['Destination_Address'] = df_copiado['Destination_Address'].astype(str)
    if 'UETR' in df_copiado.columns:
        df_copiado['UETR']= df_copiado['UETR'].str.replace('121:','')

    # Cambiar los nombres de la columna 'Direction' según la lógica
    df_copiado['Direction'] = df_copiado['Direction'].apply(lambda x: 'Outgoing' if x == 'I' else 'Incoming' if x == 'O' else x)

    df_copiado['Sender_InstructingAgent_Requestor'] = df_copiado.apply(
    lambda row: str(row['LTAddress'])[:8] if row['Direction'] == 'Outgoing' and pd.notna(row['LTAddress']) else str(row['MIR'])[6:14] if pd.notna(row['MIR']) else None,
    axis=1
    )

    df_copiado['Receiver_InstructedAgent_Responder'] = df_copiado.apply(
    lambda row: str(row['LTAddress'])[:8] if row['Direction'] == 'Incoming' and pd.notna(row['LTAddress']) else str(row['Destination_Address'])[:8] if pd.notna(row['Destination_Address']) else None,
    axis=1
    )

    df_copiado['OwnBIC8'] = df_copiado.apply(
        lambda row: row['Sender_InstructingAgent_Requestor'] if row['Direction'] == 'Outgoing' else row['Receiver_InstructedAgent_Responder'],
        axis=1
    )

    df_copiado['Own_Country'] = df_copiado['OwnBIC8'].str[4:6]

    lista_Account_Servicing_Institution_52A_10_Identifier_Code=[]
    lista_Account_Servicing_Institution_52C_10=[]
    for i in range(10,0,-1):
        lista_Account_Servicing_Institution_52A_10_Identifier_Code.append(f'Account_Servicing_Institution_52A_{i}_Identifier_Code')
        lista_Account_Servicing_Institution_52C_10.append(f'Account_Servicing_Institution_52C_{i}')
    lista_Account_Servicing_Institution_52A_10_Identifier_Code=lista_Account_Servicing_Institution_52A_10_Identifier_Code+lista_Account_Servicing_Institution_52C_10
    lista_Account_Servicing_Institution_52A_10_Identifier_Code.extend(['Ordering_Institution_52A_1_Identifier_Code','Ordering_Institution_52A_Identifier_Code','Ordering_Institution_52D_1_Party_Identifier','Ordering_Institution_52D_Party_Identifier','Account_Servicing_Institution_52A_Identifier_Code','Account_Servicing_Institution_52C','Sender_InstructingAgent_Requestor'])
    df_copiado['Ordering_Institution'] = np.nan  # Inicializar con NaN
    if 'Account_Servicing_Institution_52A_10_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, 'Ordering_Institution',lista_Account_Servicing_Institution_52A_10_Identifier_Code)
    else:
        df_copiado['Account_Servicing_Institution_52A_10_Identifier_Code'] = np.nan
        fillna_if_exist(df_copiado, 'Ordering_Institution',lista_Account_Servicing_Institution_52A_10_Identifier_Code)

    df_copiado["Beneficiary_Institution"] = np.nan  # Inicializar con NaN
    if 'Beneficiary_Institution_58A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Institution", ['Beneficiary_Institution_58A_Identifier_Code', 'Beneficiary_Institution_58D_Party_Identifier'])
    elif 'Beneficiary_Institution_58D_Party_Identifier' in df_copiado.columns:
        df_copiado["Beneficiary_Institution"] = df_copiado['Beneficiary_Institution_58D_Party_Identifier']

    lista_Account_With_Institution_57A_10_Identifier_Code=[]
    lista_Account_With_Institution_57C_10=[]
    lista_Account_With_Institution_57D_10_Party_Identifier=[]
    for i in range(10,1,-1):
        lista_Account_With_Institution_57A_10_Identifier_Code.append(f'Account_With_Institution_57A_{i}_Identifier_Code')
        lista_Account_With_Institution_57C_10.append(f'Account_With_Institution_57C_{i}')
        lista_Account_With_Institution_57D_10_Party_Identifier.append(f'Account_With_Institution_57D_{i}_Party_Identifier')
    lista_Account_With_Institution_57A_10_Identifier_Code= lista_Account_With_Institution_57A_10_Identifier_Code+lista_Account_With_Institution_57C_10+lista_Account_With_Institution_57D_10_Party_Identifier
    lista_Account_With_Institution_57A_10_Identifier_Code.extend(['Account_With_Institution_57A_Identifier_Code','Account_With_Institution_57B_1_Party_Identifier' 'Account_With_Institution_57B_Party_Identifier','Account_With_Institution_57C_1_Party_Identifier','Account_With_Institution_57C_Party_Identifier','Account_With_Institution_57C_1','Account_With_Institution_57C','Account_With_Institution_57D_Party_Identifier','Beneficiary_Institution','Receiver_InstructedAgent_Responder'])
    df_copiado["Account_With_Institution"] = np.nan  # Inicializar con NaN
    if 'Account_With_Institution_57A_10_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, 'Account_With_Institution',lista_Account_With_Institution_57A_10_Identifier_Code )
    else:
        df_copiado["Account_With_Institution_57A_10_Identifier_Code"]=np.nan
        fillna_if_exist(df_copiado, 'Account_With_Institution',lista_Account_With_Institution_57A_10_Identifier_Code )

    df_copiado["Issuing_Bank_Name"] = np.nan  # Inicializar con NaN
    if 'Issuing_Bank_52D_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issuing_Bank_Name", ['Issuing_Bank_52D_Name_and_Address', 'Issuer_52D_Name_and_Address','Issuing_Bank_of_the_Original_Documentary_Credit_52D_Name_and_Address'])
    else:
        df_copiado["Issuing_Bank_52D_Name_and_Address"] = np.nan
        fillna_if_exist(df_copiado, "Issuing_Bank_Name", ['Issuing_Bank_52D_Name_and_Address', 'Issuer_52D_Name_and_Address','Issuing_Bank_of_the_Original_Documentary_Credit_52D_Name_and_Address'])

    lista_Remittance_Information_70=[]
    for i in range(10,0,-1):
        lista_Remittance_Information_70.append(f'Remittance_Information_70_{i}')
    lista_Remittance_Information_70.extend(['Remittance_Information_70'])
    df_copiado["Remittance_Information"] = np.nan  # Inicializar con NaN
    if 'Remittance_Information_70_10' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Remittance_Information",lista_Remittance_Information_70 )
    else:
        df_copiado["Remittance_Information_70_10"] = np.nan
        fillna_if_exist(df_copiado, "Remittance_Information",lista_Remittance_Information_70 )

    df_copiado["Advise_Through_Bank"] = np.nan  # Inicializar con NaN
    if 'Advise_Through_Bank_57A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Advise_Through_Bank", ['Advise_Through_Bank_57A_Identifier_Code', 'Advise_Through_Bank_57B_Party_Idendifier', 'Advise_Through_Bank_57D_Party_Idendifier','Receiver_InstructedAgent_Responder'])
    elif 'Advise_Through_Bank_57B_Party_Idendifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Advise_Through_Bank", ['Advise_Through_Bank_57B_Party_Idendifier', 'Advise_Through_Bank_57D_Party_Idendifier','Receiver_InstructedAgent_Responder'])
    elif 'Advise_Through_Bank_57D_Party_Idendifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Advise_Through_Bank", ['Advise_Through_Bank_57D_Party_Idendifier','Receiver_InstructedAgent_Responder'])
    elif 'Receiver_InstructedAgent_Responder' in df_copiado.columns:
        df_copiado["Advise_Through_Bank"] = df_copiado['Receiver_InstructedAgent_Responder']

    df_copiado["Issuing_Bank"] = np.nan  # Inicializar con NaN
    if 'Issuer_52A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issuing_Bank", ['Issuer_52A_Identifier_Code', 'Issuer_52D_Party_Identifier','Issuing_Bank_52A_Identifier_Code','Issuing_Bank_52D_Party_Identifier','Issuing_Bank_of_the_Original_Documentary_Credit_52A_Identifier_Code','Issuing_Bank_of_the_Original_Documentary_Credit_52D_Party_Identifier','Sender_InstructingAgent_Requestor'])
    else:
        df_copiado["Issuer_52A_Identifier_Code"] = np.nan
        fillna_if_exist(df_copiado, "Issuing_Bank", ['Issuer_52A_Identifier_Code', 'Issuer_52D_Party_Identifier','Issuing_Bank_52A_Identifier_Code','Issuing_Bank_52D_Party_Identifier','Issuing_Bank_of_the_Original_Documentary_Credit_52A_Identifier_Code','Issuing_Bank_of_the_Original_Documentary_Credit_52D_Party_Identifier','Sender_InstructingAgent_Requestor'])


    df_copiado['Role'] = np.nan  # Inicializar con NaN
    df_copiado['Role'] = df_copiado['Role'].astype(object)  # Convertir a tipo object
    for index, row in df_copiado.iterrows():
        if not str(row['Message_type']).startswith('7'):
            if 'Ordering_Institution' in df_copiado.columns and pd.notna(row['OwnBIC8']) and pd.notna(row['Ordering_Institution']) and row['OwnBIC8'] in row['Ordering_Institution']:
                df_copiado.at[index, 'Role'] = "Debtor"
            elif 'Account_With_Institution' in df_copiado.columns and pd.notna(row['OwnBIC8']) and pd.notna(row['Account_With_Institution']) and row['OwnBIC8'] in row['Account_With_Institution']:
                df_copiado.at[index, 'Role'] = "Creditor"
            elif 'Beneficiary_Institution' in df_copiado.columns and pd.notna(row['OwnBIC8']) and pd.notna(row['Beneficiary_Institution']) and row['OwnBIC8'] in row['Beneficiary_Institution']:
                df_copiado.at[index, 'Role'] = "Creditor"
            else:
                df_copiado.at[index, 'Role'] = "Intermediary"
        elif pd.notna(row['Message_type']) and str(row['Message_type']).startswith('7'):
            if 'Advise_Through_Bank' in df_copiado.columns and pd.notna(row['OwnBIC8']) and pd.notna(row['Advise_Through_Bank']) and row['OwnBIC8'] in row['Advise_Through_Bank']:
                df_copiado.at[index, 'Role'] = "Advisor"
            elif 'Issuing_Bank' in df_copiado.columns and pd.notna(row['OwnBIC8']) and pd.notna(row['Issuing_Bank']) and row['OwnBIC8'] in row['Issuing_Bank']:
                df_copiado.at[index, 'Role'] = "Issuer"
            else:
                df_copiado.at[index, 'Role'] = "Intermediary"
        else:
            df_copiado.at[index, 'Role'] = np.nan

    df_copiado["Ordering_Institution_Name"] = np.nan  # Inicializar con NaN
    if 'Ordering_Institution_52D_1_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution_Name", ['Ordering_Institution_52D_1_Name_and_Address','Ordering_Institution_52D_Name_and_Address'])
    elif 'Ordering_Institution_52D_Name_and_Address' in df_copiado.columns:
        df_copiado["Ordering_Institution_Name"] = df_copiado['Ordering_Institution_52D_Name_and_Address']

    df_copiado["Description_Goods"] = np.nan  # Inicializar con NaN
    if 'Description_of_Goods_and/or_Services_45A' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Description_Goods", ['Description_of_Goods_and/or_Services_45A','Description_of_Goods_and/or_Services_45B'])
    elif 'Description_of_Goods_and/or_Services_45B' in df_copiado.columns:
        df_copiado["Description_Goods"] = df_copiado['Description_of_Goods_and/or_Services_45B']

    #Columnas repetidas porque no se pueden separar
    if 'Ordering_Institution_Name' in df_copiado.columns:
        df_copiado["Ordering_Institution_Address"] = df_copiado['Ordering_Institution_Name']
    if 'Applicant_Name' in df_copiado.columns:
        df_copiado["Applicant_Address"] = df_copiado['Applicant_Name']
    if 'Local_Applicant_Name' in df_copiado.columns:
        df_copiado["Local_Applicant_Address"] = df_copiado['Local_Applicant_Name']
    if 'Issuing_Bank_Name' in df_copiado.columns:
        df_copiado["Issuing_Bank_Address"] = df_copiado['Issuing_Bank_Name']
    if 'Requested_Confirmation_Party_Name' in df_copiado.columns:
        df_copiado["Requested_Confirmation_Party_Address"] = df_copiado['Requested_Confirmation_Party_Name']
    if 'Drawee_Name' in df_copiado.columns:
        df_copiado["Drawee_Address"] = df_copiado['Drawee_Name']
    if 'Reimbursing_Bank_Name' in df_copiado.columns:
        df_copiado["Reimbursing_Bank_Address"] = df_copiado['Reimbursing_Bank_Name']
    if 'Advising_Bank_Name' in df_copiado.columns:
        df_copiado["Advising_Bank_Address"] = df_copiado['Advising_Bank_Name']
    if 'Local_Beneficiary_Name' in df_copiado.columns:
        df_copiado["Local_Beneficiary_Address"] = df_copiado['Local_Beneficiary_Name']
    if 'Beneficiary_Institution_Name' in df_copiado.columns:
        df_copiado["Beneficiary_Institution_Address"] = df_copiado['Beneficiary_Institution_Name']

    df_copiado["Senders_Correspondent"] = np.nan  # Inicializar con NaN
    if 'Senders_Correspondent_53A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Correspondent", ['Senders_Correspondent_53A_Identifier_Code', 'Senders_Correspondent_53D_Party_Identifier', 'Senders_Correspondent_53B_Party_Identifier'])
    elif 'Senders_Correspondent_53D_Party_Identifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Correspondent", ['Senders_Correspondent_53D_Party_Identifier', 'Senders_Correspondent_53B_Party_Identifier'])
    elif 'Senders_Correspondent_53B_Party_Identifier' in df_copiado.columns:
        df_copiado["Senders_Correspondent"] = df_copiado['Senders_Correspondent_53B_Party_Identifier']

    df_copiado["InstructionId"] = np.nan  # Inicializar con NaN
    if 'Senders_Reference_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "InstructionId", ['Senders_Reference_20','Transaction_Reference_Number_20'])
    elif 'Transaction_Reference_Number_20' in df_copiado.columns:
        df_copiado["InstructionId"] = df_copiado['Transaction_Reference_Number_20']

    df_copiado["Senders_Correspondent_Name_and_Address"] = np.nan  # Inicializar con NaN
    if 'Senders_Correspondent_53D_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Correspondent_Name_and_Address", ['Senders_Correspondent_53D_Name_and_Address', 'Senders_Correspondent_53B_Location'])
    elif 'Sender_Correspondent_53B_Location' in df_copiado.columns:
        df_copiado["Senders_Correspondent_Name_and_Address"] = df_copiado['Senders_Correspondent_53B_Location']

    df_copiado["Receivers_Correspondent"] = np.nan  # Inicializar con NaN
    if 'Receivers_Correspondent_54A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Receivers_Correspondent", ['Receivers_Correspondent_54A_Identifier_Code', 'Receivers_Correspondent_54D_Party_Identifier', 'Receivers_Correspondent_54B_Party_Identifier'])
    elif 'Receivers_Correspondent_54D_Party_Identifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Receivers_Correspondent", ['Receivers_Correspondent_54D_Party_Identifier', 'Receivers_Correspondent_54B_Party_Identifier'])
    elif 'Receivers_Correspondent_54B_Party_Identifier' in df_copiado.columns:
        df_copiado["Receivers_Correspondent"] = df_copiado['Receivers_Correspondent_54B_Party_Identifier']

    df_copiado["Receivers_Correspondent_Name_and_Address"] = np.nan  # Inicializar con NaN
    if 'Receivers_Correspondent_53D_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Receivers_Correspondent_Name_and_Address", ['Receivers_Correspondent_53D_Name_and_Address', 'Receivers_Correspondent_53B_Location'])
    elif 'Receivers_Correspondent_53B_Location' in df_copiado.columns:
        df_copiado["Receivers_Correspondent_Name_and_Address"] = df_copiado['Receivers_Correspondent_53B_Location']

    df_copiado["Documents_Required"] = np.nan  # Inicializar con NaN
    if 'Documents_Required_46A' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Documents_Required", ['Documents_Required_46A', 'Documents_Required_46B'])
    elif 'Sender_Correspondent_53B_Location' in df_copiado.columns:
        df_copiado["Documents_Required"] = df_copiado['Documents_Required_46B']

    df_copiado["Instructions"] = np.nan  # Inicializar con NaN
    if 'Instructions_to_the_Paying/Accepting/Negotiating_Bank_78' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Instructions", ['Instructions_to_the_Paying/Accepting/Negotiating_Bank_78', 'Document_and_Presentation_Instructions_45C'])
    elif 'Sender_Correspondent_53B_Location' in df_copiado.columns:
        df_copiado["Instructions"] = df_copiado['Document_and_Presentation_Instructions_45C']

    df_copiado["Additional_Conditions"] = np.nan  # Inicializar con NaN
    if 'Additional_Conditions_47A' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Additional_Conditions", ['Additional_Conditions_47A', 'Additional_Conditions_47B'])
    elif 'Additional_Conditions_47B' in df_copiado.columns:
        df_copiado["Additional_Conditions"] = df_copiado['Additional_Conditions_47B']

    df_copiado["PortDischarge_AirportDestination"] = np.nan  # Inicializar con NaN
    if 'Port_of_Discharge/Airport_of_Departure_44F' in df_copiado.columns:
        fillna_if_exist(df_copiado, "PortDischarge_AirportDestination", ['Port_of_Discharge/Airport_of_Departure_44F', 'Port_of_Discharge/Airport_of_Destination_44F'])
    elif 'Port_of_Discharge/Airport_of_Destination_44F' in df_copiado.columns:
        df_copiado["PortDischarge_AirportDestination"] = df_copiado['Port_of_Discharge/Airport_of_Destination_44F']

    df_copiado["Third_Reimbursement_Institution"] = np.nan  # Inicializar con NaN
    if 'Third_Reimbursement_Institution_55A_Party_Identifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Third_Reimbursement_Institution", ['Third_Reimbursement_Institution_55A_Party_Identifier', 'Third_Reimbursement_Institution_55B_Party_Identifier', 'Third_Reimbursement_Institution_55D_Party_Identifier'])
    elif 'Third_Reimbursement_Institution_55B_Party_Identifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Third_Reimbursement_Institution", ['Third_Reimbursement_Institution_55B_Party_Identifier', 'Third_Reimbursement_Institution_55D_Party_Identifier'])
    elif 'Third_Reimbursement_Institution_55D_Party_Identifier' in df_copiado.columns:
        df_copiado["Third_Reimbursement_Institution"] = df_copiado['Third_Reimbursement_Institution_55D_Party_Identifier']

    df_copiado["Third_Reimbursement_Institution_Name_and_Address"] = np.nan  # Inicializar con NaN
    if 'Third_Reimbursement_Institution_55D_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Third_Reimbursement_Institution_Name_and_Address", ['Third_Reimbursement_Institution_55D_Name_and_Address', 'Third_Reimbursement_Institution_55B_Location'])
    elif 'Third_Reimbursement_Institution_55B_Location' in df_copiado.columns:
        df_copiado["Third_Reimbursement_Institution_Name_and_Address"] = df_copiado['Third_Reimbursement_Institution_55B_Location']

    lista_Beneficiary_59_Account_nueva=[]
    for i in range(10,0,-1):
        lista_Beneficiary_59_Account_nueva.append(f'Beneficiary_59_Account_{i}')
    lista_Beneficiary_59_Account_nueva.extend( ['Beneficiary_59_Account_nueva','Beneficiary_Customer_59_Account', 'Beneficiary_Customer_59A_Account','Beneficiary_59_Account','Beneficiary_Customer_59F_Account'])
    df_copiado["Beneficiary_Account"] = np.nan  # Inicializar con NaN
    if 'Beneficiary_59_Account_10' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Account",lista_Beneficiary_59_Account_nueva)
    else:
        df_copiado["Beneficiary_59_Account_10"] = np.nan
        fillna_if_exist(df_copiado, "Beneficiary_Account",lista_Beneficiary_59_Account_nueva)

    lista_Account_With_Institution_57D_1_Name_and_Address=[]
    for i in range(10,0,-1):
        lista_Account_With_Institution_57D_1_Name_and_Address.append(f'Account_With_Institution_57D_{i}_Name_and_Address')
    lista_Account_With_Institution_57D_1_Name_and_Address.extend(['Account_With_Institution_57D_Name_and_Address','Account_With_Institution_57B_1_Location','Account_With_Institution_57B_Location'])
    df_copiado["Account_With_Institution_Name_and_Address"] = np.nan  # Inicializar con NaN
    if 'Account_With_Institution_57D_10_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, 'Account_With_Institution_Name_and_Address',lista_Account_With_Institution_57D_1_Name_and_Address)
    else:
        df_copiado["Account_With_Institution_57D_10_Name_and_Address"] = np.nan
        fillna_if_exist(df_copiado, 'Account_With_Institution_Name_and_Address',lista_Account_With_Institution_57D_1_Name_and_Address)

    df_copiado["Type_Purpose"] = np.nan
    if 'Form_of_Documentary_Credit_40A' in df_copiado.columns:
        fillna_if_exist(df_copiado, 'Type_Purpose', ['Form_of_Documentary_Credit_40A', 'Purpose_of_Message_22A','Form_of_Documentary_Credit_40B_Type'])
    elif 'Purpose_of_Message_22A' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Type_Purpose", ['Purpose_of_Message_22A', 'Form_of_Documentary_Credit_40B_Type'])
    elif 'Form_of_Documentary_Credit_40B_Type' in df_copiado.columns:
        df_copiado["Type_Purpose"] = df_copiado['Form_of_Documentary_Credit_40B_Type']

    df_copiado["Issuing_Banks_Reference"] = np.nan  # Inicializar con NaN
    if 'Issuing_Banks_Reference_23' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issuing_Banks_Reference", ['Issuing_Banks_Reference_23', 'Documentary_Credit_Number_20','Documentary_Credit_Number_21','Undertaking_Number_20','Senders_TRN_20'])
    elif 'Documentary_Credit_Number_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issuing_Banks_Reference", ['Documentary_Credit_Number_20','Documentary_Credit_Number_21','Undertaking_Number_20','Senders_TRN_20'])
    elif 'Documentary_Credit_Number_21' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issuing_Banks_Reference", ['Documentary_Credit_Number_21','Undertaking_Number_20','Senders_TRN_20'])
    elif 'Undertaking_Number_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issuing_Banks_Reference", ['Undertaking_Number_20','Senders_TRN_20'])
    elif 'Senders_TRN_20' in df_copiado.columns:
        df_copiado["Issuing_Banks_Reference"] = df_copiado['Senders_TRN_20']


    df_copiado["Senders_Reference"] = np.nan  # Inicializar con NaN
    if 'Senders_Reference_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Reference", ['Senders_Reference_20', 'Documentary_Credit_Number_20','Transferring_Banks_Reference_20','Transaction_Reference_Number_20','Undertaking_Number_20'])
    elif 'Documentary_Credit_Number_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Reference", ['Documentary_Credit_Number_20','Transferring_Banks_Reference_20','Transaction_Reference_Number_20','Undertaking_Number_20'])
    elif 'Transferring_Banks_Reference_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Reference", ['Transferring_Banks_Reference_20','Transaction_Reference_Number_20','Undertaking_Number_20'])
    elif 'Transaction_Reference_Number_20' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Senders_Reference", ['Transaction_Reference_Number_20','Undertaking_Number_20'])
    elif 'Undertaking_Number_20' in df_copiado.columns:
        df_copiado["Senders_Reference"] = df_copiado['Undertaking_Number_20']

    df_copiado["Receivers_Reference"] = np.nan  # Inicializar con NaN
    if 'Receivers_Reference_21' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Receivers_Reference", ['Receivers_Reference_21','Documentary_Credit_Number_21','Presenting_Banks_Reference_21','Related_Reference_21'])
    elif 'Documentary_Credit_Number_21' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Receivers_Reference", ['Documentary_Credit_Number_21','Presenting_Banks_Reference_21','Related_Reference_21'])
    elif 'Presenting_Banks_Reference_21' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Receivers_Reference", ['Presenting_Banks_Reference_21', 'Related_Reference_21'])
    elif 'Related_Reference_21' in df_copiado.columns:
        df_copiado["Receivers_Reference"] = df_copiado['Related_Reference_21']

    df_copiado["Issue_Date"] = np.nan  # Inicializar con NaN
    if 'Date_of_Issue_31C' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issue_Date", ['Date_of_Issue_31C','Date_and_Amount_of_Utilisation_32A_Date','Date_of_Issue_30'])
    elif 'Date_and_Amount_of_Utilisation_32A_Date' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Issue_Date", ['Date_and_Amount_of_Utilisation_32A_Date','Date_of_Issue_30'])
    elif 'Date_of_Issue_30' in df_copiado.columns:
        df_copiado["Issue_Date"] = df_copiado['Date_of_Issue_30']

    df_copiado["Expiry_Date"] = np.nan  # Inicializar con NaN
    if 'Date_and_Place_of_Expiry_31D_Date' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Expiry_Date", ['Date_and_Place_of_Expiry_31D_Date', 'Date_of_Expiry_31E'])
    elif 'Date_of_Expiry_31E' in df_copiado.columns:
        df_copiado["Expiry_Date"] = df_copiado['Date_of_Expiry_31E']

    df_copiado["Applicant_Bank"] = np.nan  # Inicializar con NaN
    if 'Applicant_Bank_51A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Applicant_Bank", ['Applicant_Bank_51A_Identifier_Code', 'Applicant_Bank_51D_Party_Idendifier', 'Issuing_Bank'])
    elif 'Applicant_Bank_51D_Party_Idendifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Applicant_Bank", ['Applicant_Bank_51D_Party_Idendifier', 'Issuing_Bank'])
    elif 'Issuing_Bank' in df_copiado.columns:
        df_copiado["Applicant_Bank"] = df_copiado['Issuing_Bank']

    df_copiado["Currency"] = np.nan  # Inicializar con NaN
    if 'Currency_Code_Amount_32B_Currency' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Currency", ['Currency_Code_Amount_32B_Currency','Date_and_Amount_of_Utilisation_32A_Currency','Increase_of_Documentary_Credit_Amount_32B_Currency','Undertaking_Amount_32B_Currency'])
    elif 'Date_and_Amount_of_Utilisation_32A_Currency' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Currency", ['Date_and_Amount_of_Utilisation_32A_Currency','Increase_of_Documentary_Credit_Amount_32B_Currency','Undertaking_Amount_32B_Currency'])
    elif 'Increase_of_Documentary_Credit_Amount_32B_Currency' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Currency", ['Increase_of_Documentary_Credit_Amount_32B_Currency','Undertaking_Amount_32B_Currency'])
    elif 'Undertaking_Amount_32B_Currency' in df_copiado.columns:
        df_copiado["Currency"] = df_copiado['Undertaking_Amount_32B_Currency']

    df_copiado["Local_Currency"] = np.nan  # Inicializar con NaN
    if 'Undertaking_Amount_32B_1_Currency' in df_copiado.columns:
        df_copiado["Local_Currency"] = df_copiado['Undertaking_Amount_32B_1_Currency']


    df_copiado["Amount"] = np.nan  # Inicializar con NaN
    if 'Currency_Code_Amount_32B_Amount' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Amount", ['Currency_Code_Amount_32B_Amount','Date_and_Amount_of_Utilisation_32A_Amount','Increase_of_Documentary_Credit_Amount_32B_Amount','Undertaking_Amount_32B_Amount'])
    elif 'Date_and_Amount_of_Utilisation_32A_Amount' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Amount", ['Date_and_Amount_of_Utilisation_32A_Amount','Increase_of_Documentary_Credit_Amount_32B_Amount','Undertaking_Amount_32B_Amount'])
    elif 'Increase_of_Documentary_Credit_Amount_32B_Amount' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Amount", ['Increase_of_Documentary_Credit_Amount_32B_Amount','Undertaking_Amount_32B_Amount'])
    elif 'Undertaking_Amount_32B_Amount' in df_copiado.columns:
        df_copiado["Amount"] = df_copiado['Undertaking_Amount_32B_Amount']

    df_copiado["Local_Amount"] = np.nan  # Inicializar con NaN
    if 'Undertaking_Amount_32B_1_Amount' in df_copiado.columns:
        df_copiado["Local_Amount"] = df_copiado['Undertaking_Amount_32B_1_Amount']

    df_copiado["Available_Bank"] = np.nan  # Inicializar con NaN
    if 'Available_With_By_41A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Available_Bank", ['Available_With_By_41A_Identifier_Code', 'Available_With_41F'])
    elif 'Available_With_41F' in df_copiado.columns:
        df_copiado["Available_Bank"] = df_copiado['Available_With_41F']

    df_copiado["Available_Bank_Name"] = np.nan  # Inicializar con NaN
    if 'Available_With_By_41D_Name_and_Address' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Available_Bank_Name", ['Available_With_By_41D_Name_and_Address', 'Available_With_41G'])
    elif 'Available_With_41G' in df_copiado.columns:
        df_copiado["Available_Bank_Name"] = df_copiado['Available_With_41G']
    df_copiado["Available_Bank_Address"] = df_copiado['Available_Bank_Name']

    df_copiado["Drawee"] = np.nan  # Inicializar con NaN
    if 'Drawee_42A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Drawee", ['Drawee_42A_Identifier_Code', 'Drawee_42D_Party_Identifier'])
    elif 'Drawee_42D_Party_Identifier' in df_copiado.columns:
        df_copiado["Drawee"] = df_copiado['Drawee_42D_Party_Identifier']

    df_copiado["Requested_Confirmation_Party"] = np.nan  # Inicializar con NaN
    if 'Requested_Confirmation_Party_58A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Requested_Confirmation_Party", ['Requested_Confirmation_Party_58A_Identifier_Code', 'Requested_Confirmation_Party_58D_Party_Identifier'])
    elif 'Requested_Confirmation_Party_58D_Party_Identifier' in df_copiado.columns:
        df_copiado["Requested_Confirmation_Party"] = df_copiado['Requested_Confirmation_Party_58D_Party_Identifier']

    df_copiado["Reimbursing_Bank"] = np.nan  # Inicializar con NaN
    if 'Reimbursing_Bank_53A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Reimbursing_Bank", ['Reimbursing_Bank_53A_Identifier_Code', 'Reimbursing_Bank_53D_Party_Identifier'])
    elif 'Reimbursing_Bank_53D_Party_Identifier' in df_copiado.columns:
        df_copiado["Reimbursing_Bank"] = df_copiado['Reimbursing_Bank_53D_Party_Identifier']

    df_copiado["Advising_Bank"] = np.nan  # Inicializar con NaN
    if 'Advising_Bank_56A_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Advising_Bank", ['Advising_Bank_56A_Identifier_Code','Advising_Bank_56A_Party_Identifier','Advising_Bank_56D_Party_Identifier'])
    elif 'Advising_Bank_56A_Party_Identifier' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Advising_Bank", ['Advising_Bank_56A_Party_Identifier', 'Advising_Bank_56D_Party_Identifier'])
    elif 'Advising_Bank_56D_Party_Identifier' in df_copiado.columns:
        df_copiado["Advising_Bank"] = df_copiado['Advising_Bank_56D_Party_Identifier']

    df_copiado["Advise_Through_Bank_Name_and_Address"] = np.nan  # Inicializar con NaN
    if 'Advise_Through_Bank_57B_Location' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Advise_Through_Bank_Name_and_Address", ['Advise_Through_Bank_57B_Location', 'Advise_Through_Bank_57D_Name_and_Address'])
    elif 'Advise_Through_Bank_57D_Name_and_Address' in df_copiado.columns:
        df_copiado["Advise_Through_Bank_Name_and_Address"] = df_copiado['Advise_Through_Bank_57D_Name_and_Address']

    df_copiado["Related_Message_Id"] = np.nan
    lista_Transaction_Reference_21_10=[]
    for i in range(10,0,-1):
        lista_Transaction_Reference_21_10.append(f'Transaction_Reference_21_{i}')
    lista_Transaction_Reference_21_10.extend(['Transaction_Reference_21','Related_Reference_21'])
    if 'Transaction_Reference_21_10' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Related_Message_Id",lista_Transaction_Reference_21_10)
    else:
        df_copiado["Transaction_Reference_21_10"] = np.nan
        fillna_if_exist(df_copiado, "Related_Message_Id",lista_Transaction_Reference_21_10)

    if 'Sender_to_Receiver_Information_72_1' in df_copiado.columns and not df_copiado['Sender_to_Receiver_Information_72_1'].isnull().all():
        df_copiado['Sender_to_Receiver_Information_72_1'] = df_copiado['Sender_to_Receiver_Information_72_1'].apply(lambda x: f'72 Sender to Receiver Information: {x}' if pd.notnull(x) else x)
    if 'Sender_to_Receiver_Information_72' in df_copiado.columns and not df_copiado['Sender_to_Receiver_Information_72'].isnull().all():
        df_copiado['Sender_to_Receiver_Information_72'] = df_copiado['Sender_to_Receiver_Information_72'].apply(lambda x: f'72 Sender to Receiver Information: {x}' if pd.notnull(x) else x)
    if 'Sender_to_Receiver_Information_72Z' in df_copiado.columns and not df_copiado['Sender_to_Receiver_Information_72Z'].isnull().all():
        df_copiado['Sender_to_Receiver_Information_72Z'] = df_copiado['Sender_to_Receiver_Information_72Z'].apply(lambda x: f'72Z Sender to Receiver Information: {x}' if pd.notnull(x) else x)
    if 'Discrepancies_77J' in df_copiado.columns and not df_copiado['Discrepancies_77J'].isnull().all():
        df_copiado['Discrepancies_77J'] = df_copiado['Discrepancies_77J'].apply(lambda x: f'77J Discrepancies: {x}' if pd.notnull(x) else x)
    if 'Sender_to_Receiver_Information_72Z' in df_copiado.columns and 'Discrepancies_77J' in df_copiado.columns:
        df_copiado['Combinada_72Z_77J'] = df_copiado.apply(lambda row: str(row['Sender_to_Receiver_Information_72Z']) + ' ' + str(row['Discrepancies_77J']) if pd.notna(row['Sender_to_Receiver_Information_72Z']) and pd.notna(row['Discrepancies_77J']) else None, axis=1)

    df_copiado["Sender_to_Receiver_Information"] = np.nan  # Inicializar con NaN
    if 'Sender_to_Receiver_Information_72_1' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Sender_to_Receiver_Information",['Sender_to_Receiver_Information_72_1','Sender_to_Receiver_Information_72','Combinada_72Z_77J','Sender_to_Receiver_Information_72Z','Discrepancies_77J'])
    elif 'Sender_to_Receiver_Information_72' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Sender_to_Receiver_Information",['Sender_to_Receiver_Information_72','Combinada_72Z_77J','Sender_to_Receiver_Information_72Z','Discrepancies_77J'])
    elif 'Combinada_72Z_77J' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Sender_to_Receiver_Information",['Combinada_72Z_77J','Sender_to_Receiver_Information_72Z','Discrepancies_77J'])
    elif 'Sender_to_Receiver_Information_72Z' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Sender_to_Receiver_Information",['Sender_to_Receiver_Information_72Z','Discrepancies_77J'])
    elif 'Discrepancies_77J' in df_copiado.columns:
        df_copiado["Sender_to_Receiver_Information"] = df_copiado['Discrepancies_77J']

    df_copiado["Interbank_Settled_Date"] = np.nan  # Inicializar con NaN
    if 'Value_Date/Currency/Interbank_Settled_Amount_32A_Date' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Interbank_Settled_Date", ['Value_Date/Currency/Interbank_Settled_Amount_32A_Date','Requested_Execution_Date_30','Value_Date_Currency_Code_Amount_32A_Date'])
    else:
        df_copiado["Value_Date/Currency/Interbank_Settled_Amount_32A_Date"] = np.nan
        fillna_if_exist(df_copiado, "Interbank_Settled_Date", ['Value_Date/Currency/Interbank_Settled_Amount_32A_Date','Requested_Execution_Date_30','Value_Date_Currency_Code_Amount_32A_Date'])
    if 'Interbank_Settled_Date' in df_copiado.columns:
        df_copiado['Interbank_Settled_Date'] = df_copiado['Interbank_Settled_Date'].astype(str)
        df_copiado['Interbank_Settled_Date'] = df_copiado['Interbank_Settled_Date'].str.replace(' ', '')
        mask = df_copiado['Interbank_Settled_Date'].str.len() == 6  # Suponiendo que el formato anterior tiene 6 caracteres
        df_copiado.loc[mask, 'Interbank_Settled_Date'] = df_copiado.loc[mask, 'Interbank_Settled_Date'].apply(lambda x: pd.to_datetime(x, format='%y%m%d').strftime('%Y/%m/%d'))

    df_copiado["Interbank_Settled_Currency"] = np.nan  # Inicializar con NaN
    lista_Currency_Transaction_Amount_32B_10_Currency=[]
    lista_Currency_Transaction_Amount_32B_10_Amount=[]
    for i in range(10,0,-1):
        lista_Currency_Transaction_Amount_32B_10_Currency.append(f'Currency/Transaction_Amount_32B_{i}_Currency')
        lista_Currency_Transaction_Amount_32B_10_Amount.append(f'Currency/Transaction_Amount_32B_{i}_Amount')
    lista_Currency_Transaction_Amount_32B_10_Currency.extend(['Currency/Transaction_Amount_32B_Currency','Value_Date/Currency/Interbank_Settled_Amount_32A_Currency', 'Value_Date_Currency_Code_Amount_32A_Currency'])
    lista_Currency_Transaction_Amount_32B_10_Amount.extend(['Currency/Transaction_Amount_32B_Amount','Value_Date/Currency/Interbank_Settled_Amount_32A_Amount', 'Value_Date_Currency_Code_Amount_32A_Amount'])
    if 'Currency/Transaction_Amount_32B_10_Currency' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Interbank_Settled_Currency",lista_Currency_Transaction_Amount_32B_10_Currency )
    else:
        df_copiado["Currency/Transaction_Amount_32B_10_Currency"] = np.nan  # Inicializar con NaN
        fillna_if_exist(df_copiado, "Interbank_Settled_Currency", lista_Currency_Transaction_Amount_32B_10_Currency)

    df_copiado["Interbank_Settled_Amount"] = np.nan  # Inicializar con NaN
    if 'Currency/Transaction_Amount_32B_10_Amount' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Interbank_Settled_Amount", lista_Currency_Transaction_Amount_32B_10_Amount)
    else:
        df_copiado["Currency/Transaction_Amount_32B_10_Amount"] = np.nan  # Inicializar con NaN
        fillna_if_exist(df_copiado, "Interbank_Settled_Amount",lista_Currency_Transaction_Amount_32B_10_Amount)

    lista_Intermediary_56A_10_Identifier_Code=[]
    lista_Intermediary_56D_10_Name_and_Address=[]
    for i in range(10,0,-1):
        lista_Intermediary_56A_10_Identifier_Code.append(f'Intermediary_56A_{i}_Identifier_Code')
        lista_Intermediary_56D_10_Name_and_Address.append(f'Intermediary_56D_{i}_Name_and_Address')
    lista_Intermediary_56A_10_Identifier_Code=lista_Intermediary_56A_10_Identifier_Code+lista_Intermediary_56D_10_Name_and_Address
    lista_Intermediary_56A_10_Identifier_Code.extend(['Intermediary_Institution_56A_Identifier_Code','Intermediary_56A_Identifier_Code','Intermediary_56D_Name_and_Address'])
    df_copiado["Intermediary_Institution"] = np.nan
    if 'Intermediary_56A_10_Identifier_Code' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Intermediary_Institution",lista_Intermediary_56A_10_Identifier_Code )
    else:
        df_copiado["Intermediary_56A_10_Identifier_Code"] = np.nan
        fillna_if_exist(df_copiado, "Intermediary_Institution",lista_Intermediary_56A_10_Identifier_Code)

    df_copiado['Narrative'] = df_copiado.apply(lambda row: extract_copy_value(row), axis=1)
    if 'Narrative' in df_copiado.columns:
        df_copiado['Narrative'] = df_copiado['Narrative'].str.replace(r'\{.*$', '', regex=True)
        df_copiado['Narrative'] = df_copiado['Narrative'].str.replace(':79:', ':79Narrative:').str.replace(':75:', ':75Queries:').str.replace(':77A:', ':77Narrative:').str.replace(':76:', ':76Answers:')
        df_copiado['Narrative']=  df_copiado['Narrative'].str.replace(r':11S:.*?:', '', regex=True)
        df_copiado['Narrative']=  df_copiado['Narrative'].str.replace(r':11R:.*?:', '', regex=True)

    if 'Latest_Date_Shipment' in df_copiado.columns and not df_copiado['Latest_Date_Shipment'].isnull().all():
        df_copiado['Latest_Date_Shipment'] = df_copiado['Latest_Date_Shipment'].str.replace('-', '')
    if 'Interbank_Settled_Amount' in df_copiado.columns and not df_copiado['Interbank_Settled_Amount'].isnull().all():
        df_copiado['Interbank_Settled_Amount'] = df_copiado['Interbank_Settled_Amount'].str.replace(',', '.')
        df_copiado['Interbank_Settled_Amount'] = df_copiado['Interbank_Settled_Amount'].str.rstrip('0').str.rstrip('.')
    if 'Instructed_Amount' in df_copiado.columns and not df_copiado['Instructed_Amount'].isnull().all():
        df_copiado['Instructed_Amount'] = df_copiado['Instructed_Amount'].str.replace(',', '.')
        df_copiado['Instructed_Amount'] = df_copiado['Instructed_Amount'].str.rstrip('0').str.rstrip('.')
    if 'Amount' in df_copiado.columns and not df_copiado['Amount'].isnull().all():
        df_copiado['Amount'] = df_copiado['Amount'].str.replace(',', '.')
        df_copiado['Amount'] = df_copiado['Amount'].str.rstrip('0').str.rstrip('.')
    if 'Expiry_Date' in df_copiado.columns and not df_copiado['Expiry_Date'].isnull().all():
        df_copiado['Expiry_Date'] = df_copiado['Expiry_Date'].str.replace('-', '')
    if 'Beneficiary_Name' in df_copiado.columns and not df_copiado['Beneficiary_Name'].isnull().all():
        df_copiado['Beneficiary_Name'] = df_copiado['Beneficiary_Name'].str.lstrip(':')

    mask = df_copiado['Message_type'].str.endswith(('90', '91', '92', '93', '94', '95', '96', '97', '98', '99'))
    df_copiado.loc[mask, ['Role', 'Ordering_Institution','Account_With_Institution']] = np.nan

    columnas_deseadas = ['Filename','Message_Date','Message_Content','Creation_date','Interbank_Settled_Date', 'Direction', 'Sender_InstructingAgent_Requestor', 'Receiver_InstructedAgent_Responder', 'OwnBIC8', 'Own_Country', 'Role', 'Format', 'Message_type', 'InstructionId', 'End2EndId', 'UETR', 'Related_Message_Id', 'Transaction_Reference', 'Interbank_Settled_Currency', 'Interbank_Settled_Amount', 'Instructed_Currency', 'Instructed_Amount', 'Ordering_Customer_Account', 'Ordering_Customer_Name','Ordering_Customer_Id', 'Ordering_Customer_Address','Ordering_Institution', 'Ordering_Institution_Name','Ordering_Institution_Address', "Senders_Correspondent", "Senders_Correspondent_Name_and_Address", "Receivers_Correspondent",'Receivers_Correspondent_Name_and_Address', 'Third_Reimbursement_Institution', 'Third_Reimbursement_Institution_Name_and_Address', 'Intermediary_Institution', 'Account_With_Institution','Account_With_Institution_Name_and_Address', 'Beneficiary_Account','Beneficiary_Name', 'Beneficiary_Address','Beneficiary_Address_Country', 'Remittance_Information', 'Sender_to_Receiver_Information','Ordering_Customer_Address_Country','Ordering_Customer_Date_Of_Birth','Ordering_Customer_Place_Of_Birth','Ordering_Customer_ID_Number','Ordering_Customer_National_ID_Number','Ordering_Customer_Additional_Information','Beneficiary_Id','Type_Purpose','Senders_Reference','Receivers_Reference','Issue_Date','Expiry_Date','Expiry_Place','Applicant_Bank','Applicant_Bank_Name_and_Address','Applicant_Bank_Name','Applicant_Bank_Address','Instructing_Party','Issuing_Bank','Issuing_Bank_Name','Issuing_Bank_Address','Currency','Amount','Available_Bank','Available_Bank_Name','Available_Bank_Address','Drawee','Drawee_Name','Drawee_Address','Place_Dispatch_Receipt','PortLoading_AirportDeparture','PortDischarge_AirportDestination','Place_Destination_Delivery','Latest_Date_Shipment','Description_Goods','Documents_Required','Requested_Confirmation_Party','Requested_Confirmation_Party_Name','Requested_Confirmation_Party_Address','Reimbursing_Bank','Reimbursing_Bank_Name','Reimbursing_Bank_Address','Instructions','Advising_Bank','Advising_Bank_Name','Advising_Bank_Address','Advise_Through_Bank','Advise_Through_Bank_Name','Advise_Through_Bank_Address','Advise_Through_Bank_Name_and_Address','Undertaking_Terms','Transaction_Details','Local_Transaction_Details','Local_Type_Purpose','Applicant_Name','Applicant_Address','Local_Applicant_Name','Local_Applicant_Address','Local_Beneficiary_Account','Local_Beneficiary_Name','Local_Beneficiary_Address','Local_Currency','Local_Amount','Local_Undertaking_Terms','Narrative','Issuing_Banks_Reference','Additional_Conditions','Beneficiary_Institution','Beneficiary_Institution_Name','Beneficiary_Institution_Address']

    columnas_existen = [col for col in columnas_deseadas if col in df_copiado.columns]

    df_filtrado = df_copiado[columnas_existen]
    df_filtrado = df_filtrado.drop_duplicates()
    condicion = (df_filtrado['Message_type'].isin(['700','701', '707','708','710','711','720','721','760','761']))
    No_700_df = df_filtrado[~condicion]
    df_filtrado = df_filtrado[condicion]
    condicion = (df_filtrado['Message_type'].isin(['700', '707', '710', '720','760'])) & (df_filtrado.duplicated(subset=['Senders_Reference', 'Message_type'], keep='first'))
    df_primeras_repeticiones = df_filtrado[~condicion]
    df_primeras_repeticiones = df_primeras_repeticiones.sort_values(by='Message_type')
    condicion_segundas_repeticiones = (df_filtrado['Message_type'].isin(['700', '707', '710', '720','760'])) & (df_filtrado.duplicated(subset=['Senders_Reference', 'Message_type'], keep='last'))
    df_segundas_repeticiones = df_filtrado[~condicion_segundas_repeticiones]
    df_segundas_repeticiones = df_segundas_repeticiones.sort_values(by='Message_type')
    columnas_combinar = ['Description_Goods', 'Documents_Required', 'Additional_Conditions','Message_Content','Local_Undertaking_Terms','Undertaking_Terms']
    columnas_presentes_1 = [col for col in columnas_combinar if col in df_filtrado.columns]
    agg_functions_1 = {col: ' '.join if col in columnas_presentes_1 else 'first' for col in df_primeras_repeticiones.columns if col != 'Senders_Reference'}
    agg_functions_2 = {col: ' '.join if col in columnas_presentes_1 else 'first' for col in df_segundas_repeticiones.columns if col != 'Senders_Reference'}
    for col in columnas_presentes_1:
        df_primeras_repeticiones[col] = df_primeras_repeticiones[col].fillna('')
        df_segundas_repeticiones[col] = df_segundas_repeticiones[col].fillna('')
    mask_700 = df_primeras_repeticiones['Message_type'].isin(['700', '701'])
    mask_707 = df_primeras_repeticiones['Message_type'].isin(['707', '708'])
    mask_710 = df_primeras_repeticiones['Message_type'].isin(['710', '711'])
    mask_720 = df_primeras_repeticiones['Message_type'].isin(['720', '721'])
    mask_760 = df_primeras_repeticiones['Message_type'].isin(['760', '761'])
    combined_row_1 = df_primeras_repeticiones[mask_700].groupby('Senders_Reference').agg(agg_functions_1).reset_index()
    combined_row_2 = df_primeras_repeticiones[mask_707].groupby('Senders_Reference').agg(agg_functions_1).reset_index()
    combined_row_3 = df_primeras_repeticiones[mask_710].groupby('Senders_Reference').agg(agg_functions_1).reset_index()
    combined_row_4 = df_primeras_repeticiones[mask_720].groupby('Senders_Reference').agg(agg_functions_1).reset_index()
    combined_row_5 = df_primeras_repeticiones[mask_760].groupby('Senders_Reference').agg(agg_functions_1).reset_index()
    df_primeras_repeticiones = pd.concat([combined_row_1, combined_row_2, combined_row_3, combined_row_4, combined_row_5], ignore_index=True)
    mask_700 = df_segundas_repeticiones['Message_type'].isin(['700', '701'])
    mask_707 = df_segundas_repeticiones['Message_type'].isin(['707', '708'])
    mask_710 = df_segundas_repeticiones['Message_type'].isin(['710', '711'])
    mask_720 = df_segundas_repeticiones['Message_type'].isin(['720', '721'])
    mask_760 = df_segundas_repeticiones['Message_type'].isin(['760', '761'])
    combined_row_1_1 = df_segundas_repeticiones[mask_700].groupby('Senders_Reference').agg(agg_functions_2).reset_index()
    combined_row_2_1 = df_segundas_repeticiones[mask_707].groupby('Senders_Reference').agg(agg_functions_2).reset_index()
    combined_row_3_1 = df_segundas_repeticiones[mask_710].groupby('Senders_Reference').agg(agg_functions_2).reset_index()
    combined_row_4_1 = df_segundas_repeticiones[mask_720].groupby('Senders_Reference').agg(agg_functions_2).reset_index()
    combined_row_5_1 = df_segundas_repeticiones[mask_760].groupby('Senders_Reference').agg(agg_functions_2).reset_index()
    df_segundas_repeticiones = pd.concat([combined_row_1_1, combined_row_2_1, combined_row_3_1, combined_row_4_1, combined_row_5_1], ignore_index=True)
    df_filtrado = pd.concat([df_primeras_repeticiones, df_segundas_repeticiones], ignore_index=True)
    df_filtrado = df_filtrado.drop_duplicates()
    df_filtrado = pd.concat([df_filtrado, No_700_df], ignore_index=True)
    columnas_a_excluir = [columna for columna in df_filtrado.columns if 'date' in columna.lower()]
    columnas_a_excluir.extend(['Message_Content','Receivers_Reference','Senders_Reference','UETR','InstructionId','Issuing_Banks_Reference'])
    for columna in df_filtrado.columns:
        if columna not in columnas_a_excluir:
            df_filtrado[columna] = df_filtrado[columna].apply(lambda x: str(x).replace('3/', ' ').replace('2/', ' ').replace('/', ' ').replace('-', '').replace('}', ''))

    return df_filtrado

In [0]:
#New_MX_Names
import numpy as np
import pandas as pd
import re
from datetime import datetime

##############################################################################################################
## Funcion 1: FUNCIÓN PARA ORDENAR LAS COLUMNAS ADDRESS Y CONTACT
## ------------------------------------------------------------
##############################################################################################################

palabras_clave_address = ["_AdrTp", "_CareOf", "_Dept", "_SubDept", "_StrtNm", "_BldgNb", "_BldgNm", "_Flr", "_UnitNb", "_PstBx", "_Room", "_PstCd", "_TwnNm", "_TwnLctnNm", "_DstrctNm", "_CtrySubDvsn", "_Ctry"]
palabras_clave_contact = ["_NmPrfx","_Nm" ,"_PhneNb","_MobNb","_FaxNb","_URLAdr" ,"_EmailAdr","_EmailPurp","_JobTitl","_Rspnsblty","_Dept","_Othr","_PrefrdMtd"]

def ordenar_por_palabras_clave(lista, palabras_clave):
    resultado = []
    palabras_no_encontradas = []
    for palabra in palabras_clave:
        for elemento in lista:
            if palabra in elemento:
                resultado.append(elemento)
    for elemento in lista:
        if not any(palabra in elemento for palabra in palabras_clave):
            palabras_no_encontradas.append(elemento)
    resultado.extend(palabras_no_encontradas)
    lista_sin_duplicados = []
    for palabra in resultado:
        if palabra not in lista_sin_duplicados:
            lista_sin_duplicados.append(palabra)
    return lista_sin_duplicados

##############################################################################################################
## Funcion 1: FUNCIÓN PARA CAMBIAR EL FORMATO DE LA HORA
## ------------------------------------------------------------
##############################################################################################################

def convertir_formato(fecha_str):
    try:
        # Intenta convertir la fecha al formato deseado
        fecha_datetime = datetime.strptime(fecha_str, 'MX_%Y%m%d-%H%M%S%f')
        return fecha_datetime.strftime('%d-%b-%Y %H:%M:%S')
    except ValueError:
        # En caso de error, devuelve el valor original
        return fecha_str

##############################################################################################################
## Funcion 2: FUNCIÓN PARA JUNTAR EL APPHEADER CON EL DOCUMENT HACIENDOLOS UN ÚNICO MENSAJE
## ------------------------------------------------------------
## Se aplica en SWIFT_parsing.py
##############################################################################################################

def rellenar_nan_con_prefijo(df):
    # Obtener el prefijo común
    df['Common_Prefix'] = df['Filename'].str.extract(r'(\w+)_\w+\.xml')[0]

    # Obtener un diccionario de relleno para cada columna
    fill_dict = df.groupby('Common_Prefix').first().to_dict(orient='index')

    # Rellenar NaN en cada columna usando el diccionario de relleno
    for col in df.columns:
        df[col] = df.apply(lambda row: fill_dict.get(row['Common_Prefix'], {}).get(col, np.nan) if pd.isna(row[col]) else row[col], axis=1)

    # Fusionar el contenido de 'Message_Content' por el prefijo común
    df['Message_Content'] = df.groupby('Common_Prefix')['Message_Content'].transform(lambda x: ' '.join(x.dropna()))

    # Eliminar duplicados basados en 'Common_Prefix'
    df = df.drop_duplicates(subset=['Common_Prefix'], keep='first')

    # Modificar 'Filename'
    df['Filename'] = df['Filename'].str.replace('_Document', '').str.replace('_AppHdr', '')

    # Eliminar la columna 'Common_Prefix' ya que no es necesaria
    df = df.drop('Common_Prefix', axis=1)

    print(df.shape)
    return df

##############################################################################################################
## Funcion 3: FUNCIÓN PARA RELLENAR LOS NAN POR OTRA COLUMNA DEL DATAFRAME SI ESTA COLUMNA EXISTE
## ------------------------------------------------------------
## Se aplica en la función cambiar_nombres_MX
##############################################################################################################

def fillna_if_exist(df, target_column, source_columns, default=None):
    for source_column in source_columns:
        try:
            df[target_column] = df[target_column].fillna(df[source_column])
        except KeyError:
            pass  # La columna de origen no existe, continuar con la siguiente

    if default is not None:
        df[target_column].fillna(default, inplace=True)

def cambiar_nombres_MX(df):
    # Copiar el DataFrame para evitar modificar el original
    nombres_nuevos = {
    'MsgDefIdr': 'Message_type',
    'CdtTrfTxInf_PmtId_EndToEndId': 'End2EndId',
    'CdtTrfTxInf_PmtId_TxId': 'Transaction_Reference',
    'CdtTrfTxInf_InstdAmt_Ccy':'Instructed_Currency',
    'CdtTrfTxInf_InstdAmt': 'Instructed_Amount',
    'CdtTrfTxInf_IntrBkSttlmDt':'Interbank_Settled_Date',
    'GrpHdr_SttlmInf_InstgRmbrsmntAgt_FinInstnId_BICFI': "Senders_Correspondent",
    'GrpHdr_SttlmInf_InstdRmbrsmntAgt_FinInstnId_BICFI':"Receivers_Correspondent",
    'CdtTrfTxInf_Cdtr_PstlAdr_Ctry':'Beneficiary_Address_Country'
    }
    df_copiado = df.copy()
    prefijos = ['FIToFIPmtCxlReq_','CretMmb_','GetAcct_','FICdtTrf_','RtrAcct_','GetTx_','RtrTx_','ModfyTx_','CclTx_','GetLmt_','RtrLmt_','ModfyLmt_','DelLmt_','GetMmb_','RtrMmb_','ModfyMmb_','GetCcyXchgRate_','RtrCcyXchgRate_','GetBizDayInf_','RtrBizDayInf_','GetGnlBizInf_','RtrGnlBizInf_','BckpPmt_','ModfyStgOrdr_','Rct_','GetRsvatn_','RtrRsvatn_','ModfyRsvatn_','DelRsvatn_','LqdtyCdtTrf_','LqdtyDbtTrf_','GetStgOrdr_','RtrStgOrdr_','DelStgOrdr_','CretLmt_','CretStgOrdr_','CretRsvatn_','CretMmb_','FIToFIPmtStsRpt_', 'FIToFICstmrDrctDbt_', 'PmtRtr_', 'FIToFIPmtRvsl_', 'FIToFICstmrCdtTrf_', 'FIToFIPmtStsReq_', 'FIDrctDbt_', 'FIToFIPmtStsReq_','RsltnOfInvstgtn_']
    for prefijo in prefijos:
        df_copiado.columns = df_copiado.columns.map(lambda x: x[len(prefijo):] if x.startswith(prefijo) else x)
    df_copiado = df_copiado.groupby(level=0, axis=1).first()


    df_copiado.rename(columns=nombres_nuevos, inplace=True)
    df_copiado = df_copiado.replace({None: np.nan})
    df_copiado = df_copiado.sort_index(axis=1)
    df_copiado['Direction']=df_copiado['Direction'].astype(str)

    for index, value in df_copiado['Direction'].items():
        # Verificar si el valor contiene " (to SWIFT)"
        if " (to SWIFT)" in value:
            # Realizar el reemplazo
            df_copiado.at[index, 'Direction'] = value.replace(" (to SWIFT)", "")
        elif " (from SWIFT)" in value:
            # Realizar el reemplazo
            df_copiado.at[index, 'Direction'] = value.replace(" (from SWIFT)", "")
        elif 'TransmissionReport_Message_SubFormat' in df_copiado :
            df_copiado.at[index, 'Direction'] = df_copiado.at[index, 'TransmissionReport_Message_SubFormat']
        else:
            df_copiado.at[index, 'Direction'] = np.nan

    mask = (df_copiado['Message_type'].str.startswith('pacs.009') &
            ~df_copiado['Message_type'].str.endswith('COV') &
            df_copiado.filter(like='CdtTrfTxInf_UndrlygCstmrCdtTrf').notnull().any(axis=1))
    df_copiado.loc[mask, 'Message_type'] = df_copiado.loc[mask, 'Message_type'] + 'COV'


    #df_copiado['Direction'] = df_copiado['Direction'].replace({'Outgoing': 'O', 'Incoming': 'I', 'Input': 'I', 'Output': 'O'})
    df_copiado['Direction'] = df_copiado['Direction'].replace({'Input': 'Outgoing', 'Output': 'Incoming'})

    df_copiado['Sender_InstructingAgent_Requestor'] = df_copiado['Fr_FIId_FinInstnId_BICFI'].str[:8]
    df_copiado['Receiver_InstructedAgent_Responder'] = df_copiado['To_FIId_FinInstnId_BICFI'].str[:8]

    df_copiado['OwnBIC8'] = df_copiado.apply(
        lambda row: row['Sender_InstructingAgent_Requestor'] if row['Direction'] == 'Outgoing' else row['Receiver_InstructedAgent_Responder'],
        axis=1
    )

    df_copiado['OwnBIC8'] = df_copiado['OwnBIC8'].astype(str)

    df_copiado['Own_Country'] = df_copiado['OwnBIC8'].str[4:6]

    df_copiado['Role'] = np.nan  # Inicializar con NaN
    df_copiado['Role'] = df_copiado['Role'].astype(object)  # Convertir a tipo object

    df_copiado['Ordering_Customer_Account'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_DbtrAcct_Id_IBAN' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Account", ['CdtTrfTxInf_DbtrAcct_Id_IBAN', 'CdtTrfTxInf_DbtrAcct_Id_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_IBAN','CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_Othr'])
    elif 'CdtTrfTxInf_DbtrAcct_Id_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Account", ['CdtTrfTxInf_DbtrAcct_Id_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_IBAN','CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_Othr'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_IBAN' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Account", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_IBAN','CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_Othr'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_Othr' in df_copiado.columns:
        df_copiado["Ordering_Customer_Account"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAcct_Id_Othr']

    df_copiado['Beneficiary_Id'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Cdtr_Id_OrgId_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Id", ['CdtTrfTxInf_Cdtr_Id_OrgId_Othr_Id', 'CdtTrfTxInf_Cdtr_Id_PrvtId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_OrgId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_PrvtId_Othr_Id'])
    elif 'CdtTrfTxInf_Cdtr_Id_PrvtId_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Id", ['CdtTrfTxInf_Cdtr_Id_PrvtId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_OrgId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_PrvtId_Othr_Id'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_OrgId_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Id", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_OrgId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_PrvtId_Othr_Id'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_PrvtId_Othr_Id' in df_copiado.columns:
        df_copiado["Beneficiary_Id"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Id_PrvtId_Othr_Id']

    df_copiado['Beneficiary_Account'] = np.nan
    if 'CdtTrfTxInf_CdtrAcct_Id_IBAN' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Account", ['CdtTrfTxInf_CdtrAcct_Id_IBAN', 'CdtTrfTxInf_CdtrAcct_Id_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_IBAN','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_Othr'])
    elif 'CdtTrfTxInf_CdtrAcct_Id_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Account", ['CdtTrfTxInf_CdtrAcct_Id_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_IBAN','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_Othr'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_IBAN' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Account", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_IBAN','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_Othr'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_Othr' in df_copiado.columns:
        df_copiado["Beneficiary_Account"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAcct_Id_Othr']

    df_copiado['Ordering_Customer_Id'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Dbtr_Id_OrgId_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Id", ['CdtTrfTxInf_Dbtr_Id_OrgId_Othr_Id', 'CdtTrfTxInf_Dbtr_Id_PrvtId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_OrgId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_PrvtId_Othr_Id'])
    elif 'CdtTrfTxInf_Dbtr_Id_PrvtId_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Id", ['CdtTrfTxInf_Dbtr_Id_PrvtId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_OrgId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_PrvtId_Othr_Id'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_OrgId_Othr_Id' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Id", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_OrgId_Othr_Id','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_PrvtId_Othr_Id'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_PrvtId_Othr_Id' in df_copiado.columns:
        df_copiado["Ordering_Customer_Id"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Id_PrvtId_Othr_Id']

    df_copiado['Beneficiary_Name'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Cdtr_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Name", ['CdtTrfTxInf_Cdtr_Nm', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Nm'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Nm' in df_copiado.columns:
        df_copiado["Beneficiary_Name"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_Nm']

    df_copiado['Beneficiary_Residence_Country'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Cdtr_CtryOfRes' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Residence_Country", ['CdtTrfTxInf_Cdtr_CtryOfRes', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtryOfRes'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtryOfRes' in df_copiado.columns:
        df_copiado["Beneficiary_Residence_Country"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtryOfRes']

    df_copiado['Remittance_Information'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_RmtInf_Ustrd' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Remittance_Information", ['CdtTrfTxInf_RmtInf_Ustrd', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_RmtInf_Ustrd'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_RmtInf_Ustrd' in df_copiado.columns:
        df_copiado["Remittance_Information"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_RmtInf_Ustrd']

    df_copiado['Ordering_Customer_Name'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Dbtr_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Name", ['CdtTrfTxInf_Dbtr_Nm', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Nm'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Nm' in df_copiado.columns:
        df_copiado["Ordering_Customer_Name"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_Nm']

    df_copiado['Account_With_Institution_Name_and_Address'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_CdtrAgt_FinInstnId_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution_Name_and_Address", ['CdtTrfTxInf_CdtrAgt_FinInstnId_Nm','TxInfAndSts_InstdAgt_FinInstnId_Nm','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_Nm'])
    elif 'TxInfAndSts_InstdAgt_FinInstnId_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution_Name_and_Address", ['TxInfAndSts_InstdAgt_FinInstnId_Nm','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_Nm'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_Nm' in df_copiado.columns:
        df_copiado["Account_With_Institution_Name_and_Address"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_Nm']

    df_copiado['Ordering_Institution_Name'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAgt_FinInstnId_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution_Name", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_DbtrAgt_FinInstnId_Nm', 'CdtTrfTxInf_DbtrAgt_FinInstnId_Nm','CdtTrfTxInf_Dbtr_FinInstnId_Nm'])
    elif 'CdtTrfTxInf_DbtrAgt_FinInstnId_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution_Name", ['CdtTrfTxInf_DbtrAgt_FinInstnId_Nm','CdtTrfTxInf_Dbtr_FinInstnId_Nm'])
    elif 'CdtTrfTxInf_Dbtr_FinInstnId_Nm' in df_copiado.columns:
        df_copiado["Ordering_Institution_Name"] = df_copiado['CdtTrfTxInf_Dbtr_FinInstnId_Nm']

    df_copiado['Ordering_Customer_Address_Country'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Dbtr_PstlAdr_Ctry' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Address_Country", ['CdtTrfTxInf_Dbtr_PstlAdr_Ctry', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Ctry'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Ctry' in df_copiado.columns:
        df_copiado["Ordering_Customer_Address_Country"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Ctry']

    df_copiado['Ordering_Customer_Residence_Country'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Dbtr_CtryOfRes' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Residence_Country", ['CdtTrfTxInf_Dbtr_CtryOfRes', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtryOfRes'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtryOfRes' in df_copiado.columns:
        df_copiado["Ordering_Customer_Residence_Country"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtryOfRes']

    df_copiado['Ordering_Institution'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_DbtrAgt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution", ['CdtTrfTxInf_DbtrAgt_FinInstnId_BICFI', 'TxInf_RtrChain_DbtrAgt_FinInstnId_BICFI','TxInf_RtrChain_Dbtr_Agt_FinInstnId_BICFI','TxInf_RtrChain_Dbtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_Dbtr_FinInstnId_BICFI'])
    elif 'TxInf_RtrChain_DbtrAgt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution", ['TxInf_RtrChain_DbtrAgt_FinInstnId_BICFI','TxInf_RtrChain_Dbtr_Agt_FinInstnId_BICFI','TxInf_RtrChain_Dbtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_Dbtr_FinInstnId_BICFI'])
    elif 'TxInf_RtrChain_Dbtr_Agt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution", ['TxInf_RtrChain_Dbtr_Agt_FinInstnId_BICFI','TxInf_RtrChain_Dbtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_Dbtr_FinInstnId_BICFI'])
    elif 'TxInf_RtrChain_Dbtr_Pty_Id_OrgId_AnyBIC' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution", ['TxInf_RtrChain_Dbtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_Dbtr_FinInstnId_BICFI'])
    elif 'CdtTrfTxInf_Dbtr_FinInstnId_BICFI' in df_copiado.columns:
        df_copiado["Ordering_Institution"] = df_copiado['CdtTrfTxInf_Dbtr_FinInstnId_BICFI']

    df_copiado['Account_With_Institution'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_CdtrAgt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution", ['CdtTrfTxInf_CdtrAgt_FinInstnId_BICFI', 'TxInf_RtrChain_CdtrAgt_FinInstnId_BICFI','TxInf_RtrChain_Cdtr_Agt_FinInstnId_BICFI','TxInf_RtrChain_Cdtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_BICFI','TxInfAndSts_InstdAgt_FinInstnId_BICFI'])
    elif 'TxInf_RtrChain_CdtrAgt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution", ['TxInf_RtrChain_CdtrAgt_FinInstnId_BICFI','TxInf_RtrChain_Cdtr_Agt_FinInstnId_BICFI','TxInf_RtrChain_Cdtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_BICFI','TxInfAndSts_InstdAgt_FinInstnId_BICFI'])
    elif 'TxInf_RtrChain_Cdtr_Agt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution", ['TxInf_RtrChain_Cdtr_Agt_FinInstnId_BICFI','TxInf_RtrChain_Cdtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_BICFI','TxInfAndSts_InstdAgt_FinInstnId_BICFI'])
    elif 'TxInf_RtrChain_Cdtr_Pty_Id_OrgId_AnyBIC' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution", ['TxInf_RtrChain_Cdtr_Pty_Id_OrgId_AnyBIC','CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_BICFI','TxInfAndSts_InstdAgt_FinInstnId_BICFI'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_CdtrAgt_FinInstnId_BICFI','TxInfAndSts_InstdAgt_FinInstnId_BICFI'])
    elif 'TxInfAndSts_InstdAgt_FinInstnId_BICFI' in df_copiado.columns:
        df_copiado["Account_With_Institution"] = df_copiado['TxInfAndSts_InstdAgt_FinInstnId_BICFI']

    for index, row in df_copiado.iterrows():
        if pd.notna(row['OwnBIC8']):
            # Verificar si la columna 'Ordering_Institution' existe en el DataFrame
            if 'Ordering_Institution' in df_copiado.columns and pd.notna(row['Ordering_Institution']) and row['OwnBIC8'] in row['Ordering_Institution']:
                df_copiado.at[index, 'Role'] = "Debtor"
            # Verificar si la columna 'Account_With_Institution' existe en el DataFrame
            elif 'Account_With_Institution' in df_copiado.columns and pd.notna(row['Account_With_Institution']) and row['OwnBIC8'] in row['Account_With_Institution']:
                df_copiado.at[index, 'Role'] = "Creditor"
            else:
                df_copiado.at[index, 'Role'] = "Intermediary"
        else:
            df_copiado.at[index, 'Role'] = "Intermediary"

    df_copiado["Ordering_Customer_Address"] = np.nan  # Inicializar con NaN
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_Dbtr_PstlAdr') and col != 'CdtTrfTxInf_Dbtr_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['Ordering_Customer_Address_Combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr') and col != 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    if 'Ordering_Customer_Address_Combinada' in df_copiado.columns:
        df_copiado['Ordering_Customer_Address_Combinada'] = df_copiado['Ordering_Customer_Address_Combinada'].replace('', np.nan)
    if 'CdtTrfTxInf_Dbtr_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Address", ['CdtTrfTxInf_Dbtr_PstlAdr_AdrLine', 'Ordering_Customer_Address_Combinada','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_AdrLine', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Combinada'])
    elif 'Ordering_Customer_Address_Combinada' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Address", ['Ordering_Customer_Address_Combinada','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_AdrLine','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Combinada'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Customer_Address", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_AdrLine','CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Combinada'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Combinada' in df_copiado.columns:
        df_copiado["Ordering_Customer_Address"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_PstlAdr_Combinada']

    df_copiado["Beneficiary_Address"] = np.nan  # Inicializar con NaN
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_Cdtr_PstlAdr') and col != 'CdtTrfTxInf_Cdtr_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['CdtTrfTxInf_Cdtr_PstlAdr_Combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr') and col != 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_Combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    if 'CdtTrfTxInf_Cdtr_PstlAdr_Combinada' in df_copiado.columns:
        df_copiado['CdtTrfTxInf_Cdtr_PstlAdr_Combinada'] = df_copiado['CdtTrfTxInf_Cdtr_PstlAdr_Combinada'].replace('', np.nan)
    if 'CdtTrfTxInf_Cdtr_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Address", ['CdtTrfTxInf_Cdtr_PstlAdr_AdrLine', 'CdtTrfTxInf_Cdtr_PstlAdr_Combinada','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_AdrLine','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_Combinada'])
    elif 'CdtTrfTxInf_Cdtr_PstlAdr_Combinada' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Address", ['CdtTrfTxInf_Cdtr_PstlAdr_Combinada','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_AdrLine','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_Combinada'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Address", ['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_AdrLine','CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_Combinada'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_Combinada' in df_copiado.columns:
        df_copiado["Beneficiary_Address"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_PstlAdr_Combinada']

    df_copiado["Account_With_Institution_Address"] = np.nan  # Inicializar con NaN
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr') and col != 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['Account_With_Institution_Address_Combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    if 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Account_With_Institution_Address", ['CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine', 'Account_With_Institution_Address_Combinada'])
    elif 'Account_With_Institution_Address_Combinada' in df_copiado.columns:
        df_copiado["Account_With_Institution_Address"] = df_copiado['Account_With_Institution_Address_Combinada']

    df_copiado["Ordering_Institution_Address"] = np.nan
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_DbtrAgt_FinInstnId_PstlAdr') and col != 'CdtTrfTxInf_DbtrAgt_FinInstnId_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['Ordering_Institution_Address_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    if 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Ordering_Institution_Address", ['CdtTrfTxInf_DbtrAgt_FinInstnId_PstlAdr_AdrLine', 'Ordering_Institution_Address_combinada'])
    elif 'Ordering_Institution_Address_combinada' in df_copiado.columns:
        df_copiado["Ordering_Institution_Address"] = df_copiado['Ordering_Institution_Address_combinada']

    if 'Beneficiary_Address' in df_copiado.columns:
        if 'Beneficiary_Address_Country' not in df_copiado.columns:
            df_copiado['Beneficiary_Address_Country'] = None
        patron = r'.*\/([a-zA-Z]{2})\/.*'
        df_copiado['Beneficiary_Address_Country'] = df_copiado.apply(lambda row: re.search(patron, row['Beneficiary_Address']).group(1) if pd.isna(row['Beneficiary_Address_Country']) and pd.notna(row['Beneficiary_Address']) and re.match(patron, row['Beneficiary_Address']) else row['Beneficiary_Address_Country'], axis=1)

    if 'Ordering_Customer_Address' in df_copiado.columns:
        if 'Ordering_Customer_Address_Country' not in df_copiado.columns:
            df_copiado['Ordering_Customer_Address_Country'] = None
        patron = r'.*\/([a-zA-Z]{2})\/.*'
        df_copiado['Ordering_Customer_Address_Country'] = df_copiado.apply(lambda row: re.search(patron, row['Ordering_Customer_Address']).group(1) if pd.isna(row['Ordering_Customer_Address_Country']) and pd.notna(row['Ordering_Customer_Address']) and re.match(patron, row['Ordering_Customer_Address']) else row['Ordering_Customer_Address_Country'], axis=1)

    if 'Ordering_Customer_Account' in df_copiado.columns:
        if 'Ordering_Customer_Id'  in df_copiado.columns:
            df_copiado['Ordering_Customer_Id'] = df_copiado['Ordering_Customer_Id'].astype(str)
            condicion = df_copiado['Ordering_Customer_Id'].str.match('^[a-zA-Z]{2}\d+$', na=False) & pd.notna(df_copiado['Ordering_Customer_Id'])
            df_copiado.loc[condicion, 'Ordering_Customer_Account'] = df_copiado.loc[condicion, 'Ordering_Customer_Id']

    if 'Beneficiary_Account' in df_copiado.columns:
        if 'Beneficiary_Id'  in df_copiado.columns:
            df_copiado['Beneficiary_Id'] = df_copiado['Beneficiary_Id'].astype(str)
            condicion = df_copiado['Beneficiary_Id'].str.match('^[a-zA-Z]{2}\d+$', na=False) & pd.notna(df_copiado['Beneficiary_Id'])
            df_copiado.loc[condicion, 'Beneficiary_Account'] = df_copiado.loc[condicion, 'Beneficiary_Id']

    df_copiado['UETR'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_PmtId_UETR' in df_copiado.columns:
        fillna_if_exist(df_copiado, "UETR", ['CdtTrfTxInf_PmtId_UETR','TxInf_OrgnlUETR','TxInfAndSts_OrgnlUETR','Undrlyg_TxInf_OrgnlUETR'])
    elif 'TxInf_OrgnlUETR' in df_copiado.columns:
        fillna_if_exist(df_copiado, "UETR", ['TxInf_OrgnlUETR','TxInfAndSts_OrgnlUETR','Undrlyg_TxInf_OrgnlUETR'])
    elif 'TxInfAndSts_OrgnlUETR' in df_copiado.columns:
        fillna_if_exist(df_copiado, "UETR", ['TxInfAndSts_OrgnlUETR', 'Undrlyg_TxInf_OrgnlUETR'])
    elif 'Undrlyg_TxInf_OrgnlUETR' in df_copiado.columns:
        df_copiado["UETR"] = df_copiado['Undrlyg_TxInf_OrgnlUETR']

    df_copiado['InstructionId'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_PmtId_InstrId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "InstructionId", ['CdtTrfTxInf_PmtId_InstrId','Undrlyg_TxInf_CxlId','TxInf_RtrId','CxlDtls_TxInfAndSts_CxlStsId','GrpHdr_MsgId'])
    elif 'Undrlyg_TxInf_CxlId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "InstructionId", ['Undrlyg_TxInf_CxlId','TxInf_RtrId','CxlDtls_TxInfAndSts_CxlStsId','GrpHdr_MsgId'])
    elif 'TxInf_RtrId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "InstructionId", ['TxInf_RtrId','CxlDtls_TxInfAndSts_CxlStsId','GrpHdr_MsgId'])
    elif 'CxlDtls_TxInfAndSts_CxlStsId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "InstructionId", ['CxlDtls_TxInfAndSts_CxlStsId','GrpHdr_MsgId'])
    elif 'GrpHdr_MsgId' in df_copiado.columns:
        df_copiado["InstructionId"] = df_copiado['GrpHdr_MsgId']

    df_copiado['Related_Message_Id'] = np.nan  # Inicializar con NaN
    if 'TxInf_OrgnlGrpInf_OrgnlMsgId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Related_Message_Id", ['TxInf_OrgnlGrpInf_OrgnlMsgId','Undrlyg_TxInf_OrgnlGrpInf_OrgnlMsgId','TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','CxlDtls_TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','TxInf_OrgnlInstrId'])
    elif 'Undrlyg_TxInf_OrgnlGrpInf_OrgnlMsgId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Related_Message_Id", ['Undrlyg_TxInf_OrgnlGrpInf_OrgnlMsgId','TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','CxlDtls_TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','TxInf_OrgnlInstrId'])
    elif 'TxInfAndSts_OrgnlGrpInf_OrgnlMsgId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Related_Message_Id", ['TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','CxlDtls_TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','TxInf_OrgnlInstrId'])
    elif 'CxlDtls_TxInfAndSts_OrgnlGrpInf_OrgnlMsgId' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Related_Message_Id", ['CxlDtls_TxInfAndSts_OrgnlGrpInf_OrgnlMsgId','TxInf_OrgnlInstrId'])
    elif 'TxInf_OrgnlInstrId' in df_copiado.columns:
        df_copiado["Related_Message_Id"] = df_copiado['TxInf_OrgnlInstrId']

    df_copiado['Interbank_Settled_Amount'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_IntrBkSttlmAmt' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Interbank_Settled_Amount", ['CdtTrfTxInf_IntrBkSttlmAmt', 'TxInf_RtrdIntrBkSttlmAmt'])
    elif 'TxInf_RtrdIntrBkSttlmAmt' in df_copiado.columns:
        df_copiado["Interbank_Settled_Amount"] = df_copiado['TxInf_RtrdIntrBkSttlmAmt']

    if 'CxlDtls_TxInfAndSts_CxlStsRsnInf_Rsn_Cd' in df_copiado.columns:
        df_copiado["Narrative_CxlStsRsnInf"] = df_copiado['CxlDtls_TxInfAndSts_CxlStsRsnInf_Rsn_Cd']
    else:
        df_copiado["Narrative_CxlStsRsnInf"] = np.nan
    if 'CxlDtls_TxInfAndSts_CxlStsRsnInf_AddtlInf' in df_copiado.columns:
        df_copiado["Narrative_AddtlInf"] = df_copiado['CxlDtls_TxInfAndSts_CxlStsRsnInf_AddtlInf']
    else:
        df_copiado["Narrative_AddtlInf"] = np.nan
    df_copiado['Narrative_AddtlInf_CxlStsRsnInf_combinada'] = df_copiado['Narrative_CxlStsRsnInf'].fillna('')+' '+df_copiado['Narrative_AddtlInf'].fillna('')
    df_copiado['Narrative_AddtlInf_CxlStsRsnInf_combinada'] = df_copiado['Narrative_AddtlInf_CxlStsRsnInf_combinada'].str.strip()
    df_copiado['Narrative_AddtlInf_CxlStsRsnInf_combinada'] = df_copiado['Narrative_AddtlInf_CxlStsRsnInf_combinada'].replace('', np.nan)


    df_copiado['Narrative'] = np.nan  # Inicializar con NaN
    if 'TxInf_RtrRsnInf_Rsn_Cd' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Narrative", ['TxInf_RtrRsnInf_Rsn_Cd','TxInfAndSts_TxSts','Narrative_AddtlInf_CxlStsRsnInf_combinada','Undrlyg_TxInf_CxlRsnInf_Rsn_Cd'])
    elif 'TxInfAndSts_TxSts' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Narrative", ['TxInfAndSts_TxSts','Narrative_AddtlInf_CxlStsRsnInf_combinada','Undrlyg_TxInf_CxlRsnInf_Rsn_Cd'])
    elif 'Narrative_AddtlInf_CxlStsRsnInf_combinada' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Narrative", ['Narrative_AddtlInf_CxlStsRsnInf_combinada','Undrlyg_TxInf_CxlRsnInf_Rsn_Cd'])
    elif 'Undrlyg_TxInf_CxlRsnInf_Rsn_Cd' in df_copiado.columns:
        df_copiado["Narrative"] = df_copiado['Undrlyg_TxInf_CxlRsnInf_Rsn_Cd']

    df_copiado['Interbank_Settled_Currency'] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_IntrBkSttlmAmt_Ccy' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Interbank_Settled_Currency", ['CdtTrfTxInf_IntrBkSttlmAmt_Ccy', 'TxInf_RtrdIntrBkSttlmAmt_Ccy'])
    elif 'TxInf_RtrdIntrBkSttlmAmt_Ccy' in df_copiado.columns:
        df_copiado["Interbank_Settled_Currency"] = df_copiado['TxInf_RtrdIntrBkSttlmAmt_Ccy']

    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_Dbtr_CtctDtls')]
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_contact)
        df_copiado['CdtTrfTxInf_Dbtr_CtctDtls_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtctDtls')]
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_contact)
        df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtctDtls_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    df_copiado["Ordering_Customer_Id_Additional_Information"] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Dbtr_CtctDtls_combinada' in df_copiado.columns:
        df_copiado['CdtTrfTxInf_Dbtr_CtctDtls_combinada'] = df_copiado['CdtTrfTxInf_Dbtr_CtctDtls_combinada'].replace('', np.nan)
        fillna_if_exist(df_copiado, "Ordering_Customer_Id_Additional_Information", ['CdtTrfTxInf_Dbtr_CtctDtls_combinada', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtctDtls_combinada'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtctDtls_combinada' in df_copiado.columns:
        df_copiado["Ordering_Customer_Id_Additional_Information"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Dbtr_CtctDtls_combinada']

    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_Cdtr_CtctDtls')]
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_contact)
        df_copiado['CdtTrfTxInf_Cdtr_CtctDtls_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtctDtls')]
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_contact)
        df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtctDtls_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass
    df_copiado["Beneficiary_Additional_Information"] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Cdtr_CtctDtls_combinada' in df_copiado.columns:
        df_copiado['CdtTrfTxInf_Cdtr_CtctDtls_combinada'] = df_copiado['CdtTrfTxInf_Cdtr_CtctDtls_combinada'].replace('', np.nan)
        fillna_if_exist(df_copiado, "Beneficiary_Additional_Information", ['CdtTrfTxInf_Cdtr_CtctDtls_combinada', 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtctDtls_combinada'])
    elif 'CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtctDtls_combinada' in df_copiado.columns:
        df_copiado["Beneficiary_Additional_Information"] = df_copiado['CdtTrfTxInf_UndrlygCstmrCdtTrf_Cdtr_CtctDtls_combinada']

    df_copiado["Beneficiary_Institution"] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_CdtrAgt_FinInstnId_BICFI' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Institution", ['CdtTrfTxInf_CdtrAgt_FinInstnId_BICFI', 'CdtTrfTxInf_Cdtr_FinInstnId_BICFI'])
    elif 'CdtTrfTxInf_Cdtr_FinInstnId_BICFI' in df_copiado.columns:
        df_copiado["Beneficiary_Institution"] = df_copiado['CdtTrfTxInf_Cdtr_FinInstnId_BICFI']

    df_copiado["Beneficiary_Institution_Name"] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_CdtrAgt_FinInstnId_Nm' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Institution_Name", ['CdtTrfTxInf_CdtrAgt_FinInstnId_Nm', 'CdtTrfTxInf_Cdtr_FinInstnId_Nm'])
    elif 'CdtTrfTxInf_Cdtr_FinInstnId_Nm' in df_copiado.columns:
        df_copiado["Beneficiary_Institution_Name"] = df_copiado['CdtTrfTxInf_Cdtr_FinInstnId_Nm']

    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr') and col != 'CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass

    try:
        columnas_a_combinar = [col for col in df_copiado.columns if col.startswith('CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr') and col != 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine']
        columnas_a_combinar = ordenar_por_palabras_clave(columnas_a_combinar, palabras_clave_address)
        df_copiado['CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_combinada'] = df_copiado[columnas_a_combinar].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    except:
        pass

    df_copiado["Beneficiary_Institution_Address"] = np.nan  # Inicializar con NaN
    if 'CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Institution_Address", ['CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_AdrLine', 'CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_combinada','CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine','CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_combinada'])
    elif 'CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_combinada' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Institution_Address", ['CdtTrfTxInf_Cdtr_FinInstnId_PstlAdr_combinada','CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine','CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_combinada'])
    elif 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine' in df_copiado.columns:
        fillna_if_exist(df_copiado, "Beneficiary_Institution_Address", ['CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_AdrLine', 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_combinada'])
    elif 'CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_combinada' in df_copiado.columns:
        df_copiado["Beneficiary_Institution_Address"] = df_copiado['CdtTrfTxInf_CdtrAgt_FinInstnId_PstlAdr_combinada']

    df_copiado['Creation_date'] = df_copiado['Creation_date'].fillna(df_copiado['Filename'].apply(convertir_formato))

    df_copiado['Creation_date'] = df_copiado['Creation_date'].fillna(df_copiado['Filename'].apply(convertir_formato))
    mask = (df_copiado['Message_type'].str.startswith('pacs.009') &
            ~df_copiado['Message_type'].str.endswith('COV'))
    df_copiado.loc[mask, 'Account_With_Institution'] = np.nan

    columnas_deseadas = ['Filename','Message_Date','Message_Content','Creation_date', 'Direction', 'Sender_InstructingAgent_Requestor', 'Receiver_InstructedAgent_Responder', 'OwnBIC8', 'Own_Country', 'Role', 'Format', 'Message_type', 'InstructionId', 'End2EndId', 'UETR', 'Related_Message_Id', 'Transaction_Reference','Interbank_Settled_Date','Interbank_Settled_Currency', 'Interbank_Settled_Amount', 'Instructed_Currency', 'Instructed_Amount', 'Ordering_Customer_Account', 'Ordering_Customer_Name','Ordering_Customer_Id', 'Ordering_Customer_Address','Ordering_Customer_Address_Country','Ordering_Customer_Residence_Country', 'Ordering_Institution','Ordering_Institution_Name','Ordering_Institution_Address', 'Ordering_Institution_Name_and_Address', "Senders_Correspondent", "Senders_Correspondent_Name_and_Address", "Receivers_Correspondent", "Receivers_Correspondent_Name_and_Address", 'Third_Reimbursement_Institution', 'Third_Reimbursement_Institution_Name_and_Address', 'Intermediary_Institution', 'Account_With_Institution','Account_With_Institution_Address', 'Account_With_Institution_Name_and_Address', 'Beneficiary_Account', 'Beneficiary_Name','Beneficiary_Id', 'Beneficiary_Address','Beneficiary_Address_Country','Beneficiary_Residence_Country', 'Remittance_Information', 'Sender_to_Receiver_Information','Ordering_Customer_Date_Of_Birth','Ordering_Customer_Place_Of_Birth','Ordering_Customer_ID_Number','Ordering_Customer_National_ID_Number','Ordering_Customer_Id_Additional_Information','Narrative','Beneficiary_Institution','Beneficiary_Institution_Name','Beneficiary_Institution_Address','Beneficiary_Additional_Information']


    columnas_existen = [col for col in columnas_deseadas if col in df_copiado.columns]

    df_filtrado = df_copiado[columnas_existen]

    columnas_a_excluir = [columna for columna in df_filtrado.columns if 'date' in columna.lower()]
    columnas_a_excluir.extend(['Message_Content','Receivers_Reference','Senders_Reference','UETR'])

    for columna in df_filtrado.columns:
        if columna not in columnas_a_excluir:
            df_filtrado[columna] = df_filtrado[columna].apply(lambda x: str(x).replace('3/', ' ').replace('2/', ' ').replace('/', ' '))



    return df_filtrado

In [0]:
#MTFunctions
import re
import os


##########################################################################################################
## Funcion 1: FUNCIÓN PARA SACAR TODO EL CONTENIDO DE LOS MENSAJES MT
## ------------------------------------------------------------
## Con esta función ademas de sacar el contenido dividimos los tags en un diccionario
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##########################################################################################################

def parseMT(ruta_al_archivo):
    with open(ruta_al_archivo, 'r') as archivo:
        contenido = archivo.read()

        # Usamos expresiones regulares para encontrar y extraer cada tag
        resultado_tag1 = re.search(r'{1:(.*?)\}', contenido)
        resultado_tag2 = re.search(r'{2:(.*?)\}', contenido)
        resultado_tag3 = re.search(r'{3:(.*?)}\}', contenido)
        # Usamos una expresión regular para encontrar el tag4
        indice_inicio_tag4 = contenido.find("{4:") + 4
        indice_fin_tag4 = contenido.find("}", indice_inicio_tag4)

        if indice_inicio_tag4 != -1 and indice_fin_tag4 != -1:
            tag4 = contenido[indice_inicio_tag4:indice_fin_tag4]
        else:
            tag4 = "No se encontró información en el tag4"

        # Igual que el 1, 2 y 3
        resultado_tag5 = re.search(r'{5:(.*?)}\}', contenido)

        # Creamos un diccionario con los resultados y lo retornamos
        contenido_tag = {
            "tag1": resultado_tag1.group(1) if resultado_tag1 else "No se encontró información en el tag1",
            "tag2": resultado_tag2.group(1) if resultado_tag2 else "No se encontró información en el tag2",
            "tag3": resultado_tag3.group(1) + '}' if resultado_tag3 else "No se encontró información en el tag3",
            "tag4": tag4,
            "tag5": resultado_tag5.group(1) if resultado_tag5 else "No se encontró información en el tag5"
        }

        return contenido_tag, contenido

##########################################################################################################
## Funcion 2: FUNCIÓN PARA EXTRAER LOS CAMPOS DEL TAG 1
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##########################################################################################################

def parseMTTag1(contenido):
    resultados_tag1={}
    tag1 = contenido["tag1"]
    # Aplicar las funciones a tag1
    if tag1[0] == "F" or tag1[0] == "A" or tag1[0] == "L":
        AppID = tag1[0]
    else:
        AppID = 'ERROR'

    if tag1[1:3] == "01" or tag1[1:3] == "21":
        ServiceID = tag1[1:3]
    else:
        ServiceID = 'ERROR'

    if tag1[11].isalpha() and tag1[11].isupper():
        LTAddress = tag1[3:15]
    else:
        LTAddress = 'ERROR'

    if tag1[15:19].isdigit():
        SessionNumber = tag1[15:19]
    else:
        SessionNumber = 'ERROR'

    if tag1[19:].isdigit():
        SequenceNumber = tag1[19:]
    else:
        SequenceNumber = 'ERROR'

    # Almacenar los resultados en el diccionario resultados_tags
    resultados_tag1 = {
        "AppID": AppID,
        "ServiceID": ServiceID,
        "LTAddress": LTAddress,
        "SessionNumber": SessionNumber,
        "SequenceNumber": SequenceNumber
    }
    return resultados_tag1

##########################################################################################################
## Funcion 3: FUNCIÓN PARA EXTRAER LOS CAMPOS DEL TAG 2
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##########################################################################################################

def parseMTTag2(contenido):
    resultados_tag2 = {}
    tag2 = contenido["tag2"]

    if tag2[0]=='I':
        Input_Output= tag2[0]
    elif tag2[0]=='O':
        Input_Output= tag2[0]
    else:
        Input_Output=  'ERROR'

    if tag2[1:4].isdigit():
        SWIFT_Message_Type= tag2[1:4]
    else:
        SWIFT_Message_Type= 'ERROR'

    # CARACTERÍSTICAS ÚNICAS DEL IMPUT

    if tag2[0]=='I':
        Destination_Address= tag2[4:16]
    else:
        Destination_Address= 'ERROR'

    if tag2[0]=='I' and len(tag2)==17 and (tag2[16]=='S' or tag2[16]=='U' or tag2[16]=='N'):
        Priority_I= tag2[16]
    else:
        Priority_I= '---'

    if tag2[0]=='I' and len(tag2)==18 and (tag2[17]=='1' or tag2[17]=='2' or tag2[17]=='3'):
        Delivery_Monitoring= tag2[17]
    else:
        Delivery_Monitoring= '---'

    if tag2[0]=='I' and len(tag2)==21 and (tag2[17:20]=='003' or tag2[17:20]=='020'):
        Obsolescence_Period= tag2[17:20]
    else:
        Obsolescence_Period= '---'

    #CARACTERÍSTICAS ÚNICAS DEL OUTPUT

    if tag2[0]=='O' and tag2[4:8].isdigit():
        Input_Time= tag2[4:8]
    else:
        Input_Time= 'ERROR'

    if tag2[0]=='O' and tag2[8:14].isdigit() and tag2[27:37].isdigit():
        MIR= tag2[8:36]
    else:
        MIR= 'ERROR'

    if tag2[0]=='O' and tag2[36:42].isdigit():
        Output_Date= tag2[36:42]
    else:
        Output_Date= 'ERROR'

    if tag2[0]=='O' and tag2[42:46].isdigit():
        Output_Time= tag2[42:46]
    else:
        Output_Time= 'ERROR'

    if tag2[0]=='O' and len(tag2)==47:
        Priority= tag2[46] ## esto no se si es 47 o 46
    else:
        Priority= '---'

    # Almacenar los resultados en el diccionario resultados_tags
    if tag2[0]=='I':
        resultados_tag2 = {
            "Input_Output": Input_Output,
            "SWIFT_Message_Type": SWIFT_Message_Type,
            "Destination_Address": Destination_Address,
            "Priority_I": Priority_I,
            "Delivery_Monitoring": Delivery_Monitoring,
            "Obsolescence_Period": Obsolescence_Period
        }
    else:
        resultados_tag2 = {
            "Input_Output": Input_Output,
            "SWIFT_Message_Type": SWIFT_Message_Type,
            "Input_Time": Input_Time,
            "MIR": MIR,
            "Output_Date": Output_Date,
            "Output_Time": Output_Time,
            "Priority": Priority
        }
    # Aplicar el filtro para eliminar las claves con valor '---'
    resultados_tag2= {clave: valor for clave, valor in resultados_tag2.items() if valor != '---'}

    return resultados_tag2

##########################################################################################################
## Funcion 4: FUNCIÓN PARA EXTRAER LOS CAMPOS DEL TAG 3
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##########################################################################################################

def parseMTTag3(contenido):
    resultados_tag3 = {}
    tag3 = contenido["tag3"]

    Tag103_Service_Identifier = re.search(r'103:(.*?)\}', tag3)
    if Tag103_Service_Identifier:
        Tag103_Service_Identifier = '103:'+Tag103_Service_Identifier.group(1)
    else:
        Tag103_Service_Identifier= '---'

    Tag113_Banking_Priority = re.search(r'113:(.*?)\}', tag3)
    if Tag113_Banking_Priority:
        Tag113_Banking_Priority = '113:'+Tag113_Banking_Priority.group(1)
    else:
        Tag113_Banking_Priority= '---'

    Tag108_MUR = re.search(r'108:(.*?)\}', tag3)
    if Tag108_MUR:
        Tag108_MUR = '108:'+Tag108_MUR.group(1)
    else:
        Tag108_MUR= '---'

    Tag119_Validation_Flag = re.search(r'119:(.*?)\}', tag3)
    if Tag119_Validation_Flag:
        Tag119_Validation_Flag = '119:'+Tag119_Validation_Flag.group(1)
    else:
        Tag119_Validation_Flag= '---'

    Tag423_Balance_checkpoint_date_and_time= re.search(r'423:(.*?)\}', tag3)
    if Tag423_Balance_checkpoint_date_and_time:
        Tag423_Balance_checkpoint_date_and_time = '423:'+Tag423_Balance_checkpoint_date_and_time.group(1)
    else:
        Tag423_Balance_checkpoint_date_and_time= '---'

    Tag106_MIR = re.search(r'106:(.*?)\}', tag3)
    if Tag106_MIR:
        Tag106_MIR = '106:'+Tag106_MIR.group(1)
    else:
        Tag106_MIR= '---'

    Tag424_Related_reference = re.search(r'424:(.*?)\}', tag3)
    if Tag424_Related_reference:
        Tag424_Related_reference = '424:'+Tag424_Related_reference.group(1)
    else:
        Tag424_Related_reference= '---'

    Tag111_Service_type_identifier = re.search(r'111:(.*?)\}', tag3)
    if Tag111_Service_type_identifier:
        Tag111_Service_type_identifier = '111:'+Tag111_Service_type_identifier.group(1)
    else:
        Tag111_Service_type_identifier= '---'

    Tag121_Unique_end_to_end_transaction_reference = re.search(r'121:(.*?)\}', tag3)
    if Tag121_Unique_end_to_end_transaction_reference:
        Tag121_Unique_end_to_end_transaction_reference = '121:'+Tag121_Unique_end_to_end_transaction_reference.group(1)
    else:
        Tag121_Unique_end_to_end_transaction_reference= '---'

    Tag115_Addressee_Information = re.search(r'115:(.*?)\}', tag3)
    if Tag115_Addressee_Information:
        Tag115_Addressee_Information = '115:'+Tag115_Addressee_Information.group(1)
    else:
        Tag115_Addressee_Information= '---'

    Tag165_Payment_release_information_receiver = re.search(r'165:(.*?)\}', tag3)
    if Tag165_Payment_release_information_receiver:
        Tag165_Payment_release_information_receiver = '165:'+Tag165_Payment_release_information_receiver.group(1)
    else:
        Tag165_Payment_release_information_receiver= '---'

    Tag433_Sanctions_screening_information_for_receiver = re.search(r'433:(.*?)\}', tag3)
    if Tag433_Sanctions_screening_information_for_receiver:
        Tag433_Sanctions_screening_information_for_receiver = '433:'+Tag433_Sanctions_screening_information_for_receiver.group(1)
    else:
        Tag433_Sanctions_screening_information_for_receiver= '---'

    Tag434_Payment_controls_information_for_receiver = re.search(r'434:(.*?)\}', tag3)
    if Tag434_Payment_controls_information_for_receiver:
        Tag434_Payment_controls_information_for_receiver = '434:'+Tag434_Payment_controls_information_for_receiver.group(1)
    else:
        Tag434_Payment_controls_information_for_receiver= '---'

    # Almacenar los resultados en el diccionario resultados_tags
    resultados_tag3 = {
        "Service_Identifier_103": Tag103_Service_Identifier,
        "Banking_Priority_113": Tag113_Banking_Priority,
        "MUR_108": Tag108_MUR,
        "Validation_Flag_119": Tag119_Validation_Flag,
        "Balance_checkpoint_date_and_time_423": Tag423_Balance_checkpoint_date_and_time,
        "MIR_106": Tag106_MIR,
        "Related_reference_424": Tag424_Related_reference,
        "Service_type_identifier_111": Tag111_Service_type_identifier,
        "UETR": Tag121_Unique_end_to_end_transaction_reference,
        "Addressee_Information_115": Tag115_Addressee_Information,
        "Payment_release_information_receiver_165": Tag165_Payment_release_information_receiver,
        "Sanctions_screening_information_for_receiver_433": Tag433_Sanctions_screening_information_for_receiver,
        "Payment_controls_information_for_receiver_434": Tag434_Payment_controls_information_for_receiver
    }
    # Aplicar el filtro para eliminar las claves con valor '---'
    resultados_tag3= {clave: valor for clave, valor in resultados_tag3.items() if valor != '---'}
    return resultados_tag3

##############################################################################################################
## Funcion 5: FUNCIÓN PARA INDICAR COMO SE VA A LLAMAR CADA CAMPO DEL TAG4
## ------------------------------------------------------------
## Las listas que utiliza para definir este mapeo están en MT_properites.py
## Se aplica en la funcion ProcessMessage en initialFunctions.py y lo utilizaremos para la función parseMTTag4
##############################################################################################################

def selectMT(contenido):
    tag2 = contenido["tag2"]
    lista_subcategorias_valor=[]
    lista_subcategorias=[]
    # Obtiene el diccionario global del módulo miarchivo
    variables_globales = MT_properties.__dict__
    # Filtra las variables que son listas
    listas_en_miarchivo = {nombre: valor for nombre, valor in variables_globales.items() if isinstance(valor, list)}
    for nombre, lista in listas_en_miarchivo.items():
        if nombre == "lista_"+ tag2[1:4]:
            lista_subcategorias=lista
        elif nombre.startswith("lista_valor_"+tag2[1:4]):
            lista_subcategorias_valor=lista

    mapeo = dict(zip(lista_subcategorias_valor, lista_subcategorias))


    return lista_subcategorias,lista_subcategorias_valor,mapeo

##############################################################################################################
## Funcion 6: FUNCIÓN PARA EXTRAER LOS CAMPOS DEL TAG 4
## ------------------------------------------------------------
## Este tag es más complicado de extraer que los demás, por eso utilizaremos más de una función
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##############################################################################################################

def parseMTTag4(contenido,mapeo,lista_subcategorias_valor):
    tag4 = contenido["tag4"]
    resultados_por_archivo = {}
    if tag4!= 'No se encontró información en el tag4':
        # Expresión regular para capturar los valores
        expresion_regular = '|'.join(map(re.escape, lista_subcategorias_valor))
        expresion_regular = f'({expresion_regular})(.*?)(?={expresion_regular}|$)'
        matches = re.findall(expresion_regular, tag4, re.DOTALL)
        for match in matches:
            patron_inicial = match[0]
            if patron_inicial in mapeo:
                # Obtener el nombre original del mapeo
                nombre_original = mapeo[patron_inicial]

                # Construir la clave sin sufijo para la primera aparición
                nueva_clave = nombre_original

                # Verificar si ya hay un valor para esta clave, si no lo hay, agregar el valor
                if nueva_clave not in resultados_por_archivo:
                    resultados_por_archivo[nueva_clave] = match[1]
                else:
                    # Para las apariciones siguientes, agregar sufijo numérico
                    sufijo_numerico = 1
                    while f"{nombre_original}_{sufijo_numerico}" in resultados_por_archivo:
                        sufijo_numerico += 1
                    nueva_clave = f"{nombre_original}_{sufijo_numerico}"
                    resultados_por_archivo[nueva_clave] = match[1]

    return resultados_por_archivo, mapeo

#######################################################################################################################
## Funcion 7: FUNCIÓN PARA INDICAR COMO SE VA A LLAMAR CADA CAMPO DEL TAG4
## ------------------------------------------------------------
## Esta función sustituye las subcategorías en el diccionario correspondiente al tag 4 con sus nombres originales
## Se aplica en la funcion ProcessMessage en initialFunctions.py y lo utilizaremos para la función parseMTTag4
#######################################################################################################################

def reemplazar_subcategorias(diccionario, mapeo_subcategorias):
    diccionario_actualizado = {}
    for clave, valor in diccionario.items():
        if isinstance(valor, dict):
            # Si el valor es otro diccionario, llamar a la función de forma recursiva
            diccionario_actualizado[clave] = reemplazar_subcategorias(valor, mapeo_subcategorias)
        elif clave in mapeo_subcategorias:
            # Si la clave coincide con una subcategoría en el mapeo, reemplazarla
            nueva_clave = mapeo_subcategorias[clave]
            diccionario_actualizado[nueva_clave] = valor
        else:
            diccionario_actualizado[clave] = valor
    return diccionario_actualizado

##############################################################################################################
## Funcion 8: FUNCIÓN PARA EXTRAER LOS CAMPOS DEL TAG 5
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##############################################################################################################

def parseMTTag5(contenido):
    resultados_tag5={}
    tag5 = contenido["tag5"]

    CHK_Checksum = re.search(r'CHK:(.*?)\}', tag5)
    if CHK_Checksum:
        CHK_Checksum = 'CHK:'+CHK_Checksum.group(1)
    else:
        CHK_Checksum= '---'

    TNG_Test_Training_Message = re.search(r'TNG:(.*?)\}', tag5)
    if TNG_Test_Training_Message:
        TNG_Test_Training_Message = 'TNG:'+TNG_Test_Training_Message.group(1)
    else:
        TNG_Test_Training_Message= '---'

    PDE_Possible_Duplicate_Emission = re.search(r'PDE:(.*?)\}', tag5)
    if PDE_Possible_Duplicate_Emission:
        PDE_Possible_Duplicate_Emission = 'PDE:'+PDE_Possible_Duplicate_Emission.group(1)
    else:
        PDE_Possible_Duplicate_Emission= '---'

    DLM_Delayed_Message = re.search(r'DLM:(.*?)\}', tag5)
    if DLM_Delayed_Message:
        DLM_Delayed_Message = 'DLM:'+DLM_Delayed_Message.group(1)
    else:
        DLM_Delayed_Message= '---'

    MRF_Message_Reference = re.search(r'MRF:(.*?)\}', tag5)
    if MRF_Message_Reference:
        MRF_Message_Reference = 'MRF:'+MRF_Message_Reference.group(1)
    else:
        MRF_Message_Reference= '---'

    PDM_Possible_Duplicate_Message = re.search(r'PDM:(.*?)\}', tag5)
    if PDM_Possible_Duplicate_Message:
        PDM_Possible_Duplicate_Message = 'PDM:'+PDM_Possible_Duplicate_Message.group(1)
    else:
        PDM_Possible_Duplicate_Message= '---'

    SYS_System_Originated_Message = re.search(r'SYS:(.*?)\}', tag5)
    if SYS_System_Originated_Message:
        SYS_System_Originated_Message = 'SYS:'+SYS_System_Originated_Message.group(1)
    else:
        SYS_System_Originated_Message= '---'

    # Almacenar los resultados en el diccionario resultados_tags
    resultados_tag5 = {
        "CHK_Checksum": CHK_Checksum,
        "TNG_Test_Training_Message": TNG_Test_Training_Message,
        "PDE_Possible_Duplicate_Emission": PDE_Possible_Duplicate_Emission,
        "DLM_Delayed_Message": DLM_Delayed_Message,
        "MRF_Message_Reference": MRF_Message_Reference,
        "PDM_Possible_Duplicate_Message": PDM_Possible_Duplicate_Message,
        "SYS_System_Originated_Message": SYS_System_Originated_Message
    }
    # Aplicar el filtro para eliminar las claves con valor '---'
    resultados_tag5= {clave: valor for clave, valor in resultados_tag5.items() if valor != '---'}
    return resultados_tag5

##############################################################################################################
## Funcion 9: FUNCIÓN PARA COMBINAR TODOS LOS TAGS EN UN SOLO DICCIONARIO
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage en initialFunctions.py
##############################################################################################################

def join_all_tags(resultados_tag1_MT,resultados_tag2_MT,resultados_tag3_MT,resultados_tag4_MT,resultados_tag5_MT):
    lista_de_diccionarios_SWIFT_MT=[resultados_tag1_MT,resultados_tag2_MT,resultados_tag3_MT,resultados_tag4_MT,resultados_tag5_MT]
    diccionario_SWIFT_MT = {}
    for diccionario in lista_de_diccionarios_SWIFT_MT:
        for clave, valor in diccionario.items():
            if clave in diccionario_SWIFT_MT:
                # Si la clave ya existe, actualiza el valor con el nuevo diccionario
                diccionario_SWIFT_MT[clave].update(valor)
            else:
                # Si la clave no existe, crea una nueva entrada en el diccionario combinado
                diccionario_SWIFT_MT[clave] = valor
    return diccionario_SWIFT_MT

import pandas as pd
import numpy as np

##############################################################################################################
## Funcion 10: FUNCIÓN PARA SEPARAR LOS MENSAJES MT 101 EN VARIOS MENSAJES
## ------------------------------------------------------------
## Se aplica en SWIFT_parsing.py
##############################################################################################################

def separar_mensajes_101(df):
    df_duplicated = pd.DataFrame(columns=df.columns)
    for index, row in df.iterrows():
        if row['SWIFT_Message_Type'] == '101':
            for i in range(1, 8):  # Iterar hasta la columna _7
                col_name = f'Transaction_Reference_21_{i}'
                if col_name in df and not pd.isna(row[col_name]):  # Si la columna está informada
                    duplicated_row = row.copy()  # Copiar la fila original
                    duplicated_row['Filename'] += f'_{i}'  # Cambiar el valor de la columna 'Archivo'
                    df_duplicated = pd.concat([df_duplicated, duplicated_row.to_frame().T], ignore_index=True)
    df_duplicated = pd.concat([df, df_duplicated],ignore_index=True)
    nombres_filas_con_1 = [nombre[:-2] for nombre in df_duplicated[df_duplicated['Filename'].str.endswith('_1')]['Filename']]
    if nombres_filas_con_1:
        mask = df_duplicated['Filename'].isin(nombres_filas_con_1)
        columnas_seleccionadas = df_duplicated.columns[df_duplicated.columns.get_loc('Transaction_Reference_21_1'):]
        df_duplicated.loc[mask, columnas_seleccionadas] = np.nan
        for i in range(1,5):
            try:
                mask = df_duplicated['Filename'].str.endswith(f'_{i}')
                columnas_seleccionadas = list(df_duplicated.columns[df_duplicated.columns.get_loc('Transaction_Reference_21'): df_duplicated.columns.get_loc(f'Transaction_Reference_21_{i}')])
                if f'Transaction_Reference_21_{i+1}' in df_duplicated.columns:
                    columnas_nuevas = df_duplicated.columns[df_duplicated.columns.get_loc(f'Transaction_Reference_21_{i+1}'):]
                    columnas_seleccionadas.extend(columnas_nuevas)
                    df_duplicated.loc[mask, columnas_seleccionadas] = np.nan
                else:
                    df_duplicated.loc[mask, columnas_seleccionadas] = np.nan
            except:
                pass
    return df_duplicated

In [0]:
#MXFunctions
import xml.etree.ElementTree as ET

####################################################################################################################################################
## Funcion 1: FUNCIÓN PARA SACAR TODO EL CONTENIDO DE LOS MENSAJES MX 
## ------------------------------------------------------------
## Esta función recibe un elemento XML y devuelve un diccionario que contiene los valores junto con sus etiquetas y atributos correspondientes
## Recorre de manera recursiva la estructura del XML, extrayendo valores de texto, nombres de etiquetas y atributos, y los almacena en un diccionario.
## Se aplica en la funcion ProcessMessage en initialFunctions.py
#####################################################################################################################################################

def extract_values_with_tags_and_attributes(element, parent_tags=None):
    if parent_tags is None:
        parent_tags = []

    result = {}

    # Handle XML namespaces
    ns_prefix = element.tag.split('}')[0][1:]
    ns = {'': ns_prefix}

    for child in element:
        child_tags = parent_tags + [child.tag.replace('{' + ns_prefix + '}', '')]
        child_name = '_'.join(child_tags)

        # Eliminar prefijos 'Header_AppHdr_' y 'Document_'
        child_name = child_name.replace('Header_', '').replace('Document_', '').replace('AppHdr_', '')

        if 'Body_{' in child_name:
            child_name = child_name.split('Body_{')[0] + child_name.split('}')[1]
        # Extract attributes
        for attr_key, attr_value in child.attrib.items():
            attr_name = f"{child_name}_{attr_key}"
            result[attr_name] = attr_value

        # Recursively call the function for child elements
        result.update(extract_values_with_tags_and_attributes(child, child_tags))

        # Extract text content if present
        if child.text is not None and child.text.strip() != "":
            text_name = f"{child_name}"
            try:
                # Intentar agregar el valor a la lista existente
                result[text_name].append(child.text.strip())
            except KeyError:
                # Si la clave no existe, crearla con el valor en una lista
                result[text_name] = [child.text.strip()]

    return result

####################################################################################################################################################
## Funcion 2: FUNCIÓN PARA "APLANAR" LAS LISTAS DENTRO DEL DICCIONARIO  
## ------------------------------------------------------------
## Si encuentra algún valor que sea una lista, convierte esa lista en una cadena de texto concatenando todos los elementos de la lista con un espacio como separador
## Se aplica en la funcion ProcessMessage en initialFunctions.py
#####################################################################################################################################################

def flatten_lists(result):
    flattened_result = {}

    for key, value in result.items():
        if isinstance(value, list):
            # Si el valor es una lista, convierte a una cadena uniendo los elementos con un espacio
            flattened_result[key] = ' '.join(map(str, value))
        else:
            # Si no es una lista, deja el valor como está
            flattened_result[key] = value

    return flattened_result

####################################################################################################################################################
## Funcion 3: FUNCIÓN PARA SACAR EL CONTENIDO COMPLETO DEL MENSAJE   
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage en initialFunctions.py
#####################################################################################################################################################

def obtener_contenido(elemento):
    contenido = ET.tostring(elemento, encoding='unicode', method='xml')
    return ''.join(contenido.split())

In [0]:
#MT_Properties

class MT_properties:
    lista_101= ['Senders_Reference_20','Customer_Specified_Reference_21R','Message_Index/Total_28D','Instructing_Party_50C','Instructing_Party_50L','Ordering_Customer_50F','Ordering_Customer_50G','Ordering_Customer_50H','Account_Servicing_Institution_52A','Account_Servicing_Institution_52C','Sending_Institution_51A','Requested_Execution_Date_30','Authorisation_25',
    'Transaction_Reference_21','F/X_Deal_Reference_21F','Instruction_Code_23E','Currency/Transaction_Amount_32B','Intermediary_56A','Intermediary_56C','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57C','Account_With_Institution_57D','Beneficiary_59','Beneficiary_59A','Beneficiary_59F','Remittance_Information_70','Regulatory_Reporting_77B',
    'Currency/Original_Ordered_Amount_33B','Details_of_Charges_71A','Charges_Account_25A','Exchange_Rate_36']
    lista_valor_101= [':20:',':21R:',':28D:',':50C:',':50L:',':50F:',':50G:',':50H:',':52A:',':52C:',':51A:',':30:',':25:',':21:',':21F:',':23E:',':32B:',':56A:',':56C:',':56D:',':57A:',':57C:',':57D:',':59:',':59A:',':59F:',':70:',':77B:',':33B:',':71A:',':25A:',':36:']
    
    lista_102=['File_Reference_20','Bank_Operation_Code_23','Sending_Institution_51A','Ordering_Customer_50A','Ordering_Customer_50F','Ordering_Customer_50K','Ordering_Institution_52A','Ordering_Institution_52B','Ordering_Institution_52C','Transaction_Type_Code_26T','Regulatory_Reporting_77B','Details_of_Charges_71A',
    'Exchange_Rate_36','Transaction_Reference_21','Transaction_Amount_32B','Account_With_Institution_57A','Account_With_Institution_57C','Beneficiary_Customer_59','Beneficiary_Customer_59A','Beneficiary_Customer_59F','Remittance_Information_70','Currency/Instructed_Amount_33B','Senders_Charges_71F','Receivers_Charges_71G','Value_Date_Currency_Code_Amount_32A',
    'Sum_of_Amounts_19','Sum_of_Receivers_Charges_71G','Time_Indication_13C','Senders_Correspondent_53A','Senders_Correspondent_53C','Receivers_Correspondent_54A','Sender_to_Receiver_Information_72']
    lista_valor_102= [':20:',':23:',':51A:',':50A:',':50F:',':50K:',':52A:',':52B:',':52C:',':26T:',':77B:',':71A:',':36:',':21:',':32B:',':57A:',':57C:',':59:',':59A:',':59F:',':70:',':33B:',':71F:',':71G:',':32A:',':19:',':71G:',':13C:',':53A:',':53C:',':54A:',':72:']
    
    lista_103 = ['Senders_Reference_20', 'Time_Indication_13C','Bank_Operation_Code_23B','Instruction_Code_23E','Transaction_Type_Code_26T','Value_Date/Currency/Interbank_Settled_Amount_32A','Currency/Instructed_Amount_33B','Exchange_Rate_36','Ordering_Customer_50A','Ordering_Customer_50F','Ordering_Customer_50K',
    'Sending_Institution_51A','Ordering_Institution_52A','Ordering_Institution_52D','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receiver_Correspondent_54B','Receiver_Correspondent_54D','Third_Reimbursement_Institution_55A','Third_Reimbursement_Institution_55B','Third_Reimbursement_Institution_55D',
    'Intermediary_Institution_56A','Intermediary_Institution_56C','Intermediary_Institution_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57C','Account_With_Institution_57D','Beneficiary_Customer_59','Beneficiary_Customer_59A','Beneficiary_Customer_59F','Remittance_Information_70',
    'Details_of_Charges_71A','Sender_Charges_71F','Receiver_Charges_71G','Sender_to_Receiver_Information_72','Regulatory_Reporting_77B','Envelope_Contents_77T']
    lista_valor_103 = [':20:', ':13C:',':23B:',':23E:',':26T:',':32A:',':33B:',':36:',':50A:',':50F:',':50K:',':51A:',':52A:',':52D:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':55A:',':55B:',':55D:',':56A:',':56C:',':56D:',':57A:',':57B:',':57C:',':57D:',':59:',':59A:',':59F:',':70:',':71A:',':71F:',':71G:',':72:',':77B:',':77T:']
    
    lista_104 = ['Senders_Reference_20','Customer_Specified_Reference_21R','Instruction_Code_23E','Registration_Reference_21E','Requested_Execution_Date_30','Sending_Institution_51A','Instructing_Party_50C','Instructing_Party_50L','Creditor_50A','Creditor_50K','Creditors_Bank_52A','Creditors_Bank_52C','Creditors_Bank_52D','Transaction_Type_Code_26T','Regulatory_Reporting_77B','Details_of_Charges_71A',
    'Sender_to_Receiver_Information_72','Transaction_Reference_21','Mandate_Reference_21C','Direct_Debit_Reference_21D','Currency_and_Transaction_Amount_32B','Debtors_Bank_57A','Debtors_Bank_57C','Debtors_Bank_57D','Debtor_59','Debtor_59A','Remittance_Information_70','Currency/Original_Ordered_Amount_33B','Senders_Charges_71F','Receivers_Charges_71G','Exchange_Rate_36','Currency_and_Settlement_Amount_32B','Sum_of_Amounts_19','Sum_of_Senders_Charges_71F','Sum_of_Receivers_Charges_71G','Senders_Correspondent_53A','Senders_Correspondent_53B']
    lista_valor_104 = [':20:',':21R:',':23E:',':21E:',':30:',':51A:',':50C:',':50L:',':50A:',':50K:',':52A:',':52C:',':52D:',':26T:',':77B:',':71A:',':72:',':21:',':21C:',':21D:',':32B:',':57A:',':57C:',':57D:',':59:',':59A:',':70:',':33B:',':71F:',':71G:',':36:',':32B:',':19:',':71F:',':71G:',':53A:',':53B:']
    
    lista_105 = ['Sequence_of_Total_27','Transaction_Reference_Number_20','Related_Reference_21','Sub-Message_Type_12','EDIFACT_Message_77F']
    lista_valor_105= [':27:',':20:',':21:',':12:',':77F:']
    
    lista_107 = ['Senders_Reference_20','Instruction_Code_23E','Registration_Reference_21E','Requested_Execution_Date_30','Sending_Institution_51A','Instructing_Party_50C','Instructing_Party_50L','Creditor_50A','Creditor_50K','Creditors_Bank_52A','Creditors_Bank_52C','Creditors_Bank_52D','Transaction_Type_Code_26T','Regulatory_Reporting_77B','Details_of_Charges_71A','Sender_to_Receiver_Information_72','Transaction_Reference_21','Mandate_Reference_21C','Direct_Debit_Reference_21D','Currency_and_Transaction_Amount_32B','Debtors_Bank_57A',
    'Debtors_Bank_57C','Debtors_Bank_57D','Debtor_59','Debtor_59A','Remittance_Information_70','Currency/Original_Ordered_Amount_33B','Senders_Charges_71F','Receivers_Charges_71G','Exchange_Rate_36','Currency_and_Settlement_Amount_32B','Sum_of_Amounts_19','Sum_of_Senders_Charges_71F','Sum_of_Receivers_Charges_71G','Senders_Correspondent_53A','Senders_Correspondent_53B']
    lista_valor_107= [':20:',':23E:',':21E:',':30:',':51A:',':50C:',':50L:',':50A:',':50K:',':52A:',':52C:',':52D:',':26T:',':77B:',':71A:',':72:',':21:',':21C:',':21D:',':32B:',':57A:',':57C:',':57D:',':59:',':59A:',':70:',':33B:',':71F:',':71G:',':36:',':32B:',':19:',':71F:',':71G:',':53A:',':53B:']
    
    ista_110= ['Senders_Reference_20','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D','Sender_to_Receiver_Information_72','Cheque_Number_21','Date_of_Issue_30','Amount_32A','Amount_32B',
    'Payer_50A','Payer_50F','Payer_50K','Drawer_Bank_52A','Drawer_Bank_52B','Drawer_Bank_52D','Payee_59','Payee_59F',]
    lista_valor_110= [':20:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':72:',':21:',':30:',':32A:',':32B:',':50A:',':50F:',':50K:',':52A:',':52B:',':52D:',':59:',':59F:']
    
    lista_111 = ['Senders_Reference_20','Cheque_Number_21','Date_of_Issue_30','Amount_32A','Amount_32B','Drawer_Bank_52A','Drawer_Bank_52B','Drawer_Bank_52D','Payee_59','Queries_75']
    lista_valor_111= [':20:',':21:',':30:',':32A:',':32B:',':52A:',':52B:',':52D:',':59:',':75:']
    
    lista_112 = ['Transaction_Reference_Number_20','Cheque_Number_21','Date_of_Issue_30','Amount_32A','Amount_32B','Drawer_Bank_52A','Drawer_Bank_52B','Drawer_Bank_52D','Payee_59','Answers_76']
    lista_valor_112= [':20:',':21:',':30:',':32A:',':32B:',':52A:',':52B:',':52D:',':59:',':76:']
    
    lista_190= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Value_Date_Currency_Code_Amount_32C','Value_Date_Currency_Code_Amount_32D','Ordering_Institution_52A','Ordering_Institution_52D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_190= [':20:',':21:',':25:',':32C:',':32D:',':52A:',':52D:',':71B:',':72:']
    
    lista_191= ['Transaction_Reference_Number_20','Related_Reference_21','Currency_Code_Amount_32B','Ordering_Institution_52A','Ordering_Institution_52D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_191= [':20:',':21:',':32B:',':52A:',':52D:',':57A:',':57B:',':57D:',':71B:',':72:']
    
    lista_192= ['Transaction_Reference_Number_20','Related_Reference_21','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_79']
    lista_valor_192= [':20:',':21:',':11S:',':79:']
    
    lista_195= ['Transaction_Reference_Number_20','Related_Reference_21','Queries_75','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Message_to_Which_the_Query_Relates_79']
    lista_valor_195= [':20:',':21:',':75:',':77A:',':11R:',':11S:',':79:']
    
    lista_196= ['Transaction_Reference_Number_20','Related_Reference_21','Answers_76','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_to_Which_the_Answer_Relates_79']
    lista_valor_196= [':20:',':21:',':76:',':77A:',':11R:',':11S:',':79:']
    
    lista_198= ['Transaction_Reference_Number_20','Sub-Message_Type_12','Proprietary_Message_77E']
    lista_valor_198= [':20:',':12:',':77E:']
    
    lista_199= ['Transaction_Reference_Number_20','Related_Reference_21','Narrative_79']
    lista_valor_199= [':20:',':21:',':79:']
    
    lista_200 = ['Transaction_Reference_Number_20','Value_Date_Currency_Code_Amount_32A','Senders_Correspondent_53B','Intermediary_56A','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Sender_to_Receiver_Information_72']
    lista_valor_200= [':20:',':32A:',':53B:',':56A:',':56D:',':57A:',':57B:',':57D:',':72:']
    
    lista_201 = ['Sum_of_Amounts_19','Value_Date_30','Senders_Correspondent_53B','Sender_to_Receiver_Information_72','Transaction_Reference_Number_20','Currency_Code_Amount_32B','Intermediary_56A','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D']
    lista_valor_201= [':19:',':30:',':53B:',':72:',':20:',':32B:',':56A:',':56D:',':57A:',':57B:',':57D:']
    
    lista_202 = ['Transaction_Reference_Number_20','Related_Reference_21','Time_Indication_13C','Value_Date_Currency_Code_Amount_32A','Ordering_Institution_52A','Ordering_Institution_52D','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D','Intermediary_56A','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Beneficiary_Institution_58A','Beneficiary_Institution_58D','Sender_to_Receiver_Information_72','Ordering_Customer_50A',
    'Ordering_Customer_50F','Ordering_Customer_50K','Intermediary_Institution_56A','Intermediary_Institution_56C','Intermediary_Institution_56D','Account_With_Institution_57C','Beneficiary_Customer_59','Beneficiary_Customer_59A','Beneficiary_Customer_59F','Remittance_Information_70','Currency/Instructed_Amount_33B']
    lista_valor_202= [':20:',':21:',':13C:',':32A:',':52A:',':52D:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':56A:',':56D:',':57A:',':57B:',':57D:',':58A:',':58D:',':72:',':50A:',':50F:',':50K:',':56A:',':56C:',':56D:',':57C:',':59:',':59A:',':59F:',':70:',':33B:']
    
    lista_203= ['Sum_of_Amounts_19','Value_Date_30','Ordering_Institution_52A','Ordering_Institution_52D','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D','Sender_to_Receiver_Information_72',
    'Transaction_Reference_Number_20','Related_Reference_21','Currency_Code_Amount_32B','Intermediary_56A','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Beneficiary_Institution_58A','Beneficiary_Institution_58D']
    lista_valor_203= [':19:',':30:',':52A:',':52D:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':72:',':20:',':21:',':32B:',':56A:',':56D:',':57A:',':57B:',':58A:',':58D:']
    
    lista_204 = ['Transaction_Reference_Number_20','Sum_of_Amounts_19','Value_Date_30','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Beneficiary_Institution_58A','Beneficiary_Institution_58D','Sender_to_Receiver_Information_72','Related_Reference_21','Transaction_Amount_32B','Debit_Institution_53A','Debit_Institution_53B','Debit_Institution_53D']
    lista_valor_204= [':20:',':19:',':30:',':57A:',':57B:',':57D:',':58A:',':58D:',':72:',':21:',':32B:',':53A:',':53B:',':53D:']
    
    lista_205 = ['Transaction_Reference_Number_20','Related_Reference_21','Time_Indication_13C','Value_Date_Currency_Code_Amount_32A','Ordering_Institution_52A','Ordering_Institution_52D','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Intermediary_56A','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Beneficiary_Institution_58A','Beneficiary_Institution_58D',
    'Sender_to_Receiver_Information_72','Ordering_Customer_50A','Ordering_Customer_50F','Ordering_Customer_50K','Intermediary_Institution_56A','Intermediary_Institution_56C','Intermediary_Institution_56D','Account_With_Institution_57C','Beneficiary_Customer_59','Beneficiary_Customer_59A','Beneficiary_Customer_59F','Remittance_Information_70','Currency/Instructed_Amount_33B']
    lista_valor_205= [':20:',':21:',':13C:',':32A:',':52A:',':52D:',':53A:',':53B:',':53D:',':56A:',':56D:',':57A:',':57B:',':57D:',':58A:',':58D:',':72:',':50A:',':50F:',':50K:',':56A:',':56C:',':56D:',':57C:',':59:',':59A:',':59F:',':70:',':33B:']
    
    lista_210 = ['Transaction_Reference_Number_20','Account_Identification_25','Value_Date_30','Related_Reference_21','Currency_Code_Amount_32B','Ordering_Customer_50','Ordering_Customer_50C','Ordering_Customer_50F','Ordering_Institution_52A','Ordering_Institution_52D','Intermediary_56A','Intermediary_56D']
    lista_valor_210= [':20:',':25:',':30:',':21:',':32B:',':50:',':50C:',':50F:',':52A:',':52D:',':56A:',':56D:']
    
    lista_290= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Value_Date_Currency_Code_Amount_32C','Value_Date_Currency_Code_Amount_32D','Ordering_Institution_52A','Ordering_Institution_52D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_290= [':20:',':21:',':25:',':32C:',':32D:',':52A:',':52D:',':71B:',':72:']
    
    lista_291= ['Transaction_Reference_Number_20','Related_Reference_21','Currency_Code_Amount_32B','Ordering_Institution_52A','Ordering_Institution_52D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_291= [':20:',':21:',':32B:',':52A:',':52D:',':57A:',':57B:',':57D:',':71B:',':72:']
    
    lista_292= ['Transaction_Reference_Number_20','Related_Reference_21','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_79']
    lista_valor_292= [':20:',':21:',':11S:',':79:']
    
    lista_295= ['Transaction_Reference_Number_20','Related_Reference_21','Queries_75','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Message_to_Which_the_Query_Relates_79']
    lista_valor_295= [':20:',':21:',':75:',':77A:',':11R:',':11S:',':79:']
    
    lista_296= ['Transaction_Reference_Number_20','Related_Reference_21','Answers_76','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_to_Which_the_Answer_Relates_79']
    lista_valor_296= [':20:',':21:',':76:',':77A:',':11R:',':11S:',':79:']
    
    lista_298= ['Transaction_Reference_Number_20','Sub-Message_Type_12','Proprietary_Message_77E']
    lista_valor_298= [':20:',':12:',':77E:']
    
    lista_299= ['Transaction_Reference_Number_20','Related_Reference_21','Narrative_79']
    lista_valor_299= [':20:',':21:',':79:']
    
    lista_300= ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Common_Reference_22C','Block_Trade_Indicator_17T','Split_Settlement_Indicator_17U','Payment_versus_Payment_Settlement_Indicator_17I','Party_A_82A','Party_A_82J','Party_B_87A','Party_B_87J','Fund_or_Beneficiary_Customer_83A','Fund_or_Beneficiary_Customer_83J',
    'Type_Date_Version_of_the_Agreement_77H','Terms_and_Conditions_77D','Year_of_Definitions_14C','Non-Deliverable_Indicator_17F','NDF_Open_Indicator_17O','Settlement_Currency_32E','Valuation_Date_30U','Settlement_Rate_Source_14S','Calculation_Agent_26K','Reference_to_Opening_Confirmation_21A','Clearing_or_Settlement_Session_14E','New_Sequence_15B','Trade_Date_30T','Value_Date_30V','Exchange_Rate_36',
    'Payment_Clearing_Centre_39M','Digital_Token_Identifier_35C','Currency_Amount_32B','Delivery_Agent_53A','Delivery_Agent_53J','Intermediary_56A','Intermediary_56J','Receiving_Agent_57A','Receiving_Agent_57J','Currency_Amount_33B','Beneficiary_Institution_58A','Beneficiary_Institution_58J','New_Sequence_15C','Contact_Information_29A','Dealing_Method_24D','Dealing_Branch_Party_A_84A','Dealing_Branch_Party_A_84B',
    'Dealing_Branch_Party_A_84D','Dealing_Branch_Party_A_84J','Dealing_Branch_Party_B_85A','Dealing_Branch_Party_B_85B','Dealing_Branch_Party_B_85D','Dealing_Branch_Party_B_85J','Broker_Identification_88A','Broker_Identification_88D','Broker_Identification_88J','Brokers_Commission_71F','Counterpartys_Reference_26H','Brokers_Reference_21G','Sender_to_Receiver_Information_72','New_Sequence_15D','Buy_(Sell)_Indicator_17A','Delivery_Agent_53D',
    'Intermediary_56D','Receiving_Agent_57D','Beneficiary_Institution_58D','Number_of_Settlements_16A','New_Sequence_15E','Reporting_Jurisdiction_22L','Reporting_Party_91A','Reporting_Party_91D','Reporting_Party_91J','UTI_Namespace/Issuer_Code_22M','Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Central_Counterparty_Clearing_House_(CCP)_81A','Central_Counterparty_Clearing_House_(CCP)_81D','Central_Counterparty_Clearing_House_(CCP)_81J',
    'Clearing_Broker_89A','Clearing_Broker_89D','Clearing_Broker_89J','Clearing_Exception_Party_96A','Clearing_Exception_Party_96D','Clearing_Exception_Party_96J','Clearing_Broker_Identification_22S','Cleared_Product_Identification_22T','Clearing_Threshold_Indicator_17E','Underlying_Product_Identifier_22U','Identification_of_Financial_Instrument_35B','Allocation_Indicator_17H','Collateralisation_Indicator_17P','Execution_Venue_22V','Execution_Timestamp_98D','Non_Standard_Flag_17W','Link_Swap_Identification_22W',
    'Financial_Nature_of_the_Counterparty_Indicator_17Y','Collateral_Portfolio_Indicator_17Z','Collateral_Portfolio_Code_22Q','Portfolio_Compression_Indicator_17L','Corporate_Sector_Indicator_17M','Trade_with_Non-EEA_Counterparty_Indicator_17Q','Intragroup_Trade_Indicator_17S','Commercial_or_Treasury_Financing_Indicator_17X','Confirmation_Timestamp_98G','Clearing_Timestamp_98H','Commission_and_Fees_34C','Additional_Reporting_Information_77A','New_Sequence_15F','Event_Type_and_Reference_21H','Underlying_Liability_Reference_21F',
    'Profit_and_Loss_Settlement_Date_30F','Profit_and_Loss_Settlement_Amount_32H','Outstanding_Settlement_Amount_33E']
    lista_valor_300= [':15A:',':20:',':21:',':22A:',':94A:',':22C:',':17T:',':17U:',':17I:',':82A:',':82J:',':87A:',':87J:',':83A:',':83J:',':77H:',':77D:',':14C:',':17F:',':17O:',':32E:',':30U:',':14S:',':26K:',':21A:',':14E:',':15B:',':30T:',':30V:',':36:',':39M:',':35C:',':32B:',':53A:',':53J:',':56A:',':56J:',':57A:',':57J:',':33B:',':58A:',':58J:',':15C:',':29A:',':24D:',':84A:',':84B:',':84D:',':84J:',':85A:',':85B:',':85D:',':85J:',':88A:',':88D:',':88J:',':71F:',':26H:',':21G:',':72:',':15D:',':17A:',':53D:',':56D:',':57D:',
    ':58D:',':16A:',':15E:',':22L:',':91A:',':91D:',':91J:',':22M:',':22N:',':22P:',':22R:',':81A:',':81D:',':81J:',':89A:',':89D:',':89J:',':96A:',':96D:',':96J:',':22S:',':22T:',':17E:',':22U:',':35B:',':17H:',':17P:',':22V:',':98D:',':17W:',':22W:',':17Y:',':17Z:',':22Q:',':17L:',':17M:',':17Q:',':17S:',':17X:',':98G:',':98H:',':34C:',':77A:',':15F:',':21H:',':21F:',':30F:',':32H:',':33E:']
    
    lista_304 = ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Open_Indicator_17O','Final_Close_Indicator_17F','Net_Settlement_Indicator_17N','Fund_83A','Fund_83J','Fund_Manager_82A','Fund_Manager_82J','Executing_Broker_87A','Executing_Broker_87J','Central_Counterparty_Clearing_House_CCP_81A','Central_Counterparty_Clearing_House_CCP_81D','Central_Counterparty_Clearing_House_CCP_81J','Clearing_Broker_89A','Clearing_Broker_89D','Clearing_Broker_89J','Payment_versus_Payment_Settlement_Indicator_17I',
    'Type_Date_Version_of_the_Agreement_77H','Year_of_Definitions_14C','Settlement_Currency_32E','Valuation_Date_30U','Settlement_Rate_Source_14S','Calculation_Agent_26K','Reference_to_Opening_Instruction_21A','Clearing_or_Settlement_Session_14E','New_Sequence_15B','Trade_Date_30T','Value_Date_30V','Exchange_Rate_36','Payment_Clearing_Centre_39M','Digital_Token_Identifier_35C','Currency_Amount_Bought_32B','Delivery_Agent_53A','Delivery_Agent_53J','Intermediary_56A','Intermediary_56J','Receiving_Agent_57A','Receiving_Agent_57J','Currency_Amount_Sold_33B','Beneficiary_Institution_58A',
    'Beneficiary_Institution_58J','New_Sequence_15C','Reference_to_the_Associated_Trade_21A','Executing_Brokers_Reference_21G','Reporting_Jurisdiction_22L','UTI_Namespace/Issuer_Code_22M','Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Underlying_Product_Identifier_22U','Identification_of_Financial_Instrument_35B','Execution_Venue_22V','Execution_Timestamp_98D','Confirmation_Timestamp_98G','Clearing_Timestamp_98H','Contact_Information_29A','Commission_and_Fees_34C','Sender_to_Receiver_Information_72','New_Sequence_15D','Reference_to_Previous_Deals_21P','Gain_Loss_Indicator_17G',
    'Currency_Amount_32G','Commission_and_Fees_-_Currency_and_Amount_34B','Commission_and_Fees_-_Settlement_Date_30F','New_Sequence_15E','Delivery_Agent_53D','Intermediary_56D','Receiving_Agent_57D','Beneficiary_Institution_58D']
    lista_valor_304= [':15A:',':20:',':21:',':22A:',':94A:',':17O:',':17F:',':17N:',':83A:',':83J:',':82A:',':82J:',':87A:',':87J:',':81A:',':81D:',':81J:',':89A:',':89D:',':89J:',':17I:',':77H:',':14C:',':32E:',':30U:',':14S:',':26K:',':21A:',':14E:',':15B:',':30T:',':30V:',':36:',':39M:',':35C:',':32B:',':53A:',':53J:',':56A:',':56J:',':57A:',':57J:',':33B:',':58A:',':58J:',':15C:',':21A:',':21G:',':22L:',':22M:',':22N:',':22P:',':22R:',':22U:',':35B:',':22V:',':98D:',':98G:',':98H:',':29A:',':34C:',':72:',':15D:',':21P:',':17G:',':32G:',':34B:',':30F:',':15E:',':53D:',':56D:',':57D:',':58D:']
    
    lista_305 = ['New_Sequence_15A','Transaction_Reference_Number_20','Related_Reference_21','Code/Common_Reference_22','Further_Identification_23','Scope_of_Operation_94A','Party_A_82A','Party_A_82J','Party_B_87A','Party_B_87J','Fund_or_Beneficiary_Customer_83A','Fund_or_Beneficiary_Customer_83J','Date_Contract_Agreed/Amended_30','Earliest_Exercise_Date_31C','Expiry_Details_31G','Final_Settlement_Date_31E','Settlement_Type_26F','Payment_Clearing_Centre_39M','Non-Deliverable_Indicator_17F','Settlement_Rate_Source_14S','Settlement_Currency_32E','Digital_Token_Identifier_35C','Calculation_Agent_26K','Underlying_Currency_and_Amount_32B',
    'Strike_Price_36','Counter_Currency_and_Amount_33B','Premium_Price_37K','Premium_Payment_34P','Premium_Payment_34R','Senders_Correspondent_53A','Senders_Correspondent_53J','Intermediary_56A','Intermediary_56J','Account_With_Institution_57A','Account_With_Institution_57J','Type_Date_Version_of_the_Agreement_77H','Year_of_Definitions_14C','Sender_to_Receiver_Information_72','New_Sequence_15B','Reporting_Jurisdiction_22L','Reporting_Party_91A','Reporting_Party_91D','Reporting_Party_91J','UTI_Namespace/Issuer_Code_22M','Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Central_Counterparty_Clearing_House_CCP_81A',
    'Central_Counterparty_Clearing_House_CCP_81D','Central_Counterparty_Clearing_House_CCP_81J','Clearing_Broker_89A','Clearing_Broker_89D','Clearing_Broker_89J','Clearing_Exception_Party_96A','Clearing_Exception_Party_96D','Clearing_Exception_Party_96J','Clearing_Broker_Identification_22S','Cleared_Product_Identification_22T','Clearing_Threshold_Indicator_17E','Underlying_Product_Identifier_22U','Identification_of_Financial_Instrument_35B','Allocation_Indicator_17H','Collateralisation_Indicator_17P','Execution_Venue_22V','Execution_Timestamp_98D','Non_Standard_Flag_17W','Financial_Nature_of_the_Counterparty_Indicator_17Y','Collateral_Portfolio_Indicator_17Z','Collateral_Portfolio_Code_22Q',
    'Portfolio_Compression_Indicator_17L','Corporate_Sector_Indicator_17M','Trade_with_Non-EEA_Counterparty_Indicator_17Q','Intragroup_Trade_Indicator_17S','Commercial_or_Treasury_Financing_Indicator_17X','Commission_and_Fees_34C','Additional_Reporting_Information_77A']
    lista_valor_305= [':15A:',':20:',':21:',':22:',':23:',':94A:',':82A:',':82J:',':87A:',':87J:',':83A:',':83J:',':30:',':31C:',':31G:',':31E:',':26F:',':39M:',':17F:',':14S:',':32E:',':35C:',':26K:',':32B:',':36:',':33B:',':37K:',':34P:',':34R:',':53A:',':53J:',':56A:',':56J:',':57A:',':57J:',':77H:',':14C:',':72:',':15B:',':22L:',':91A:',':91D:',':91J:',':22M:',':22N:',':22P:',':22R:',':81A:',':81D:',':81J:',':89A:',':89D:',':89J:',':96A:',':96D:',':96J:',':22S:',':22T:',':17E:',':22U:',':35B:',':17H:',':17P:',':22V:',':98D:',':17W:',':17Y:',':17Z:',':22Q:',':17L:',':17M:',':17Q:',':17S:',':17X:',':34C:',':77A:']
    
    lista_306 = ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Common_Reference_22C','Contract_Number_Party_A_21N','Contract_Number_Party_B_21B','Option_Style_12F','Expiration_Style_12E','Option_Type_12D','Barrier_Indicator_17A','Non-Deliverable_Indicator_17F','Type_of_Event_22K','Date_of_Barrier_Event_30U','Location_of_Barrier_Event_29H','Party_A_82A','Party_A_82J','Party_B_87A','Party_B_87J','Fund_or_Beneficiary_Customer_83A','Fund_or_Beneficiary_Customer_83J','Type_Date_Version_of_the_Agreement_77H','Additional_Conditions_77D','Year_of_Definitions_14C','New_Sequence_15B','Buy_Sell_Indicator_17V','Trade_Date_30T','Expiration_or_Valuation_Date_30X',
    'Expiration_or_Valuation_Location_and_Time_29E','Final_Settlement_Date_30F','Final_Settlement_Date_30J','Payment_Business_Day_29H','Settlement_Rate_Source_14S','Payment_Clearing_Centre_39M','Premium_Price_37K','Premium_Payment_Date_30V','Premium_Currency_and_Amount_34B','Calculation_Agent_84A','Calculation_Agent_84B','Calculation_Agent_84D','Calculation_Agent_84J','New_Sequence_15C','Delivery_Agent_53A','Delivery_Agent_53J','Intermediary_2_86A','Intermediary_2_86J','Intermediary_56A','Intermediary_56J','Receiving_Agent_57A','Receiving_Agent_57J','Beneficiary_Institution_58A','Beneficiary_Institution_58J','New_Sequence_15D','Earliest_Exercise_Date_30P','Intermediate_Exercise_Date_30Q','Settlement_Type_26F','Put_Currency_and_Amount_32B',
    'Strike_Price_36','Call_Currency_and_Amount_33B','New_Sequence_15E','Currency_Amount_33E','Binary_Amount_Payment_Date_30H','New_Sequence_15F','Type_of_Barrier_22G','Barrier_Event_Equal_Modifier_17C','Barrier_Level_37J','Lower_Barrier_Level_37L','Barrier_Settlement_Rate_Source_14S','Barrier_Determination_Business_Days_29I','All_Barrier_Dates_Business_Day_Convention_14H','Interim_Barrier_Dates_Business_Day_Convention_14K','Barrier_Event_End_Date_Business_Day_Convention_14L','Barrier_Time_Type_14M','Continuous_Time_Period_29O','Spot_Market_14N','Barrier_Event_Determination_Time_29J','Barrier_Event_Determination_Time_Source_14O','Rebate_Amount_33Z','Rebate_Amount_Payment_Date_30F','Rebate_Amount_Payment_Date_30J','Barrier_Determination_Agent_84A',
    'Barrier_Determination_Agent_84B','Barrier_Determination_Agent_84D','Barrier_Determination_Agent_84J','Barrier_Monitoring_Period_30I','New_Sequence_15G','Currency_Pair_32Q','New_Sequence_15H','Settlement_Currency_32E','Calculation_Agent_26K','Digital_Token_Identifier_35C','New_Sequence_15I','Early_Termination_Style_12G','Early_Termination_Date_30T','Frequency_of_Early_Termination_22Y','Exercising_Party_85A','Exercising_Party_85D','Exercising_Party_85J','Non-Exercising_Party_88A','Non-Exercising_Party_88D','Non-Exercising_Party_88J','Commencement_Date_30Y','Expiry_Details_29L','Earliest_Exercise_Time_29E','Latest_Exercise_Time_29M','Cash_Settlement_17I','Cash_Settlement_Valuation_Details_29N','Cash_Settlement_Payment_Date_30Z','New_Sequence_15J',
    'Average_Strike_Price_Calculation_14B','Decimal_Places_16C','Number_of_Spot_Averaging_Dates_18B','Spot_Averaging_Date_30M','Spot_Averaging_Weighting_Factor_19Y','Number_of_Strike_Averaging_Dates_18C','Strike_Averaging_Date_30N','Strike_Averaging_Weighting_Factor_19Z','Adjustment_19C','Calculation_of_Settlement_Amount_23C','New_Sequence_15K','Contact_Information_29A','Dealing_Method_24D','Broker_Identification_88A','Broker_Identification_88D','Brokers_Commission_71F','Brokers_Reference_21G','Sender_to_Receiver_Information_72','New_Sequence_15L','Number_of_Repetitions_18A','Payment_Date_30F','Currency_Payment_Amount_32H','New_Sequence_15M','Reporting_Jurisdiction_22L','Reporting_Party_91A','Reporting_Party_91D','Reporting_Party_91J','UTI_Namespace/Issuer_Code_22M',
    'Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Clearing_Exception_Party_96A','Clearing_Exception_Party_96D','Clearing_Exception_Party_96J','Clearing_Broker_Identification_22S','Cleared_Product_Identification_22T','Clearing_Threshold_Indicator_17E','Underlying_Product_Identifier_22U','Identification_of_Financial_Instrument_35B','Allocation_Indicator_17H','Collateralisation_Indicator_17P','Execution_Venue_22V','Execution_Timestamp_98D','Non_Standard_Flag_17W','Financial_Nature_of_the_Counterparty_Indicator_17Y','Collateral_Portfolio_Indicator_17Z','Collateral_Portfolio_Code_22Q','Portfolio_Compression_Indicator_17L','Corporate_Sector_Indicator_17M','Trade_with_Non-EEA_Counterparty_Indicator_17Q','Intragroup_Trade_Indicator_17S','Commercial_or_Treasury_Financing_Indicator_17X','Commission_and_Fees_34C','Additional_Reporting_Information_77A']
    lista_valor_306= [':15A:',':20:',':21:',':22A:',':94A:',':22C:',':21N:',':21B:',':12F:',':12E:',':12D:',':17A:',':17F:',':22K:',':30U:',':29H:',':82A:',':82J:',':87A:',':87J:',':83A:',':83J:',':77H:',':77D:',':14C:',':15B:',':17V:',':30T:',':30X:',':29E:',':30F:',':30J:',':29H:',':14S:',':39M:',':37K:',':30V:',':34B:',':84A:',':84B:',':84D:',':84J:',':15C:',':53A:',':53J:',':86A:',':86J:',':56A:',':56J:',':57A:',':57J:',':58A:',':58J:',':15D:',':30P:',':30Q:',':26F:',':32B:',':36:',':33B:',':15E:',':33E:',':30H:',':15F:',':22G:',':17C:',':37J:',':37L:',':14S:',':29I:',':14H:',':14K:',':14L:',':14M:',':29O:',':14N:',':29J:',':14O:',':33Z:',':30F:',':30J:',':84A:',':84B:',':84D:',':84J:',':30I:',':15G:',':32Q:',':15H:',':32E:',':26K:',':35C:',':15I:',':12G:',':30T:',':22Y:',':85A:',':85D:',':85J:',':88A:',':88D:',':88J:',':30Y:',':29L:',':29E:',':29M:',':17I:',':29N:',':30Z:',':15J:',
    ':14B:',':16C:',':18B:',':30M:',':19Y:',':18C:',':30N:',':19Z:',':19C:',':23C:',':15K:',':29A:',':24D:',':88A:',':88D:',':71F:',':21G:',':72:',':15L:',':18A:',':30F:',':32H:',':15M:',':22L:',':91A:',':91D:',':91J:',':22M:',':22N:',':22P:',':22R:',':96A:',':96D:',':96J:',':22S:',':22T:',':17E:',':22U:',':35B:',':17H:',':17P:',':22V:',':98D:',':17W:',':17Y:',':17Z:',':22Q:',':17L:',':17M:',':17Q:',':17S:',':17X:',':34C:',':77A:']
    
    lista_320 = ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Type_of_Event_22B','Common_Reference_22C','Contract_Number_Party_A_21N','Party_A_82A','Party_A_82D','Party_A_82J','Party_B_87A','Party_B_87D','Party_B_87J','Fund_or_Instructing_Party_83A','Fund_or_Instructing_Party_83D','Fund_or_Instructing_Party_83J','Terms_and_Conditions_77D','New_Sequence_15B','Party_As_Role_17R','Trade_Date_30T','Value_Date_30V','Maturity_Date_30P','Currency_and_Principal_Amount_32B','Amount_to_be_Settled_32H','Next_Interest_Due_Date_30X','Currency_and_Interest_Amount_34E','Interest_Rate_37G','Day_Count_Fraction_14D','Last_Day_of_the_First_Interest_Period_30F','Number_of_Days_38J','Payment_Clearing_Centre_39M','New_Sequence_15C','Delivery_Agent_53A','Delivery_Agent_53D','Delivery_Agent_53J','Intermediary_2_86A','Intermediary_2_86D','Intermediary_2_86J',
    'Intermediary_56A','Intermediary_56D','Intermediary_56J','Receiving_Agent_57A','Receiving_Agent_57D','Receiving_Agent_57J','Beneficiary_Institution_58A','Beneficiary_Institution_58D','Beneficiary_Institution_58J','New_Sequence_15D','New_Sequence_15E','New_Sequence_15F','New_Sequence_15G','Tax_Rate_37L','Transaction_Currency_and_Net_Interest_Amount_33B','Exchange_Rate_36','Reporting_Currency_and_Tax_Amount_33E','New_Sequence_15H','Contact_Information_29A','Dealing_Method_24D','Dealing_Branch_Party_A_84A','Dealing_Branch_Party_A_84B','Dealing_Branch_Party_A_84D','Dealing_Branch_Party_A_84J','Dealing_Branch_Party_B_85A','Dealing_Branch_Party_B_85B','Dealing_Branch_Party_B_85D','Dealing_Branch_Party_B_85J','Broker_Identification_88A','Broker_Identification_88D','Broker_Identification_88J','Brokers_Commission_71F','Counterpartys_Reference_26H','Brokers_Reference_21G','Commission_and_Fees_34C',
    'Sender_to_Receiver_Information_72','New_Sequence_15I','Number_of_Repetitions_18A','Payment_Date_30F','Currency_Payment_Amount_32H']
    lista_valor_320= [':15A:',':20:',':21:',':22A:',':94A:',':22B:',':22C:',':21N:',':82A:',':82D:',':82J:',':87A:',':87D:',':87J:',':83A:',':83D:',':83J:',':77D:',':15B:',':17R:',':30T:',':30V:',':30P:',':32B:',':32H:',':30X:',':34E:',':37G:',':14D:',':30F:',':38J:',':39M:',':15C:',':53A:',':53D:',':53J:',':86A:',':86D:',':86J:',':56A:',':56D:',':56J:',':57A:',':57D:',':57J:',':58A:',':58D:',':58J:',':15D:',':15E:',':15F:',':15G:',':37L:',':33B:',':36:',':33E:',':15H:',':29A:',':24D:',':84A:',':84B:',':84D:',':84J:',':85A:',':85B:',':85D:',':85J:',':88A:',':88D:',':88J:',':71F:',':26H:',':21G:',':34C:',':72:',':15I:',':18A:',':30F:',':32H:']
    
    lista_321 = ['Start_of_Block_16R','Reference_20C','Function_of_the_Message_23G','Indicator_22H','Number_Count_99B','Linked_Message_13A','Linked_Message_13B','End_of_Block_16S','Contract_Number_Reference_20C','Date/Time_98A','Amount_19A','Interest_Rate_92A','Number_of_Days_Accrued_99B','Country_94C','Loan/Deposit_Counterparty_95P','Loan/Deposit_Counterparty_95Q','Loan/Deposit_Counterparty_95R','Investor_95P','Investor_95Q','Investor_95R','Safekeeping_Account_97A','Party_95P','Party_95Q','Party_95R','Principal_and_Interest_22H','Cash_Account_97A','Party_Contact_Narrative_70C']
    lista_valor_321= [':16R:',':20C:',':23G:',':22H:',':99B:',':13A:',':13B:',':16S:',':20C:',':98A:',':19A:',':92A:',':99B:',':94C:',':95P:',':95Q:',':95R:',':95P:',':95Q:',':95R:',':97A:',':95P:',':95Q:',':95R:',':22H:',':97A:',':70C:']
    
    lista_330 = ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Type_of_Event_22B','Common_Reference_22C','Contract_Number_Party_A_21N','Party_A_82A','Party_A_82D','Party_A_82J','Party_B_87A','Party_B_87D','Party_B_87J','Fund_or_Instructing_Party_83A','Fund_or_Instructing_Party_83D','Fund_or_Instructing_Party_83J','Terms_and_Conditions_77D','New_Sequence_15B','Party_As_Role_17R','Trade_Date_30T','Value_Date_30V','Period_of_Notice_38A','Currency_and_Balance_32B','Principal_Amount_to_be_Settled_32H','Interest_Due_Date_30X','Currency_and_Interest_Amount_34E','Interest_Rate_37G','Day_Count_Fraction_14D','Last_Day_of_the_Next_Interest_Period_30F','Number_of_Days_38J','Payment_Clearing_Centre_39M','New_Sequence_15C','Delivery_Agent_53A','Delivery_Agent_53D','Delivery_Agent_53J','Intermediary_2_86A','Intermediary_2_86D','Intermediary_2_86J',
    'Intermediary_56A','Intermediary_56D','Intermediary_56J','Receiving_Agent_57A','Receiving_Agent_57D','Receiving_Agent_57J','Beneficiary_Institution_58A','Beneficiary_Institution_58D','Beneficiary_Institution_58J','New_Sequence_15D','New_Sequence_15E','New_Sequence_15F','New_Sequence_15G','Tax_Rate_37L','Transaction_Currency_and_Net_Interest_Amount_33B','Exchange_Rate_36','Reporting_Currency_and_Tax_Amount_33E','New_Sequence_15H','Contact_Information_29A','Dealing_Method_24D','Dealing_Branch_Party_A_84A','Dealing_Branch_Party_A_84B','Dealing_Branch_Party_A_84D','Dealing_Branch_Party_A_84J','Dealing_Branch_Party_B_85A','Dealing_Branch_Party_B_85B','Dealing_Branch_Party_B_85D','Dealing_Branch_Party_B_85J','Counterpartys_Reference_26H','Commission_and_Fees_34C','Sender_to_Receiver_Information_72']
    lista_valor_330= [':15A:',':20:',':21:',':22A:',':94A:',':22B:',':22C:',':21N:',':82A:',':82D:',':82J:',':87A:',':87D:',':87J:',':83A:',':83D:',':83J:',':77D:',':15B:',':17R:',':30T:',':30V:',':38A:',':32B:',':32H:',':30X:',':34E:',':37G:',':14D:',':30F:',':38J:',':39M:',':15C:',':53A:',':53D:',':53J:',':86A:',':86D:',':86J:',':56A:',':56D:',':56J:',':57A:',':57D:',':57J:',':58A:',':58D:',':58J:',':15D:',':15E:',':15F:',':15G:',':37L:',':33B:',':36:',':33E:',':15H:',':29A:',':24D:',':84A:',':84B:',':84D:',':84J:',':85A:',':85B:',':85D:',':85J:',':26H:',':34C:',':72:']
    
    lista_340 = ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Common_Reference_22C','Type_of_FRA_23D','Contract_Number_Party_A_21N','Contract_Number_Party_B_21B','Party_A_82A','Party_A_82D','Party_B_87A','Party_B_87D','Type_Date_Version_of_the_Agreement_77H','Year_of_Definitions_14C','New_Sequence_15B','Trade_Date_30T','Currency_Notional_Amount_32B','Effective_Date_30F','Termination_Date_30P','Fixed_Rate_37M','Floating_Rate_Option_14F','Fixing_Date_30V','Contract_Period_38D','Designated_Maturity_38G','Floating_Rate_Day_Count_Fraction_14D','FRA_Discounting_17F','Number_of_Repetitions_18A','Financial_Centre_22B','Payment_Clearing_Centre_39M','New_Sequence_15C','Delivery_Agent_53A','Delivery_Agent_53D','Delivery_Agent_53J','Intermediary_2_86A','Intermediary_2_86D','Intermediary_2_86J','Intermediary_56A','Intermediary_56D','Intermediary_56J',
    'Receiving_Agent_57A','Receiving_Agent_57D','Receiving_Agent_57J','Beneficiary_Institution_58A','Beneficiary_Institution_58D','Beneficiary_Institution_58J','New_Sequence_15D','New_Sequence_15E','Contact_Information_29A','Dealing_Method_24D','Broker_Identification_88A','Broker_Identification_88D','Brokers_Commission_71F','Brokers_Reference_21G','Sender_to_Receiver_Information_72','New_Sequence_15F','Payment_Date_30F','Currency_Payment_Amount_32H','New_Sequence_15G','Reporting_Jurisdiction_22L','Reporting_Party_91A','Reporting_Party_91D','Reporting_Party_91J','UTI_Namespace/Issuer_Code_22M','Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Clearing_Exception_Party_96A','Clearing_Exception_Party_96D','Clearing_Exception_Party_96J','Clearing_Broker_Identification_22S','Cleared_Product_Identification_22T','Clearing_Threshold_Indicator_17E','Underlying_Product_Identifier_22U',
    'Identification_of_Financial_Instrument_35B','Allocation_Indicator_17H','Collateralisation_Indicator_17P','Execution_Venue_22V','Execution_Timestamp_98D','Non_Standard_Flag_17W','Financial_Nature_of_the_Counterparty_Indicator_17Y','Collateral_Portfolio_Indicator_17Z','Collateral_Portfolio_Code_22Q','Portfolio_Compression_Indicator_17L','Corporate_Sector_Indicator_17M','Trade_with_Non-EEA_Counterparty_Indicator_17Q','Intragroup_Trade_Indicator_17S','Commercial_or_Treasury_Financing_Indicator_17X','Commission_and_Fees_34C','Additional_Reporting_Information_77A']
    lista_valor_340= [':15A:',':20:',':21:',':22A:',':94A:',':22C:',':23D:',':21N:',':21B:',':82A:',':82D:',':87A:',':87D:',':77H:',':14C:',':15B:',':30T:',':32B:',':30F:',':30P:',':37M:',':14F:',':30V:',':38D:',':38G:',':14D:',':17F:',':18A:',':22B:',':39M:',':15C:',':53A:',':53D:',':53J:',':86A:',':86D:',':86J:',':56A:',':56D:',':56J:',':57A:',':57D:',':57J:',':58A:',':58D:',':58J:',':15D:',':15E:',':29A:',':24D:',':88A:',':88D:',':71F:',':21G:',':72:',':15F:',':30F:',':32H:',':15G:',':22L:',':91A:',':91D:',':91J:',':22M:',':22N:',':22P:',':22R:',':96A:',':96D:',':96J:',':22S:',':22T:',':17E:',':22U:',':35B:',':17H:',':17P:',':22V:',':98D:',':17W:',':17Y:',':17Z:',':22Q:',':17L:',':17M:',':17Q:',':17S:',':17X:',':34C:',':77A:']
    
    lista_341 = ['New_Sequence_15A','Senders_Reference_20','Related_Reference_21','Type_of_Operation_22A','Scope_of_Operation_94A','Common_Reference_22C','Type_of_FRA_23D','Contract_Number_Party_A_21N','Contract_Number_Party_B_21B','Party_A_82A','Party_A_82D','Party_B_87A','Party_B_87D','Contact_Information_29A','Sender_to_Receiver_Information_72','New_Sequence_15B','Trade_Date_30T','Currency_Notional_Amount_32B','Effective_Date_30F','Termination_Date_30P','Fixed_Rate_37M','Fixing_Date_30V','Contract_Period_38D','Payment_Clearing_Centre_39M','New_Sequence_15C','Settlement_Rate_37R','Settlement_Currency_and_Amount_34E','Delivery_Agent_53A','Delivery_Agent_53D','Delivery_Agent_53J','Intermediary_2_86A','Intermediary_2_86D','Intermediary_2_86J','Intermediary_56A','Intermediary_56D','Intermediary_56J','Receiving_Agent_57A','Receiving_Agent_57D','Receiving_Agent_57J','Beneficiary_Institution_58A','Beneficiary_Institution_58D',
    'Beneficiary_Institution_58J','New_Sequence_15D','Reporting_Jurisdiction_22L','Reporting_Party_91A','Reporting_Party_91D','Reporting_Party_91J','UTI_Namespace/Issuer_Code_22M','Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Clearing_Exception_Party_96A','Clearing_Exception_Party_96D','Clearing_Exception_Party_96J','Clearing_Broker_Identification_22S','Cleared_Product_Identification_22T','Clearing_Threshold_Indicator_17E','Underlying_Product_Identifier_22U','Identification_of_Financial_Instrument_35B','Allocation_Indicator_17H','Collateralisation_Indicator_17P','Execution_Venue_22V','Execution_Timestamp_98D','Non_Standard_Flag_17W','Financial_Nature_of_the_Counterparty_Indicator_17Y','Collateral_Portfolio_Indicator_17Z','Collateral_Portfolio_Code_22Q','Portfolio_Compression_Indicator_17L','Corporate_Sector_Indicator_17M','Trade_with_Non-EEA_Counterparty_Indicator_17Q',
    'Intragroup_Trade_Indicator_17S','Commercial_or_Treasury_Financing_Indicator_17X','Commission_and_Fees_34C','Additional_Reporting_Information_77A']
    lista_valor_341= [':15A:',':20:',':21:',':22A:',':94A:',':22C:',':23D:',':21N:',':21B:',':82A:',':82D:',':87A:',':87D:',':29A:',':72:',':15B:',':30T:',':32B:',':30F:',':30P:',':37M:',':30V:',':38D:',':39M:',':15C:',':37R:',':34E:',':53A:',':53D:',':53J:',':86A:',':86D:',':86J:',':56A:',':56D:',':56J:',':57A:',':57D:',':57J:',':58A:',':58D:',':58J:',':15D:',':22L:',':91A:',':91D:',':91J:',':22M:',':22N:',':22P:',':22R:',':96A:',':96D:',':96J:',':22S:',':22T:',':17E:',':22U:',':35B:',':17H:',':17P:',':22V:',':98D:',':17W:',':17Y:',':17Z:',':22Q:',':17L:',':17M:',':17Q:',':17S:',':17X:',':34C:',':77A:']
    
    lista_400= ['Sending_Banks_TRN_20','Related_Reference_21','Amount_Collected_32A','Amount_Collected_32B','Amount_Collected_32K','Proceeds_Remitted_33A','Ordering_Bank_52A','Ordering_Bank_52D','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D',
    'Account_With_Bank_57A','Account_With_Bank_57D','Beneficiary_Bank_58A','Beneficiary_Bank_58B','Beneficiary_Bank_58D','Details_of_Charges_71B','Sender_to_Receiver_Information_72','Details_of_Amounts_Added_73']
    lista_valor_400= [':20:',':21:',':32A:',':32B:',':32K:',':33A:',':52A:',':52D:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':57A:',':57D:',':58A:',':58B:',':58D:',':71B:',':72:',':73:']
    
    lista_500= ['Start_of_Block_16R','Senders_Message_Reference_20C','Function_of_the_Message_23G','Preparation_Date/Time_98A','Preparation_Date/Time_98C','Preparation_Date/Time_98E','Type_of_Instruction_Indicator_22F','Linkage_Type_Indicator_22F','Linked_Message_13A','Linked_Message_13B','Reference_20C','End_of_Block_16S','Reference_20D','Indicator_22F','Requested_Registration_Date/_Time_98A','Requested_Registration_Date/_Time_98C','Identification_of_the_Financial_Instrument_35B','Registration_Details_70C','Place_of_Listing_94B',
    'Type_of_Financial_Instrument_12A','Type_of_Financial_Instrument_12B','Type_of_Financial_Instrument_12C','Currency_of_Denomination_11A','Date/Time_98A','Rate_92A','Number_Identification_13A','Number_Identification_13B','Flag_17B','Price_90A','Price_90B','Quantity_of_Financial_Instrument_36B','Quantity_of_Financial_Instrument_36D','Financial_Instrument_Attribute_Narrative_70E','Quantity_of_Financial_Instrument_to_be_Registered_36B','Quantity_of_Financial_Instrument_to_be_Registered_36D','Account_Owner_95P','Account_Owner_95R',
    'Account_97A','Account_97D','Party_95P','Party_95R','Party_95S','Party_95U','Place_94C','Place_94D','Place_94G','Narrative_70C','Name_at_Birth_95U','Date/Time_98C','Additional_Text_70C','Party_95Q']
    lista_valor_500= [':16R:',':20C:',':23G:',':98A:',':98C:',':98E:',':22F:',':22F:',':13A:',':13B:',':20C:',':16S:',':20D:',':22F:',':98A:',':98C:',':35B:',':70C:',':94B:',':12A:',':12B:',':12C:',':11A:',':98A:',':92A:',':13A:',':13B:',':17B:',':90A:',':90B:',':36B:',':36D:',':70E:',':36B:',':36D:',':95P:',':95R:',':97A:',':97D:',':95P:',':95R:',':95S:',':95U:',':94C:',':94D:',':94G:',':70C:',':95U:',':98C:',':70C:',':95Q:']
    
    lista_600= ['New_Sequence_15A','Transaction_Reference_Number_20','Related_Reference_21','Code/Common_Reference_22','Scope_of_Operation_94A','Party_A_82A','Party_A_82J','Party_B_87A','Party_B_87J','Fund_or_Beneficiary_Customer_83A','Fund_or_Beneficiary_Customer_83J','Date_Contract_Agreed/Amended_30','Identification_of_the_Commodity_26C','Counterpartys_Reference_26H','Price_per_Unit_33G','Details_of_Adjustments_71C','Type_Date_Version_of_the_Agreement_77H','Terms_and_Conditions_77D','Year_of_Definitions_14C','Sender_to_Receiver_Information_72',
    'New_Sequence_15B','Quantity_of_the_Commodity_32F','Intermediary_86A','Intermediary_86B','Intermediary_86D','Receiver_of_the_Commodity_87A','Receiver_of_the_Commodity_87B','Receiver_of_the_Commodity_87D','Consideration_34P','Senders_Correspondent_53A','Senders_Correspondent_53J','Intermediary_56A','Intermediary_56J','Account_With_Institution_57A','Account_With_Institution_57J','New_Sequence_15C','Deliverer_of_the_Commodity_87A','Deliverer_of_the_Commodity_87B','Deliverer_of_the_Commodity_87D','Consideration_34R','New_Sequence_15D','Reporting_Jurisdiction_22L',
    'Reporting_Party_91A','Reporting_Party_91D','Reporting_Party_91J','UTI_Namespace/Issuer_Code_22M','Transaction_Identifier_22N','PUTI_Namespace/Issuer_Code_22P','Prior_Transaction_Identifier_22R','Clearing_Exception_Party_96A','Clearing_Exception_Party_96D','Clearing_Exception_Party_96J','Clearing_Broker_Identification_22S','Cleared_Product_Identification_22T','Clearing_Threshold_Indicator_17E','Underlying_Product_Identifier_22U','Identification_of_Financial_Instrument_35B','Allocation_Indicator_17H','Collateralisation_Indicator_17P','Execution_Venue_22V','Execution_Timestamp_98D',
    'Non_Standard_Flag_17W','Financial_Nature_of_the_Counterparty_Indicator_17Y','Collateral_Portfolio_Indicator_17Z','Collateral_Portfolio_Code_22Q','Portfolio_Compression_Indicator_17L','Corporate_Sector_Indicator_17M','Trade_with_Non-EEA_Counterparty_Indicator_17Q','Intragroup_Trade_Indicator_17S','Commercial_or_Treasury_Financing_Indicator_17X','Commission_and_Fees_34C','Additional_Reporting_Information_77A']
    lista_valor_600= [':15A:',':20:',':21:',':22:',':94A:',':82A:',':82J:',':87A:',':87J:',':83A:',':83J:',':30:',':26C:',':26H:',':33G:',':71C:',':77H:',':77D:',':14C:',':72:',':15B:',':32F:',':86A:',':86B:',':86D:',':87A:',':87B:',':87D:',':34P:',':53A:',':53J:',':56A:',':56J:',':57A:',':57J:',':15C:',':87A:',':87B:',':87D:',':34R:',':15D:',':22L:',':91A:',':91D:',':91J:',':22M:',':22N:',':22P:',':22R:',':96A:',':96D:',':96J:',':22S:',':22T:',':17E:',':22U:',':35B:',':17H:',':17P:',':22V:',':98D:',':17W:',':17Y:',':17Z:',':22Q:',':17L:',':17M:',':17Q:',':17S:',':17X:',':34C:',':77A:']
    
    lista_700 = ['Sequence_of_Total_27', 'Form_of_Documentary_Credit_40A','Documentary_Credit_Number_20','Reference_to_Pre_Advice_23','Date_of_Issue_31C','Applicable_Rules_40E','Date_and_Place_of_Expiry_31D','Applicant_Bank_51A','Applicant_Bank_51D','Applicant_50','Beneficiary_59',
    'Currency_Code_Amount_32B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Available_With_By_41A','Available_With_By_41D','Drafts_at_42C','Drawee_42A','Drawee_42D','Mixed_Payment_Details_42M','Negotiation/Deferred_Payment_Details_42P','Partial_Shipments_43P','Transhipment_43T',
    'Place_of_Taking_in_Charge/Dispatch_from/Place_of_Receipt_44A','Port_of_Loading/Airport_of_Departure_44E','Port_of_Discharge/Airport_of_Departure_44F','Place_of_Final_Destination/For_Transportation_to/Place_of_Delivery_44B','Latest_Date_of_Shipment_44C','Shipment_Period_44D','Description_of_Goods_and/or_Services_45A',
    'Documents_Required_46A','Additional_Conditions_47A','Special_Payment_Conditions_for_Beneficiary_49G','Special_Payment_Conditions_for_Bank_Only_49H','Charges_71D','Period_for_Presentation_in_Days_48','Confirmation_Instructions_49','Requested_Confirmation_Party_58A','Requested_Confirmation_Party_58D',
    'Reimbursing_Bank_53A','Reimbursing_Bank_53D','Instructions_to_the_Paying/Accepting/Negotiating_Bank_78','Advise_Through_Bank_57A','Advise_Through_Bank_57B','Advise_Through_Bank_57D','Sender_to_Receiver_Information_72Z']
    lista_valor_700 = [':27:', ':40A:',':20:',':23:',':31C:',':40E:',':31D:',':51A:',':51D:',':50:',':59:',':32B:',':39A:',':39C:',':41A:',':41D:',':42C:',':42A:',':42D:',':42M:',':42P:',':43P:',':43T:',':44A:',':44E:',':44F:',':44B:',':44C:',':44D:',':45A:',':46A:',':47A:',':49G:',':49H:',':71D:',':48:',':49:',':58A:',':58D:',':53A:',':53D:',':78:',':57A:',':57B:',':57D:',':72Z:']
    
    lista_701= ['Sequence_of_Total_27','Documentary_Credit_Number_20','Description_of_Goods_and/or_Services_45A','Documents_Required_46A','Additional_Conditions_47A','Special_Payment_Conditions_for_Beneficiary_49G','Special_Payment_Conditions_for_Bank_Only_49H']
    lista_valor_701= [':27:',':20:',':45A:',':46A:',':47A:',':49G:',':49H:']
    
    lista_705= ['Form_of_Documentary_Credit_40A','Documentary_Credit_Number_20','Date_and_Place_of_Expiry_31D','Applicant_50','Beneficiary_59','Currency_Code_Amount_32B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Available_With_By_41A','Available_With_By_41D','Place_of_Taking_in_Charge/Dispatch_from/Place_of_Receipt_44A','Port_of_Loading/Airport_of_Departure_44E','Port_of_Discharge/Airport_of_Destination_44F','Place_of_Final_Destination/For_Transportation_to/Place_of_Delivery_44B','Latest_Date_of_Shipment_44C','Shipment_Period_44D','Description_of_Goods_and/or_Services_45A','Advise_Through_Bank_57A','Advise_Through_Bank_57B','Advise_Through_Bank_57D','Narrative_79Z','Sender_to_Receiver_Information_72Z']
    lista_valor_705= [':40A:',':20:',':31D:',':50:',':59:',':32B:',':39A:',':39C:',':41A:',':41D:',':44A:',':44E:',':44F:',':44B:',':44C:',':44D:',':45A:',':57A:',':57B:',':57D:',':79Z:',':72Z:']
    
    lista_707= ['Sequence_of_Total_27','Senders_Reference_20','Receivers_Reference_21','Issuing_Banks_Reference_23','Issuing_Bank_52A','Issuing_Bank_52D','Non-Bank_Issuer_50B','Date_of_Issue_31C','Number_of_Amendment_26E','Date_of_Amendment_30','Purpose_of_Message_22A','Cancellation_Request_23S','Form_of_Documentary_Credit_40A','Applicable_Rules_40E','Date_and_Place_of_Expiry_31D','Changed_Applicant_Details_50','Beneficiary_59','Increase_of_Documentary_Credit_Amount_32B','Decrease_of_Documentary_Credit_Amount_33B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Available_With_By_41A','Available_With_By_41D','Drafts_at_42C','Drawee_42A','Drawee_42D','Mixed_Payment_Details_42M','Negotiation/Deferred_Payment_Details_42P',
    'Partial_Shipments_43P','Transhipment_43T','Place_of_Taking_in_Charge/Dispatch_from/Place_of_Receipt_44A','Port_of_Loading/Airport_of_Departure_44E','Port_of_Discharge/Airport_of_Destination_44F','Place_of_Final_Destination/For_Transportation_to/Place_of_Delivery_44B','Latest_Date_of_Shipment_44C','Shipment_Period_44D','Description_of_Goods_and/or_Services_45B','Documents_Required_46B','Additional_Conditions_47B','Special_Payment_Conditions_for_Beneficiary_49M','Special_Payment_Conditions_for_Bank_Only_49N','Charges_71D','Amendment_Charge_Payable_By_71N','Period_for_Presentation_in_Days_48','Confirmation_Instructions_49','Requested_Confirmation_Party_58A','Requested_Confirmation_Party_58D','Reimbursing_Bank_53A','Reimbursing_Bank_53D',
    'Instructions_to_the_Paying/Accepting/Negotiating_Bank_78','Advise_Through_Bank_57A','Advise_Through_Bank_57B','Advise_Through_Bank_57D','Sender_to_Receiver_Information_72Z']
    lista_valor_707= [':27:',':20:',':21:',':23:',':52A:',':52D:',':50B:',':31C:',':26E:',':30:',':22A:',':23S:',':40A:',':40E:',':31D:',':50:',':59:',':32B:',':33B:',':39A:',':39C:',':41A:',':41D:',':42C:',':42A:',':42D:',':42M:',':42P:',':43P:',':43T:',':44A:',':44E:',':44F:',':44B:',':44C:',':44D:',':45B:',':46B:',':47B:',':49M:',':49N:',':71D:',':71N:',':48:',':49:',':58A:',':58D:',':53A:',':53D:',':78:',':57A:',':57B:',':57D:',':72Z:']
    
    lista_708= ['Sequence_of_Total_27','Senders_Reference_20','Receivers_Reference_21','Issuing_Banks_Reference_23','Number_of_Amendment_26E','Date_of_Amendment_30','Description_of_Goods_and/or_Services_45B','Documents_Required_46B','Additional_Conditions_47B','Special_Payment_Conditions_for_Beneficiary_49M','Special_Payment_Conditions_for_Bank_Only_49N']
    lista_valor_708= [':27:',':20:',':21:',':23:',':26E:',':30:',':45B:',':46B:',':47B:',':49M:',':49N:']
    
    lista_710= ['Sequence_of_Total_27','Form_of_Documentary_Credit_40B','Senders_Reference_20','Documentary_Credit_Number_21','Reference_to_Pre-Advice_23','Date_of_Issue_31C','Applicable_Rules_40E','Date_and_Place_of_Expiry_31D','Issuing_Bank_52A','Issuing_Bank_52D','Non-Bank_Issuer_50B','Applicant_Bank_51A','Applicant_Bank_51D','Applicant_50','Beneficiary_59','Currency_Code_Amount_32B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Available_With_By_41A','Available_With_By_41D','Drafts_at_42C','Drawee_42A','Drawee_42D','Mixed_Payment_Details_42M','Negotiation/Deferred_Payment_Details_42P','Partial_Shipments_43P','Transhipment_43T','Place_of_Taking_in_Charge/Dispatch_from/Place_of_Receipt_44A','Port_of_Loading/Airport_of_Departure_44E',
    'Port_of_Discharge/Airport_of_Destination_44F','Place_of_Final_Destination/For_Transportation_to/Place_of_Delivery_44B','Latest_Date_of_Shipment_44C','Shipment_Period_44D','Description_of_Goods_and/or_Services_45A','Documents_Required_46A','Additional_Conditions_47A','Special_Payment_Conditions_for_Beneficiary_49G','Special_Payment_Conditions_for_Bank_Only_49H','Charges_71D','Period_for_Presentation_in_Days_48','Confirmation_Instructions_49','Requested_Confirmation_Party_58A','Requested_Confirmation_Party_58D','Reimbursing_Bank_53A','Reimbursing_Bank_53D','Instructions_to_the_Paying/Accepting/Negotiating_Bank_78','Instructions_from_Intermediary_Bank_78D','Advise_Through_Bank_57A','Advise_Through_Bank_57B','Advise_Through_Bank_57D','Sender_to_Receiver_Information_72Z']
    lista_valor_710= [':27:',':40B:',':20:',':21:',':23:',':31C:',':40E:',':31D:',':52A:',':52D:',':50B:',':51A:',':51D:',':50:',':59:',':32B:',':39A:',':39C:',':41A:',':41D:',':42C:',':42A:',':42D:',':42M:',':42P:',':43P:',':43T:',':44A:',':44E:',':44F:',':44B:',':44C:',':44D:',':45A:',':46A:',':47A:',':49G:',':49H:',':71D:',':48:',':49:',':58A:',':58D:',':53A:',':53D:',':78:',':78D:',':57A:',':57B:',':57D:',':72Z:']
    
    lista_711= ['Sequence_of_Total_27','Senders_Reference_20','Documentary_Credit_Number_21','Description_of_Goods_and/or_Services_45A','Documents_Required_46A','Additional_Conditions_47A','Special_Payment_Conditions_for_Beneficiary_49G','Special_Payment_Conditions_for_Bank_Only_49H']
    lista_valor_711= [':27:',':20:',':21:',':45A:',':46A:',':47A:',':49G:',':49H:']
    
    lista_720= ['Sequence_of_Total_27','Form_of_Documentary_Credit_40B','Transferring_Banks_Reference_20','Documentary_Credit_Number_21','Date_of_Issue_31C','Applicable_Rules_40E','Date_and_Place_of_Expiry_31D','Issuing_Bank_of_the_Original_Documentary_Credit_52A','Issuing_Bank_of_the_Original_Documentary_Credit_52D','Non-Bank_Issuer_of_the_Original_Documentary_Credit_50B','First_Beneficiary_50','Second_Beneficiary_59','Currency_Code_Amount_32B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Available_With_By_41A','Available_With_By_41D','Drafts_at_42C','Drawee_42A','Drawee_42D','Mixed_Payment_Details_42M','Negotiation/Deferred_Payment_Details_42P','Partial_Shipments_43P','Transhipment_43T','Place_of_Taking_in_Charge/Dispatch_from/Place_of_Receipt_44A',
    'Port_of_Loading/Airport_of_Departure_44E','Port_of_Discharge/Airport_of_Destination_44F','Place_of_Final_Destination/For_Transportation_to/Place_of_Delivery_44B','Latest_Date_of_Shipment_44C','Shipment_Period_44D','Description_of_Goods_and/or_Services_45A','Documents_Required_46A','Additional_Conditions_47A','Special_Payment_Conditions_for_Beneficiary_49G','Special_Payment_Conditions_for_Bank_Only_49H','Charges_71D','Period_for_Presentation_in_Days_48','Confirmation_Instructions_49','Requested_Confirmation_Party_58A','Requested_Confirmation_Party_58D','Instructions_to_the_Paying/Accepting/Negotiating_Bank_78','Instructions_from_Transferring_Bank_78D','Advise_Through_Bank_57A','Advise_Through_Bank_57B','Advise_Through_Bank_57D','Sender_to_Receiver_Information_72Z']
    lista_valor_720= [':27:',':40B:',':20:',':21:',':31C:',':40E:',':31D:',':52A:',':52D:',':50B:',':50:',':59:',':32B:',':39A:',':39C:',':41A:',':41D:',':42C:',':42A:',':42D:',':42M:',':42P:',':43P:',':43T:',':44A:',':44E:',':44F:',':44B:',':44C:',':44D:',':45A:',':46A:',':47A:',':49G:',':49H:',':71D:',':48:',':49:',':58A:',':58D:',':78:',':78D:',':57A:',':57B:',':57D:',':72Z:']
    
    lista_721=['Sequence_of_Total_27','Transferring_Banks_Reference_20','Documentary_Credit_Number_21','Description_of_Goods_and/or_Services_45A','Documents_Required_46A','Additional_Conditions_47A','Special_Payment_Conditions_for_Beneficiary_49G','Special_Payment_Conditions_for_Bank_Only_49H']
    lista_valor_721= [':27:',':20:',':21:',':45A:',':46A:',':47A:',':49G:',':49H:']
    
    lista_730= ['Senders_Reference_20','Receivers_Reference_21','Account_Identification_25','Date_of_Message_Being_Acknowledged_30','Amount_of_Charges_32B','Amount_of_Charges_32D','Account_With_Bank_57A','Account_With_Bank_57D','Charges_71D','Sender_to_Receiver_Information_72Z','Narrative_79Z']
    lista_valor_730= [':20:',':21:',':25:',':30:',':32B:',':32D:',':57A:',':57D:',':71D:',':72Z:',':79Z:']
    
    lista_732= ['Senders_TRN_20','Presenting_Banks_Reference_21','Date_of_Advice_of_Payment/Acceptance/_Negotiation_30','Amount_of_Utilisation_32B','Sender_to_Receiver_Information_72Z']
    lista_valor_732= [':20:',':21:',':30:',':32B:',':72Z:']
    
    lista_734= ['Senders_TRN_20','Presenting_Banks_Reference_21','Date_and_Amount_of_Utilisation_32A','Charges_Claimed_73A','Total_Amount_Claimed_33A','Total_Amount_Claimed_33B','Account_With_Bank_57A','Account_With_Bank_57B','Account_With_Bank_57D','Sender_to_Receiver_Information_72Z','Discrepancies_77J','Disposal_of_Documents_77B']
    lista_valor_734= [':20:',':21:',':32A:',':73A:',':33A:',':33B:',':57A:',':57B:',':57D:',':72Z:',':77J:',':77B:']
    
    lista_740= ['Documentary_Credit_Number_20','Account_Identification_25','Applicable_Rules_40F','Date_and_Place_of_Expiry_31D','Negotiating_Bank_58A','Negotiating_Bank_58D','Beneficiary_59','Credit_Amount_32B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Available_With_By_41A','Available_With_By_41D','Drafts_at_42C','Drawee_42A','Drawee_42D','Mixed_Payment_Details_42M','Negotiation/Deferred_Payment_Details_42P','Reimbursing_Banks_Charges_71A','Other_Charges_71D','Sender_to_Receiver_Information_72Z']
    lista_valor_740= [':20:',':25:',':40F:',':31D:',':58A:',':58D:',':59:',':32B:',':39A:',':39C:',':41A:',':41D:',':42C:',':42A:',':42D:',':42M:',':42P:',':71A:',':71D:',':72Z:']
    
    lista_742= ['Claiming_Banks_Reference_20','Documentary_Credit_Number_21','Date_of_Issue_31C','Issuing_Bank_52A','Issuing_Bank_52D','Principal_Amount_Claimed_32B','Additional_Amount_Claimed_as_Allowed_for_in_Excess_of_Principal_Amount_33B','Charges_71D','Total_Amount_Claimed_34A','Total_Amount_Claimed_34B','Account_With_Bank_57A','Account_With_Bank_57B','Account_With_Bank_57D','Beneficiary_Bank_58A','Beneficiary_Bank_58D','Sender_to_Receiver_Information_72Z']
    lista_valor_742= [':20:',':21:',':31C:',':52A:',':52D:',':32B:',':33B:',':71D:',':34A:',':34B:',':57A:',':57B:',':57D:',':58A:',':58D:',':72Z:']
    
    lista_744= ['Senders_Reference_20','Claiming_Banks_Reference_21','Issuing_Bank_52A','Issuing_Bank_52D','Documentary_Credit_Number_21A','Date_of_Issue_31C','Total_Amount_Claimed_34A','Total_Amount_Claimed_34B','Reason_for_Non-Payment_73R','Disposal_of_Reimbursement_Claim_73S','Reimbursing_Banks_Charges_71D','Sender_to_Receiver_Information_72Z']
    lista_valor_744= [':20:',':21:',':52A:',':52D:',':21A:',':31C:',':34A:',':34B:',':73R:',':73S:',':71D:',':72Z:']
    
    lista_747= ['Documentary_Credit_Number_20','Reimbursing_Banks_Reference_21','Date_of_the_Original_Authorisation_to_Reimburse_30','New_Date_of_Expiry_31E','Increase_of_Documentary_Credit_Amount_32B','Decrease_of_Documentary_Credit_Amount_33B','New_Documentary_Credit_Amount_After_Amendment_34B','Percentage_Credit_Amount_Tolerance_39A','Additional_Amounts_Covered_39C','Sender_to_Receiver_Information_72Z','Narrative_77']
    lista_valor_747= [':20:',':21:',':30:',':31E:',':32B:',':33B:',':34B:',':39A:',':39C:',':72Z:',':77:']
    
    lista_750= ['Senders_Reference_20','Related_Reference_21','Principal_Amount_32B','Additional_Amount_33B','Charges_to_be_Deducted_71D','Charges_to_be_Added_73A','Total_Amount_to_be_Paid_34B','Account_With_Bank_57A','Account_With_Bank_57B','Account_With_Bank_57D','Sender_to_Receiver_Information_72Z','Discrepancies_77J']
    lista_valor_750= [':20:',':21:',':32B:',':33B:',':71D:',':73A:',':34B:',':57A:',':57B:',':57D:',':72Z:',':77J:']
    
    lista_752= ['Documentary_Credit_Number_20','Presenting_Banks_Reference_21','Further_Identification_23','Date_of_Advice_of_Discrepancy_or_Mailing_30','Total_Amount_Advised_32B','Charges_Deducted_71D','Net_Amount_33A','Net_Amount_33B','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D','Sender_to_Receiver_Information_72Z','Narrative_79Z']
    lista_valor_752= [':20:',':21:',':23:',':30:',':32B:',':71D:',':33A:',':33B:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':72Z:',':79Z:']
    
    lista_754= ['Senders_Reference_20','Related_Reference_21','Principal_Amount_Paid/Accepted/Negotiated_32A','Principal_Amount_Paid/Accepted/Negotiated_32B','Additional_Amounts_33B','Charges_Deducted_71D','Charges_Added_73A','Total_Amount_Claimed_34A','Total_Amount_Claimed_34B','Reimbursing_Bank_53A','Reimbursing_Bank_53B','Reimbursing_Bank_53D','Account_With_Bank_57A','Account_With_Bank_57B','Account_With_Bank_57D','Beneficiary_Bank_58A','Beneficiary_Bank_58D','Sender_to_Receiver_Information_72Z','Narrative_77']
    lista_valor_754= [':20:',':21:',':32A:',':32B:',':33B:',':71D:',':73A:',':34A:',':34B:',':53A:',':53B:',':53D:',':57A:',':57B:',':57D:',':58A:',':58D:',':72Z:',':77:']
    
    lista_756= ['Senders_Reference_20','Presenting_Banks_Reference_21','Total_Amount_Claimed_32B','Amount_Reimbursed_or_Paid_33A','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D','Sender_to_Receiver_Information_72Z','Narrative_79Z']
    lista_valor_756= [':20:',':21:',':32B:',':33A:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':72Z:',':79Z:']
    
    lista_759= ['Sequence_of_Total_27','Transaction_Reference_Number_20','Related_Reference_Number_21','Form_of_Undertaking_22D','Undertaking_Number_23','Issuer_52A','Issuer_52D','Function_of_Message_23H','Narrative_45D','File_Identification_23X']
    lista_valor_759= [':27:',':20:',':21:',':22D:',':23:',':52A:',':52D:',':23H:',':45D:',':23X:']
    
    lista_760= ['New_Sequence_15A','Sequence_of_Total_27','Purpose_of_Message_22A','Sender_to_Receiver_Information_72Z','File_Identification_23X','New_Sequence_15B','Undertaking_Number_20','Date_of_Issue_30','Form_of_Undertaking_22D','Applicable_Rules_40C','Expiry_Type_23B','Date_of_Expiry_31E','Expiry_Condition/Event_35G','Applicant_50','Obligor/Instructing_Party_51','Issuer_52A','Issuer_52D','Beneficiary_59','Beneficiary_59A','Advising_Bank_56A','Advising_Bank_56D','Advising_Bank_Reference_23',
    'Advise_Through_Bank_57A','Advise_Through_Bank_57D','Undertaking_Amount_32B','Supplementary_Information_About_Amount_39F','Available_With_41F','Available_With_41G','Charges_71D','Document_and_Presentation_Instructions_45C','Undertaking_Terms_and_Conditions_77U','Confirmation_Instructions_49','Requested_Confirmation_Party_58A','Requested_Confirmation_Party_58D','Governing_Law/Jurisdiction_44J','Automatic_Extension_Period_23F','Automatic_Extension_Non-Extension_Notification_78',
    'Automatic_Extension_Notification_Period_26E','Automatic_Extension_Final_Expiry_Date_31S','Demand_Indicator_48B','Transfer_Indicator_48D','Transfer_Conditions_39E','Underlying_Transaction_Details_45L','Delivery_of_Original_Undertaking_24E','Delivery_To/Collection_By_24G','New_Sequence_15C','Requested_Date_of_Issue_31C','Type_of_Undertaking_22K','Requested_Local_Undertaking_Terms_and_Conditions_77L','Standard_Wording_Required_22Y','Standard_Wording_Requested_Language_40D']
    lista_valor_760= [':15A:',':27:',':22A:',':72Z:',':23X:',':15B:',':20:',':30:',':22D:',':40C:',':23B:',':31E:',':35G:',':50:',':51:',':52A:',':52D:',':59:',':59A:',':56A:',':56D:',':23:',':57A:',':57D:',':32B:',':39F:',':41F:',':41G:',':71D:',':45C:',':77U:',':49:',':58A:',':58D:',':44J:',':23F:',':78:',':26E:',':31S:',':48B:',':48D:',':39E:',':45L:',':24E:',':24G:',':15C:',':31C:',':22K:',':77L:',':22Y:',':40D:']
    
    lista_761= ['Sequence_of_Total_27','Undertaking_Number_20','Issuer_52A','Issuer_52D','Undertaking_Terms_and_Conditions_77U','Requested_Local_Undertaking_Terms_and_Conditions_77L']
    lista_valor_761= [':27:',':20:',':52A:',':52D:',':77U:',':77L:']
    
    lista_765= ['Transaction_Reference_Number_20','Related_Reference_21','Beneficiary_Reference_Number_23','Issuer_52A','Issuer_52D','Beneficiary_59','Beneficiary_59A','Date_of_Demand_31L','Demand_Type_22G','Demand_Amount_32B','Additional_Amount_Information_78','Demand_Statement_49A','Presentation_Completion_Details_77','Requested_New_Date_of_Expiry_31E','Requested_New_Date_of_Expiry_of_Local_Undertaking_31R','Intermediary_56A','Intermediary_56B','Intermediary_56D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Sender_to_Receiver_Information_72Z','File_Identification_23X']
    lista_valor_765= [':20:',':21:',':23:',':52A:',':52D:',':59:',':59A:',':31L:',':22G:',':32B:',':78:',':49A:',':77:',':31E:',':31R:',':56A:',':56B:',':56D:',':57A:',':57B:',':57D:',':72Z:',':23X:']
    
    lista_767= ['New_Sequence_15A','Sequence_of_Total_27','Related_Reference_21','Purpose_of_Message_22A','Cancellation_Request_23S','Sender_to_Receiver_Information_72Z','File_Identification_23X','New_Sequence_15B','Undertaking_Number_20','Number_of_Amendment_26E','Date_of_Amendment_30','Issuer_52A','Issuer_52D','Advising_Bank_Reference_23','Increase_of_Undertaking_Amount_32B','Decrease_of_Undertaking_Amount_33B','Expiry_Type_23B','Date_of_Expiry_31E','Expiry_Condition/Event_35G','Beneficiary_59','Beneficiary_59A','Other_Amendments_to_Undertaking_77U','Delivery_of_Amendment_To_Undertaking_24E','Delivery_To/Collection_By_24G','New_Sequence_15C','Increase_of_Local_Undertaking_Amount_32B',
    'Decrease_of_Local_Undertaking_Amount_33B','Other_Amendments_to_Local_Undertaking_77L','Delivery_of_Amendment_To_Local_Undertaking_24E']
    lista_valor_767= [':15A:',':27:',':21:',':22A:',':23S:',':72Z:',':23X:',':15B:',':20:',':26E:',':30:',':52A:',':52D:',':23:',':32B:',':33B:',':23B:',':31E:',':35G:',':59:',':59A:',':77U:',':24E:',':24G:',':15C:',':32B:',':33B:',':77L:',':24E:']
    
    lista_768= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Date_of_Message_Being_Acknowledged_30','Amount_of_Charges_32B','Amount_of_Charges_32D','Account_With_Bank_57A','Account_With_Bank_57B','Account_With_Bank_57D','Details_of_Charges_71D','Sender_to_Receiver_Information_72Z','File_Identification_23X']
    lista_valor_768= [':20:',':21:',':25:',':30:',':32B:',':32D:',':57A:',':57B:',':57D:',':71D:',':72Z:',':23X:']
    
    lista_769= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Date_of_Reduction_or_Release_30','Amount_of_Charges_32B','Amount_of_Charges_32D','Amount_Reduced_or_Released_33B','Amount_Outstanding_34B','Amount_Specification_39C','Account_With_Bank_57A','Account_With_Bank_57B','Account_With_Bank_57D','Details_of_Charges_71D','Sender_to_Receiver_Information_72Z','File_Identification_23X']
    lista_valor_769= [':20:',':21:',':25:',':30:',':32B:',':32D:',':33B:',':34B:',':39C:',':57A:',':57B:',':57D:',':71D:',':72Z:',':23X:']
    
    lista_775= ['Sequence_of_Total_27','Undertaking_Number_20','Related_Reference_21','Number_of_Amendment_26E','Issuer_52A','Issuer_52D','Other_Amendments_to_Undertaking_77U','Other_Amendments_to_Local_Undertaking_77L']
    lista_valor_775= [':27:',':20:',':21:',':26E:',':52A:',':52D:',':77U:',':77L:']
    
    lista_785= ['Undertaking_Number_20','Related_Reference_21','Issuer_52A','Issuer_52D','Date_of_Issue_31C','Beneficiary_59','Beneficiary_59A','Advising_Bank_56A','Advising_Bank_56D','Advise_Through_Bank_57A','Advise_Through_Bank_57D','Final_Date_of_Expiry_31E','Sender_to_Receiver_Information_72Z','File_Identification_23X']
    lista_valor_785= [':20:',':21:',':52A:',':52D:',':31C:',':59:',':59A:',':56A:',':56D:',':57A:',':57D:',':31E:',':72Z:',':23X:']
    
    lista_786= ['Undertaking_Number_20','Beneficiary_Reference_Number_21','Issuer_52A','Issuer_52D','Demand_Submission_Date_30','Demand_Amount_32B','Reason_for_Refusal_77J','Disposal_of_Documents_77B','Sender_to_Receiver_Information_72Z','File_Identification_23X']
    lista_valor_786= [':20:',':21:',':52A:',':52D:',':30:',':32B:',':77J:',':77B:',':72Z:',':23X:']
    
    lista_787= ['Transaction_Reference_Number_20','Related_Reference_21','Issuer_52A','Issuer_52D','Number_of_Amendment_26E','Amendment_Status_23R','Sender_to_Receiver_Information_72Z','File_Identification_23X']
    lista_valor_787= [':20:',':21:',':52A:',':52D:',':26E:',':23R:',':72Z:',':23X:']
    
    lista_790= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Value_Date_Currency_Code_Amount_32C','Value_Date_Currency_Code_Amount_32D','Ordering_Institution_52A','Ordering_Institution_52D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_790= [':20:',':21:',':25:',':32C:',':32D:',':52A:',':52D:',':71B:',':72:']
    
    lista_791= ['Transaction_Reference_Number_20','Related_Reference_21','Currency_Code_Amount_32B','Ordering_Institution_52A','Ordering_Institution_52D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_791= [':20:',':21:',':32B:',':52A:',':52D:',':57A:',':57B:',':57D:',':71B:',':72:']
    
    lista_792= ['Transaction_Reference_Number_20','Related_Reference_21','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_79']
    lista_valor_792= [':20:',':21:',':11S:',':79:']
    
    lista_795= ['Transaction_Reference_Number_20','Related_Reference_21','Queries_75','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Message_to_Which_the_Query_Relates_79']
    lista_valor_795= [':20:',':21:',':75:',':77A:',':11R:',':11S:',':79:']
    
    lista_796= ['Transaction_Reference_Number_20','Related_Reference_21','Answers_76','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_to_Which_the_Answer_Relates_79']
    lista_valor_796= [':20:',':21:',':76:',':77A:',':11R:',':11S:',':79:']
    
    lista_798= ['Transaction_Reference_Number_20','Sub-Message_Type_12','Proprietary_Message_77E']
    lista_valor_798= [':20:',':12:',':77E:']
    
    lista_799= ['Transaction_Reference_Number_20','Related_Reference_21','Narrative_79']
    lista_valor_799= [':20:',':21:',':79:']
    
    lista_801= ['Transaction_Reference_Number_20','Settlement_No/Page_No_28','Selling_Agent_51A','Selling_Agent_51C','Purchase_Agreement_ID_23','Sales_Date_30','Serial_Numbers_26A','Face_Amount_33B','Additional_Amounts_73','Selling_Agent_Amount_34B','Selling_Agents_No_of_PAs_16A']
    lista_valor_801= [':20:',':28:',':51A:',':51C:',':23:',':30:',':26A:',':33B:',':73:',':34B:',':16A:']
    
    lista_802= ['Transaction_Reference_Number_20','Settlement_No/Page_No_28','Settlement_Amount_32A','Remitting_Agent_52A','Remitting_Agent_52D','Senders_Correspondent_53A','Senders_Correspondent_53B','Senders_Correspondent_53D','Receivers_Correspondent_54A','Receivers_Correspondent_54B','Receivers_Correspondent_54D','Sender_to_Receiver_Information_72']
    lista_valor_802= [':20:',':28:',':32A:',':52A:',':52D:',':53A:',':53B:',':53D:',':54A:',':54B:',':54D:',':72:']
    
    lista_900= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Account_Identification_25P','Date/Time_Indication_13D','Value_Date_Currency_Code_Amount_32A','Ordering_Institution_52A','Ordering_Institution_52D','Sender_to_Receiver_Information_72']
    lista_valor_900= [':20:',':21:',':25:',':25P:',':13D:',':32A:',':52A:',':52D:',':72:']
    
    lista_910= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Account_Identification_25P','Date/Time_Indication_13D','Value_Date_Currency_Code_Amount_32A','Ordering_Customer_50A','Ordering_Customer_50F','Ordering_Customer_50K','Ordering_Institution_52A','Ordering_Institution_52D','Intermediary_56A','Intermediary_56D','Sender_to_Receiver_Information_72']
    lista_valor_910= [':20:',':21:',':25:',':25P:',':13D:',':32A:',':50A:',':50F:',':50K:',':52A:',':52D:',':56A:',':56D:',':72:']
    
    lista_920= ['Transaction_Reference_Number_20','Message_Requested_12','Account_Identification_25','Debit/Debit_and_Credit_Floor_Limit_Indicator_34F','Credit_Floor_Limit_Indicator_34F']
    lista_valor_920= [':20:',':12:',':25:',':34F:',':34F:']
    
    lista_935= ['Transaction_Reference_Number_20','Further_Identification_23','Account_Identification_25','Effective_Date_of_New_Rate_30','New_Interest_Rate_37H','Sender_to_Receiver_Information_72']
    lista_valor_935= [':20:',':23:',':25:',':30:',':37H:',':72:']
    
    lista_940= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Account_Identification_25P','Statement_Number/Sequence_Number_28C','Opening_Balance_60F','Opening_Balance_60M','Statement_Line_61','Information_to_Account_Owner_86','Closing_Balance_Booked_Funds_62F','Closing_Balance_Booked_Funds_62M','Closing_Available_Balance_Available_Funds_64','Forward_Available_Balance_65']
    lista_valor_940= [':20:',':21:',':25:',':25P:',':28C:',':60F:',':60M:',':61:',':86:',':62F:',':62M:',':64:',':65:']
    
    lista_941= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Account_Identification_25P','Statement_Number/Sequence_Number_28','Date/Time_Indication_13D','Opening_Balance_60F','Number_and_Sum_of_Entries_90D','Number_and_Sum_of_Entries_90C','Book_Balance_62F','Closing_Available_Balance_Available_Funds_64','Forward_Available_Balance_65','Information_to_Account_Owner_86']
    lista_valor_941= [':20:',':21:',':25:',':25P:',':28:',':13D:',':60F:',':90D:',':90C:',':62F:',':64:',':65:',':86:']
    
    lista_942= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Account_Identification_25P','Statement_Number/Sequence_Number_28C','Debit/Debit_and_Credit_Floor_Limit_Indicator_34F','Credit_Floor_Limit_Indicator_34F','Date/Time_Indication_13D','Statement_Line_61','Information_to_Account_Owner_86','Number_and_Sum_of_Entries_90D','Number_and_Sum_of_Entries_90C']
    lista_valor_942= [':20:',':21:',':25:',':25P:',':28C:',':34F:',':34F:',':13D:',':61:',':86:',':90D:',':90C:']
    
    lista_950= ['Transaction_Reference_Number_20','Account_Identification_25','Statement_Number/Sequence_Number_28C','Opening_Balance_60F','Opening_Balance_60M','Statement_Line_61','Closing_Balance_Booked_Funds_62F','Closing_Balance_Booked_Funds_62M','Closing_Available_Balance_Available_Funds_64']
    lista_valor_950= [':20:',':25:',':28C:',':60F:',':60M:',':61:',':62F:',':62M:',':64:']
    
    lista_970= ['Transaction_Reference_Number_20','Account_Identification_25','Statement_Number/Sequence_Number_28C','Opening_Balance_60F','Opening_Balance_60M','Statement_Line_61','Closing_Balance_62F','Closing_Balance_62M','Closing_Available_Balance_64']
    lista_valor_970= [':20:',':25:',':28C:',':60F:',':60M:',':61:',':62F:',':62M:',':64:']
    
    lista_971= ['Transaction_Reference_Number_20','Account_Identification_25','Final_Closing_Balance_62F']
    lista_valor_971= [':20:',':25:',':62F:']
    
    lista_972= ['Transaction_Reference_Number_20','Account_Identification_25','Statement_Number/Sequence_Number_28C','Opening_Balance_60F','Opening_Balance_60M','Statement_Line_61','Closing_Balance_62F','Closing_Balance_62M','Closing_Available_Balance_64']
    lista_valor_972= [':20:',':25:',':28C:',':60F:',':60M:',':61:',':62F:',':62M:',':64:']
    
    lista_973= ['Transaction_Reference_Number_20','Message_Type_12','Account_Identification_25']
    lista_valor_973= [':20:',':12:',':25:']
    
    lista_985= ['Transaction_Reference_Number_20','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Enquired_Party_59','Queries_75']
    lista_valor_985= [':20:',':57A:',':57B:',':57D:',':59:',':75:']
    
    lista_986= ['Transaction_Reference_Number_20','Related_Reference_21','Enquired_Party_59','Narrative_79']
    lista_valor_986= [':20:',':21:',':59:',':79:']
    
    lista_990= ['Transaction_Reference_Number_20','Related_Reference_21','Account_Identification_25','Value_Date_Currency_Code_Amount_32C','Value_Date_Currency_Code_Amount_32D','Ordering_Institution_52A','Ordering_Institution_52D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_990= [':20:',':21:',':25:',':32C:',':32D:',':52A:',':52D:',':71B:',':72:']
    
    lista_991= ['Transaction_Reference_Number_20','Related_Reference_21','Currency_Code_Amount_32B','Ordering_Institution_52A','Ordering_Institution_52D','Account_With_Institution_57A','Account_With_Institution_57B','Account_With_Institution_57D','Details_of_Charges_71B','Sender_to_Receiver_Information_72']
    lista_valor_991= [':20:',':21:',':32B:',':52A:',':52D:',':57A:',':57B:',':57D:',':71B:',':72:']
    
    lista_992= ['Transaction_Reference_Number_20','Related_Reference_21','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_79']
    lista_valor_992= [':20:',':21:',':11S:',':79:']
    
    lista_995= ['Transaction_Reference_Number_20','Related_Reference_21','Queries_75','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Message_to_Which_the_Query_Relates_79']
    lista_valor_995= [':20:',':21:',':75:',':77A:',':11R:',':11S:',':79:']
    
    lista_996= ['Transaction_Reference_Number_20','Related_Reference_21','Answers_76','Narrative_77A','MT_and_Date_of_the_Original_Message_11R','MT_and_Date_of_the_Original_Message_11S','Narrative_Description_of_the_Original_Message_to_Which_the_Answer_Relates_79']
    lista_valor_996= [':20:',':21:',':76:',':77A:',':11R:',':11S:',':79:']
    
    lista_998= ['Transaction_Reference_Number_20','Sub-Message_Type_12','Proprietary_Message_77E']
    lista_valor_998= [':20:',':12:',':77E:']
    
    lista_999= ['Transaction_Reference_Number_20','Related_Reference_21','Narrative_79']
    lista_valor_999= [':20:',':21:',':79:']

In [0]:
#General_properties

#listas de payments and trade Message type
lista_payments_message_type=['101', '103','103_REMIT','103_STP','192', '195', '196', '199', '202','202_COV', '205','205_COV','292', '295', '296', '299']
lista_trade_message_type=['700', '701', '707', '708', '710', '711', '720', '721', '734', '792', '795', '796', '799', '760', '761']
lista_payments_MX=['pacs.008','pacs.004','pacs.002','pacs.009','camt.029','camt.056']
lista_trade_MX=[]
# lista de las columnas payment
lista_payments = ['Filename','Message_Content','Creation_date', 'Direction', 'Sender_InstructingAgent_Requestor', 'Receiver_InstructedAgent_Responder', 'OwnBIC8', 'Own_Country', 'Role', 'Format', 'Message_type', 'InstructionId', 'End2EndId', 'UETR', 'Related_Message_Id', 'Transaction_Reference','Interbank_Settled_Date','Interbank_Settled_Currency', 'Interbank_Settled_Amount', 'Instructed_Currency','Instructed_Amount','Ordering_Customer_Account','Ordering_Customer_Name','Ordering_Customer_Id','Ordering_Customer_Address','Ordering_Customer_Address_Country','Ordering_Customer_Residence_Country','Ordering_Customer_Date_Of_Birth','Ordering_Customer_Place_Of_Birth','Ordering_Customer_ID_Number','Ordering_Customer_National_ID_Number','Ordering_Customer_Id_Additional_Information','Ordering_Institution','Ordering_Institution_Name','Ordering_Institution_Address', "Senders_Correspondent", "Senders_Correspondent_Name_and_Address", "Receivers_Correspondent", "Receivers_Correspondent_Name_and_Address",'Third_Reimbursement_Institution','Third_Reimbursement_Institution_Name_and_Address','Intermediary_Institution','Beneficiary_Institution','Beneficiary_Institution_Name','Beneficiary_Institution_Address','Account_With_Institution','Account_With_Institution_Address','Account_With_Institution_Name_and_Address','Beneficiary_Account','Beneficiary_Name','Beneficiary_Id','Beneficiary_Address','Beneficiary_Address_Country','Beneficiary_Residence_Country','Beneficiary_Additional_Information','Remittance_Information', 'Narrative','Sender_to_Receiver_Information','Clearance_System']
# lista de las columnas trade
lista_trade= ['Filename','Message_Content','Creation_date','Direction', 'Sender_InstructingAgent_Requestor', 'Receiver_InstructedAgent_Responder','OwnBIC8','Own_Country','Role','Format','Message_type','Type_Purpose','Senders_Reference','Receivers_Reference','Issuing_Banks_Reference','Issue_Date','Expiry_Date','Expiry_Place','Applicant_Name','Applicant_Address','Applicant_Bank','Applicant_Bank_Name_and_Address','Instructing_Party','Issuing_Bank','Issuing_Bank_Name','Issuing_Bank_Address','Beneficiary_Account','Beneficiary_Name','Beneficiary_Address','Currency','Amount','Available_Bank','Available_Bank_Name','Available_Bank_Address','Drawee','Drawee_Name','Drawee_Address','Place_Dispatch_Receipt','PortLoading_AirportDeparture','PortDischarge_AirportDestination','Place_Destination_Delivery','Latest_Date_Shipment','Description_Goods','Documents_Required','Additional_Conditions','Requested_Confirmation_Party','Requested_Confirmation_Party_Name','Requested_Confirmation_Party_Address','Reimbursing_Bank','Reimbursing_Bank_Name','Reimbursing_Bank_Address','Instructions','Advising_Bank','Advising_Bank_Name','Advising_Bank_Address','Advise_Through_Bank','Advise_Through_Bank_Name_and_Address','Sender_to_Receiver_Information','Undertaking_Terms','Transaction_Details','Local_Type_Purpose','Local_Applicant_Name','Local_Applicant_Address','Local_Beneficiary_Account','Local_Beneficiary_Name','Local_Beneficiary_Address','Local_Currency','Local_Amount','Local_Undertaking_Terms','Local_Transaction_Details','Narrative']

In [0]:
# Funciones iniciales
import os 
import xml.etree.ElementTree as ET
from datetime import datetime
import importlib
import re


##########################################################################################################
## Funcion 1: FUNCIONES PARA SABER SI EL MENSAJE ES MT O MX
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage
##########################################################################################################

def isMT(ruta_al_archivo):
    if parseMT(ruta_al_archivo)[1].startswith('{1:'):
        return True
    else:
        return False

def isMX(ruta_al_archivo):
    try:
        tree = ET.parse(ruta_al_archivo)
        root = tree.getroot()
        return True
    except:
        return False

#############################################################################################################################################
## Funcion 2: FUNCIÓN PARA APLICAR LOS "COMPLEX_TAGS"
## ------------------------------------------------------------
## Estos son los MT que tienen tags que hay que descomponer, por ejemplo: "USD1099" esto habría que separarlo en "USD" Y "1099"
## Se aplica en la funcion ProcessMessage y utiliza todos los "MT_{}_COMPLEX_TAGS"
#############################################################################################################################################

def process_tag(contenido, data):
    tag2 = contenido["tag2"]
    number = tag2[1:4]
    module_name = "MT_{}_COMPLEX_TAGS".format(number)
    try:
        module = importlib.import_module(module_name)
        function = getattr(module, "complex_tags")  # Ajusta el nombre de la función según sea necesario
        result = function(data)
        return result
    except ImportError:
        return data

#############################################################################################################################################
## Funcion 3: FUNCIÓN PARA DETECTAR SI ES COV, REMIT O STP el mensaje
## ------------------------------------------------------------
## Se aplica en la funcion ProcessMessage
#############################################################################################################################################

def modify_swift_message_type(input_dict):
    COV_205=['Ordering_Customer_50A','Ordering_Customer_50F','Ordering_Customer_50K','Intermediary_Institution_56A','Intermediary_Institution_56C','Intermediary_Institution_56D','Account_With_Institution_57C','Beneficiary_Customer_59','Beneficiary_Customer_59A','Beneficiary_Customer_59F','Remittance_Information_70','Currency/Instructed_Amount_33B']
    COV_202=['Ordering_Customer_50A','Ordering_Customer_50F','Ordering_Customer_50K','Intermediary_Institution_56A','Intermediary_Institution_56C','Intermediary_Institution_56D','Account_With_Institution_57C','Beneficiary_Customer_59','Beneficiary_Customer_59A','Beneficiary_Customer_59F','Remittance_Information_70','Currency/Instructed_Amount_33B']
    for key, value in input_dict.items():
        if 'SWIFT_Message_Type' in value and 'Envelope_Contents_77T' in value:
            if value['SWIFT_Message_Type'] == '103':
                value['SWIFT_Message_Type'] = '103_REMIT'
        if 'Tag119_Validation_Flag' in value and value['Tag119_Validation_Flag'] == '119:REMIT':
                if value['SWIFT_Message_Type'] == '103':
                    value['SWIFT_Message_Type'] = '103_REMIT'
        if 'Tag119_Validation_Flag' in value and value['Tag119_Validation_Flag'] == '119:STP':
                if value['SWIFT_Message_Type'] == '103':
                    value['SWIFT_Message_Type'] = '103_STP'
                if value['SWIFT_Message_Type'] == '102':
                    value['SWIFT_Message_Type'] = '102_STP'
        if any(key in COV_202 for key in value):
            if value['SWIFT_Message_Type'] == '202':
                value['SWIFT_Message_Type'] = '202_COV'
        if any(key in COV_205 for key in value):
            if value['SWIFT_Message_Type'] == '205':
                value['SWIFT_Message_Type'] = '205_COV'
    return input_dict

PARSER_LOGGER=SWIFTParserLogger.getSWIFTParserLogger('INFO')

#############################################################################################################################################
## Funcion 4: ES LA FUNCIÓN QUE PROCESA ARCHIVOS MT y MX LOS DEVUELVE EN FORMA DE DICCIONARIO.
#############################################################################################################################################

def ProcessMessage(carpeta,filename,date):
    #cosas que necesitamos para los mt
    contenido_tags_parseMT={}
    resultados_tag1_MT={}
    resultados_tag2_MT={}
    resultados_tag3_MT={}
    resultados_tag4_MT={}
    resultados_tag5_MT={}
    ALL_TAGS={}
    # cosas que necesitamos para los mx
    atributos_mx = {}
    resultados_MX={}
    PARSER_LOGGER.debug("PROCESANDO: "+ filename)
    ruta_al_archivo = os.path.join(carpeta, filename)
    if isMT(ruta_al_archivo):
        PARSER_LOGGER.debug("Es un archivo MT: "+ filename)
        contenido_tags = parseMT(ruta_al_archivo)[0]
        contenido=parseMT(ruta_al_archivo)[0]
        PARSER_LOGGER.debug(contenido)
        contenido_tags_parseMT[os.path.basename(ruta_al_archivo)] = contenido_tags  # Almacena el resultado
        contenido_tag1 = parseMTTag1(contenido)
        contenido_tag1['Message_Content']= parseMT(ruta_al_archivo)[1]
        contenido_tag1['Extraction_Date']= str(datetime.now())
        contenido_tag1['Payment_Date']=date
        contenido_tag1['Format']='MT'
        PARSER_LOGGER.debug(contenido_tag1)
        resultados_tag1_MT[os.path.basename(ruta_al_archivo)] = contenido_tag1  # Almacena el resultado
        contenido_tag2 = parseMTTag2(contenido)
        PARSER_LOGGER.debug(contenido_tag2)
        resultados_tag2_MT[os.path.basename(ruta_al_archivo)] = contenido_tag2  # Almacena el resultado
        contenido_tag3 = parseMTTag3(contenido)
        PARSER_LOGGER.debug(contenido_tag3)
        resultados_tag3_MT[os.path.basename(ruta_al_archivo)] = contenido_tag3  # Almacena el resultado
        mapeo = selectMT(contenido)[2]
        lista_subcategorias_valor = selectMT(contenido)[1]
        contenido_tag4 = parseMTTag4(contenido,mapeo,lista_subcategorias_valor)[0]
        contenido_tag4_adaptado = reemplazar_subcategorias(contenido_tag4,mapeo)
        PARSER_LOGGER.debug(contenido_tag4_adaptado)
        resultados_tag4_MT[os.path.basename(ruta_al_archivo)] = contenido_tag4_adaptado  # Almacena el resultado
        resultados_tag4_MT= process_tag(contenido,resultados_tag4_MT)
        contenido_tag5 = parseMTTag5(contenido)
        PARSER_LOGGER.debug(contenido_tag5)
        resultados_tag5_MT[os.path.basename(ruta_al_archivo)] = contenido_tag5  # Almacena el resultado
        ALL_TAGS= join_all_tags(resultados_tag1_MT,resultados_tag2_MT,resultados_tag3_MT,resultados_tag4_MT,resultados_tag5_MT)
        ALL_TAGS= modify_swift_message_type(ALL_TAGS)

    elif isMX(ruta_al_archivo):
        PARSER_LOGGER.debug("Es un archivo MX: "+ filename)
        tree = ET.parse(ruta_al_archivo)
        root = tree.getroot()
        atributos_mx_finales= extract_values_with_tags_and_attributes(root)
        atributos_mx_finales= flatten_lists(atributos_mx_finales)
        atributos_mx_finales['Message_Content']= obtener_contenido(root)
        atributos_mx_finales['Extraction_Date']= str(datetime.now())
        atributos_mx_finales['Payment_Date']= date
        atributos_mx_finales['Format']='MX'
        PARSER_LOGGER.debug(atributos_mx_finales)
        resultados_MX[os.path.basename(ruta_al_archivo)] = atributos_mx_finales
    else:
        PARSER_LOGGER.error("ERROR NO ES UN ARCHIVO MT O MX: "+ filename)

    return ALL_TAGS, resultados_MX

#############################################################################################################################################
## Funcion 5: ES LA FUNCIÓN QUE SE OCUPA DE SEPARAR LOS ARCHIVOS UNKNOWN PARA PODER ANALIZARLOS.
#############################################################################################################################################
def separar_secciones(carpeta):
    try:
        # Leer el contenido del archivo
        with open(carpeta, 'r', encoding='utf-8') as f:
            contenido = f.read()
        # Encontrar la sección AppHdr
        match_apphdr = re.search(r'<AppHdr[^>]*>.*?</AppHdr>', contenido, re.DOTALL)
        if match_apphdr:
            apphdr_xml = match_apphdr.group(0)
            if apphdr_xml is not None:
                nombre_archivo = carpeta.replace(".unknown", "_AppHdr.xml")
                root = ET.fromstring(apphdr_xml)
                tree = ET.ElementTree(root)
                tree.write(nombre_archivo, encoding='utf-8', xml_declaration=True)
                print(f"Contenido guardado en '{nombre_archivo}'.")

        # Encontrar la sección Document
        match_document = re.search(r'<Document[^>]*>.*?</Document>', contenido, re.DOTALL)
        if match_document:
            document_xml = match_document.group(0)
            if document_xml is not None:
                nombre_archivo = carpeta.replace(".unknown", "_Document.xml")
                root = ET.fromstring(document_xml)
                tree = ET.ElementTree(root)
                tree.write(nombre_archivo, encoding='utf-8', xml_declaration=True)
                print(f"Contenido guardado en '{nombre_archivo}'.")

    except Exception as e:
        print(f'Error al procesar {carpeta}: {e}')